# Chapter 6: Always Blocks and Processes

## Introduction

Always blocks are fundamental constructs in SystemVerilog that describe how hardware behaves over time. They define processes that execute continuously during simulation and represent different types of hardware structures. SystemVerilog provides three specialized always blocks that make design intent clearer and help avoid common modeling mistakes.

## Types of Always Blocks

SystemVerilog introduces three specialized always blocks:

- `always_comb` - For combinational logic
- `always_ff` - For sequential logic (flip-flops)
- `always_latch` - For latches

These replace the generic `always` block from Verilog and provide better checking and clearer intent.

### always_comb for Combinational Logic

The `always_comb` block is used to model combinational logic where outputs change immediately when inputs change.

#### Basic Syntax

```systemverilog
always_comb begin
    // Combinational logic statements
end
```

#### Key Features

- Automatically sensitive to all inputs (no sensitivity list needed)
- Executes immediately when any input changes
- Should not contain clocked logic or memory elements
- Helps catch incomplete sensitivity lists

#### Examples

##### Simple Multiplexer
```systemverilog
module mux2to1 (
    input  logic sel,
    input  logic a, b,
    output logic y
);

always_comb begin
    if (sel)
        y = b;
    else
        y = a;
end

endmodule
```

##### ALU Example
```systemverilog
module simple_alu (
    input  logic [3:0] a, b,
    input  logic [1:0] op,
    output logic [3:0] result,
    output logic       zero
);

always_comb begin
    case (op)
        2'b00: result = a + b;    // Add
        2'b01: result = a - b;    // Subtract
        2'b10: result = a & b;    // AND
        2'b11: result = a | b;    // OR
        default: result = 4'b0;
    endcase
    
    zero = (result == 4'b0);
end

endmodule
```

##### Priority Encoder
```systemverilog
module priority_encoder (
    input  logic [7:0] req,
    output logic [2:0] grant,
    output logic       valid
);

always_comb begin
    valid = |req;  // OR reduction - true if any bit set
    
    if (req[7])      grant = 3'd7;
    else if (req[6]) grant = 3'd6;
    else if (req[5]) grant = 3'd5;
    else if (req[4]) grant = 3'd4;
    else if (req[3]) grant = 3'd3;
    else if (req[2]) grant = 3'd2;
    else if (req[1]) grant = 3'd1;
    else if (req[0]) grant = 3'd0;
    else             grant = 3'd0;
end

endmodule
```

### always_ff for Sequential Logic

The `always_ff` block is used to model sequential logic elements like flip-flops and registers.

#### Basic Syntax

```systemverilog
always_ff @(posedge clk) begin
    // Sequential logic statements
end

// With reset
always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
        // Reset logic
    end else begin
        // Normal clocked logic
    end
end
```

#### Key Features

- Must have a clocking event in sensitivity list
- Models memory elements (flip-flops, registers)
- Can include asynchronous reset/set
- Should use non-blocking assignments (<=) for clocked logic

#### Examples

##### Simple D Flip-Flop
```systemverilog
module dff (
    input  logic clk,
    input  logic rst_n,
    input  logic d,
    output logic q
);

always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n)
        q <= 1'b0;
    else
        q <= d;
end

endmodule
```

##### Counter with Enable
```systemverilog
module counter (
    input  logic       clk,
    input  logic       rst_n,
    input  logic       enable,
    input  logic       load,
    input  logic [7:0] load_value,
    output logic [7:0] count
);

always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
        count <= 8'b0;
    end else if (load) begin
        count <= load_value;
    end else if (enable) begin
        count <= count + 1;
    end
    // If neither load nor enable, count maintains its value
end

endmodule
```

##### Shift Register
```systemverilog
module shift_register #(
    parameter WIDTH = 8
)(
    input  logic             clk,
    input  logic             rst_n,
    input  logic             shift_en,
    input  logic             serial_in,
    output logic             serial_out,
    output logic [WIDTH-1:0] parallel_out
);

logic [WIDTH-1:0] shift_reg;

always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
        shift_reg <= '0;
    end else if (shift_en) begin
        shift_reg <= {shift_reg[WIDTH-2:0], serial_in};
    end
end

assign serial_out = shift_reg[WIDTH-1];
assign parallel_out = shift_reg;

endmodule
```

##### State Machine Example
```systemverilog
typedef enum logic [1:0] {
    IDLE = 2'b00,
    ACTIVE = 2'b01,
    WAIT = 2'b10,
    DONE = 2'b11
} state_t;

module fsm (
    input  logic   clk,
    input  logic   rst_n,
    input  logic   start,
    input  logic   ready,
    output logic   busy,
    output logic   complete
);

state_t current_state, next_state;

// State register
always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n)
        current_state <= IDLE;
    else
        current_state <= next_state;
end

// Next state logic (combinational)
always_comb begin
    case (current_state)
        IDLE: begin
            if (start)
                next_state = ACTIVE;
            else
                next_state = IDLE;
        end
        
        ACTIVE: begin
            if (ready)
                next_state = WAIT;
            else
                next_state = ACTIVE;
        end
        
        WAIT: begin
            next_state = DONE;
        end
        
        DONE: begin
            next_state = IDLE;
        end
        
        default: next_state = IDLE;
    endcase
end

// Output logic (combinational)
always_comb begin
    busy = (current_state != IDLE) && (current_state != DONE);
    complete = (current_state == DONE);
end

endmodule
```

### always_latch for Latches

The `always_latch` block is used to model transparent latches, though latches are generally discouraged in synchronous design.

#### Basic Syntax

```systemverilog
always_latch begin
    // Latch logic statements
end
```

#### Key Features

- Models level-sensitive storage elements
- Should be avoided in most synchronous designs
- Can cause timing issues and make verification difficult
- Sometimes used for specific analog or mixed-signal applications

#### Example

```systemverilog
module d_latch (
    input  logic enable,
    input  logic d,
    output logic q
);

always_latch begin
    if (enable)
        q = d;
    // When enable is low, q retains its value (latch behavior)
end

endmodule
```

### Blocking vs. Non-Blocking Assignments

Understanding the difference between blocking (=) and non-blocking (<=) assignments is crucial for proper hardware modeling.

#### Blocking Assignments (=)

- Execute immediately in sequence
- Used in combinational logic (`always_comb`)
- Model wire-like behavior
- Can create race conditions if misused

#### Non-Blocking Assignments (<=)

- Scheduled to execute at end of time step
- Used in sequential logic (`always_ff`)
- Model register-like behavior
- Prevent race conditions in clocked logic

#### Examples Comparing Both

##### Combinational Logic - Use Blocking (=)
```systemverilog
// Correct - using blocking assignments
always_comb begin
    temp = a & b;
    y = temp | c;
end

// Incorrect - non-blocking in combinational logic
always_comb begin
    temp <= a & b;  // Wrong!
    y <= temp | c;  // Wrong! - temp not updated yet
end
```

##### Sequential Logic - Use Non-Blocking (<=)
```systemverilog
// Correct - using non-blocking assignments
always_ff @(posedge clk) begin
    q1 <= d;
    q2 <= q1;  // Creates shift register
end

// Incorrect - blocking in sequential logic
always_ff @(posedge clk) begin
    q1 = d;
    q2 = q1;   // Both update to 'd' simultaneously - not a shift register!
end
```

### Best Practices Summary

1. **Combinational logic (`always_comb`)**: Use blocking assignments (=)
2. **Sequential logic (`always_ff`)**: Use non-blocking assignments (<=)
3. **Mixed assignments**: Never mix blocking and non-blocking in the same always block

### Race Conditions and Common Pitfalls

#### Race Condition Example

```systemverilog
// Problematic code - race condition
module race_example (
    input  logic clk,
    input  logic d,
    output logic q1, q2
);

// Two separate always blocks updating at same time
always_ff @(posedge clk) begin
    q1 <= d;
end

always_ff @(posedge clk) begin
    q2 <= q1;  // Race condition! Order of execution matters
end

endmodule
```

```systemverilog
// Better approach - combine into one always block
always_ff @(posedge clk) begin
    q1 <= d;
    q2 <= q1;  // Now guaranteed to work correctly
end
```

#### Incomplete Sensitivity Lists (Verilog issue fixed by always_comb)

```systemverilog
// Old Verilog style - error prone
always @(a, b) begin  // Forgot 'c' in sensitivity list!
    y = a & b;
    z = y | c;        // 'z' won't update when 'c' changes
end

// SystemVerilog solution - automatic sensitivity
always_comb begin
    y = a & b;
    z = y | c;        // Automatically sensitive to a, b, and c
end
```

#### Process Control and Advanced Topics

#### Multiple Clock Domains

```systemverilog
module dual_clock_design (
    input  logic clk1, clk2,
    input  logic rst_n,
    input  logic data_in,
    output logic data_out
);

logic ff1, ff2;

// Clock domain 1
always_ff @(posedge clk1 or negedge rst_n) begin
    if (!rst_n)
        ff1 <= 1'b0;
    else
        ff1 <= data_in;
end

// Clock domain 2
always_ff @(posedge clk2 or negedge rst_n) begin
    if (!rst_n)
        ff2 <= 1'b0;
    else
        ff2 <= ff1;  // Potential metastability issue!
end

assign data_out = ff2;

endmodule
```

#### Generate Blocks with Always Blocks

```systemverilog
module parameterized_register #(
    parameter WIDTH = 8,
    parameter STAGES = 4
)(
    input  logic             clk,
    input  logic             rst_n,
    input  logic [WIDTH-1:0] data_in,
    output logic [WIDTH-1:0] data_out
);

logic [WIDTH-1:0] pipe_reg [STAGES-1:0];

genvar i;
generate
    for (i = 0; i < STAGES; i++) begin : pipe_stage
        always_ff @(posedge clk or negedge rst_n) begin
            if (!rst_n) begin
                pipe_reg[i] <= '0;
            end else begin
                if (i == 0)
                    pipe_reg[i] <= data_in;
                else
                    pipe_reg[i] <= pipe_reg[i-1];
            end
        end
    end
endgenerate

assign data_out = pipe_reg[STAGES-1];

endmodule
```

### Best Practices and Guidelines

#### Design Guidelines

1. **Use appropriate always block types**:
   - `always_comb` for combinational logic
   - `always_ff` for sequential logic
   - Avoid `always_latch` unless specifically needed

2. **Assignment types**:
   - Blocking (=) in `always_comb`
   - Non-blocking (<=) in `always_ff`
   - Never mix both types in the same block

3. **Reset strategy**:
   - Use asynchronous reset for critical paths
   - Consider synchronous reset for better timing
   - Initialize all registers consistently

4. **Avoid common mistakes**:
   - Incomplete case statements
   - Inferred latches from incomplete if-else chains
   - Multiple drivers to same signal

#### Code Style Example

```systemverilog
module good_style_example (
    input  logic       clk,
    input  logic       rst_n,
    input  logic [7:0] data_in,
    input  logic       enable,
    output logic [7:0] data_out,
    output logic       valid
);

// Sequential logic - use always_ff with non-blocking
always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
        data_out <= 8'b0;
        valid    <= 1'b0;
    end else if (enable) begin
        data_out <= data_in;
        valid    <= 1'b1;
    end else begin
        valid    <= 1'b0;
        // data_out retains its value
    end
end

endmodule
```

### Summary

Always blocks are the cornerstone of behavioral modeling in SystemVerilog. The three specialized types (`always_comb`, `always_ff`, `always_latch`) provide clear intent and help prevent common coding errors. Understanding when to use blocking vs. non-blocking assignments is crucial for creating hardware that behaves as intended. Following the best practices outlined in this chapter will lead to more reliable, synthesizable, and maintainable code.


### Key Takeaways

- Use `always_comb` for combinational logic with blocking assignments (=)
- Use `always_ff` for sequential logic with non-blocking assignments (<=)
- Avoid `always_latch` unless specifically required
- Be consistent with reset strategies
- Understand race conditions and how to avoid them
- Always consider the hardware implications of your code

# SystemVerilog Always Blocks and Processes - Example Index

## always_comb for Combinational Logic

#### Basic Logic Gates
Simple AND, OR, XOR, NAND gates demonstrating basic combinational behavior

In [29]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_6_examples/example_1__basic_logic_gates/"
files = ["basic_logic_gates.sv", "basic_logic_gates_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// basic_logic_gates.sv
module basic_logic_gates (
    input  logic a,        // First input
    input  logic b,        // Second input
    output logic and_out,  // AND gate output
    output logic or_out,   // OR gate output  
    output logic xor_out,  // XOR gate output
    output logic nand_out  // NAND gate output
);

    // Combinational logic assignments
    assign and_out  = a & b;   // AND gate
    assign or_out   = a | b;   // OR gate
    assign xor_out  = a ^ b;   // XOR gate
    assign nand_out = ~(a & b); // NAND gate (inverted AND)

    // Display gate outputs whenever inputs change
    always_comb begin
        $display(
            "Inputs: a=%b, b=%b | Outputs: AND=%b, OR=%b, XOR=%b, NAND=%b", 
            a, b, and_out, or_out, xor_out, nand_out);
    end

endmodule
```

```systemverilog
// basic_logic_gates_testbench.sv
module logic_gates_testbench;

    // Test signals
    logic test_a, test_b;
    logic result_and, result_or, result_xor, result_nand;

    // Instantiate the design under test
    basic_logic_gates DUT (
        .a(test_a),
        .b(test_b),
        .and_out(result_and),
        .or_out(result_or),
        .xor_out(result_xor),
        .nand_out(result_nand)
    );

    initial begin
        // Setup waveform dumping
        $dumpfile("basic_logic_gates_testbench.vcd");
        $dumpvars(0, basic_logic_gates_testbench);
        
        $display("=== Basic Logic Gates Test ===");
        $display();
        
        // Test all input combinations (truth table)
        $display("Testing all 2-input combinations:");
        
        // Test case 1: 00
        test_a = 1'b0; test_b = 1'b0;
        #10;
        
        // Test case 2: 01  
        test_a = 1'b0; test_b = 1'b1;
        #10;
        
        // Test case 3: 10
        test_a = 1'b1; test_b = 1'b0;
        #10;
        
        // Test case 4: 11
        test_a = 1'b1; test_b = 1'b1;
        #10;
        
        $display();
        $display("=== Truth Table Complete ===");
        $display("AND: outputs 1 only when both inputs are 1");
        $display("OR:  outputs 1 when at least one input is 1");
        $display("XOR: outputs 1 when inputs are different");
        $display("NAND: outputs 0 only when both inputs are 1 (inverted AND)");
        
        $finish;
    end

endmodule
```

Verilator Simulation Output:
Inputs: a=0, b=1 | Outputs: AND=0, OR=1, XOR=1, NAND=1
Inputs: a=1, b=0 | Outputs: AND=0, OR=1, XOR=1, NAND=1
Inputs: a=1, b=0 | Outputs: AND=0, OR=1, XOR=1, NAND=1
Inputs: a=1, b=1 | Outputs: AND=1, OR=1, XOR=0, NAND=0
Inputs: a=1, b=1 | Outputs: AND=1, OR=1, XOR=0, NAND=0

=== Truth Table Complete ===
AND: outputs 1 only when both inputs are 1
OR:  outputs 1 when at least one input is 1
XOR: outputs 1 when inputs are different
NAND: outputs 0 only when both inputs are 1 (inverted AND)
- basic_logic_gates_testbench.sv:52: Verilog $finish
Inputs: a=1, b=1 | Outputs: AND=1, OR=1, XOR=0, NAND=0
Process finished with return code: 0
Removing Chapter_6_examples/example_1__basic_logic_gates/obj_dir directory...
Chapter_6_examples/example_1__basic_logic_gates/obj_dir removed successfully.


0

#### Decoder (3-to-8)
Address decoder that converts 3-bit input to 8 one-hot outputs with enable

In [30]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_6_examples/example_2__address_decoder/"
files = ["address_decoder_3to8.sv", "address_decoder_3to8_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// address_decoder_3to8.sv
module address_decoder_3to8 (
    input  logic [2:0] address,    // 3-bit address input (0-7)
    input  logic       enable,     // Active high enable signal
    output logic [7:0] decoded_out // 8-bit one-hot output
);

    // Address decoder logic using case statement
    always_comb begin
        if (enable) begin
            case (address)
                3'b000: decoded_out = 8'b00000001; // Address 0 -> bit 0
                3'b001: decoded_out = 8'b00000010; // Address 1 -> bit 1
                3'b010: decoded_out = 8'b00000100; // Address 2 -> bit 2
                3'b011: decoded_out = 8'b00001000; // Address 3 -> bit 3
                3'b100: decoded_out = 8'b00010000; // Address 4 -> bit 4
                3'b101: decoded_out = 8'b00100000; // Address 5 -> bit 5
                3'b110: decoded_out = 8'b01000000; // Address 6 -> bit 6
                3'b111: decoded_out = 8'b10000000; // Address 7 -> bit 7
                default: decoded_out = 8'b00000000; // Should never happen
            endcase
        end else begin
            // All outputs disabled when enable is low
            decoded_out = 8'b00000000; 
        end
    end

    // Display decoder operation
    always_comb begin
        if (enable) begin
            $display(
                "Address Decoder: Enable=%b, Address=%3b (%0d) -> %8b", 
                enable, address, address, decoded_out);
        end else begin
            $display(
                "Address Decoder: Enable=%b -> All outputs disabled", 
                enable);
        end
    end

endmodule
```

```systemverilog
// address_decoder_3to8_testbench.sv
module address_decoder_testbench;

    // Test signals
    logic [2:0] test_address;
    logic       test_enable;
    logic [7:0] decoded_outputs;

    // Instantiate the design under test
    address_decoder_3to8 DUT (
        .address(test_address),
        .enable(test_enable),
        .decoded_out(decoded_outputs)
    );

    initial begin
        // Setup waveform dumping
        $dumpfile("address_decoder_3to8_testbench.vcd");
        $dumpvars(0, address_decoder_3to8_testbench);
        
        $display("=== 3-to-8 Address Decoder Test ===");
        $display();
        
        // Test 1: Decoder disabled (enable = 0)
        $display("Test 1: Testing with Enable = 0 (decoder disabled)");
        test_enable = 1'b0;
        
        for (int i = 0; i < 8; i++) begin
            test_address = i[2:0];
            #10;
        end
        
        $display();
        
        // Test 2: Decoder enabled, test all addresses
        $display("Test 2: Testing with Enable = 1 (decoder enabled)");
        test_enable = 1'b1;
        
        for (int i = 0; i < 8; i++) begin
            test_address = i[2:0];
            #10;
            
            // Verify one-hot encoding
            if ($countones(decoded_outputs) == 1) begin
                $display("Address %0d correctly decoded to one-hot", i);
            end else begin
                $display("ERROR: Address %0d not one-hot output!", i);
            end
        end
        
        $display();
        
        // Test 3: Enable/disable transitions
        $display("Test 3: Testing enable/disable transitions");
        test_address = 3'b101; // Address 5
        
        $display("Setting address to 5, then toggling enable...");
        test_enable = 1'b1;
        #10;
        test_enable = 1'b0;
        #10;
        test_enable = 1'b1;
        #10;
        
        $display();
        $display("=== Address Decoder Test Complete ===");
        $display("One-hot encoding: Only one output bit high per address");
        $display("Enable control: All outputs 0 when enable is low");
        $display("Address range: 3-bit input covers addresses 0-7");
        
        $finish;
    end

    // Monitor for any unexpected behavior
    always @(decoded_outputs) begin
        if (test_enable && ($countones(decoded_outputs) != 1)) begin
            $display("WARNING: Non one-hot output detected when enabled!");
        end
    end

endmodule
```

Verilator Simulation Output:
=== 3-to-8 Address Decoder Test ===

Test 1: Testing with Enable = 0 (decoder disabled)
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled

Test 2: Testing with Enable = 1 (decoder enabled)
Address Decoder: Enable=1, Address=000 (0) -> 00000001
Address Decoder: Enable=1, 

0

#### Encoder (8-to-3 Binary)
Priority encoder that converts 8-bit input to 3-bit binary output

In [31]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_6_examples/example_3__priority_encoder/"
files = ["priority_encoder_8to3.sv", "priority_encoder_8to3_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// priority_encoder_8to3.sv
module priority_encoder_8to3 (
    input  logic [7:0] data_in,     // 8-bit input data
    output logic [2:0] encoded_out, // 3-bit encoded output
    output logic       valid_out    // Valid signal (any input active)
);

    // Priority encoder logic - highest bit wins
    always_comb begin
        if (data_in[7]) begin
            encoded_out = 3'b111;       // Input 7 has highest priority
            valid_out = 1'b1;
        end else if (data_in[6]) begin
            encoded_out = 3'b110;       // Input 6
            valid_out = 1'b1;
        end else if (data_in[5]) begin
            encoded_out = 3'b101;       // Input 5
            valid_out = 1'b1;
        end else if (data_in[4]) begin
            encoded_out = 3'b100;       // Input 4
            valid_out = 1'b1;
        end else if (data_in[3]) begin
            encoded_out = 3'b011;       // Input 3
            valid_out = 1'b1;
        end else if (data_in[2]) begin
            encoded_out = 3'b010;       // Input 2
            valid_out = 1'b1;
        end else if (data_in[1]) begin
            encoded_out = 3'b001;       // Input 1
            valid_out = 1'b1;
        end else if (data_in[0]) begin
            encoded_out = 3'b000;       // Input 0 (lowest priority)
            valid_out = 1'b1;
        end else begin
            encoded_out = 3'b000;       // No inputs active
            valid_out = 1'b0;
        end
    end

    // Display encoder operation
    always_comb begin
        if (valid_out) begin
            $display(
                "Priority Encoder: Input=%8b -> Encoded=%3b (%0d), Valid=%b", 
                data_in, encoded_out, encoded_out, valid_out);
            $display();
        end else begin
            $display(
                "Priority Encoder: Input=%8b -> No valid input (Valid=%b)", 
                data_in, valid_out);
            $display();
        end
    end

endmodule
```

```systemverilog
// priority_encoder_8to3_testbench.sv
module priority_encoder_8to3_testbench;

    // Test signals
    logic [7:0] test_data_in;
    logic [2:0] encoded_result;
    logic       valid_result;

    // Instantiate the design under test
    priority_encoder_8to3 DUT (
        .data_in(test_data_in),
        .encoded_out(encoded_result),
        .valid_out(valid_result)
    );

    initial begin
        // Setup waveform dumping
        $dumpfile("priority_encoder_8to3_testbench.vcd");
        $dumpvars(0, priority_encoder_8to3_testbench);
        
        $display("=== 8-to-3 Priority Encoder Test ===");
        $display();
        
        // Test 1: No inputs active
        $display("Test 1: No inputs active");
        test_data_in = 8'b00000000;
        #10;
        
        $display();
        
        // Test 2: Single bit tests (test each priority level)
        $display("Test 2: Testing each input individually");
        for (int i = 0; i < 8; i++) begin
            test_data_in = 8'b00000001 << i;
            #10;
            
            // Verify correct encoding
            if (valid_result && (encoded_result == i[2:0])) begin
                $display("Input bit %0d correctly encoded", i);
            end else begin
                $display("ERROR: Input bit %0d encoding failed!", i);
            end
        end
        
        $display();
        
        // Test 3: Priority tests (multiple bits active)
        $display("Test 3: Testing priority behavior");
        
        // High priority wins over low priority
        test_data_in = 8'b10000001; // Bit 7 and 0 active
        #10;
        if (encoded_result == 3'b111) begin
            $display("Bit 7 correctly wins over bit 0");
        end else begin
            $display("ERROR: Priority not working correctly!");
        end
        
        // Middle priority test
        test_data_in = 8'b00101010; // Bits 5, 3, 1 active
        #10;
        if (encoded_result == 3'b101) begin
            $display("Bit 5 correctly wins over bits 3,1");
        end else begin
            $display("ERROR: Middle priority test failed!");
        end
        
        // All bits active (highest should win)
        test_data_in = 8'b11111111;
        #10;
        if (encoded_result == 3'b111) begin
            $display("Bit 7 correctly wins when all bits active");
        end else begin
            $display("ERROR: All bits test failed!");
        end
        
        $display();
        
        // Test 4: Random priority patterns
        $display("Test 4: Testing various priority patterns");
        
        test_data_in = 8'b01100000; // Bits 6,5 active -> 6 wins
        #10;
        
        test_data_in = 8'b00011100; // Bits 4,3,2 active -> 4 wins
        #10;
        
        test_data_in = 8'b00000110; // Bits 2,1 active -> 2 wins
        #10;
        
        $display();
        $display("=== Priority Encoder Test Complete ===");
        $display("Priority order: Bit 7 (highest) down to Bit 0 (lowest)");
        $display("Valid signal: Indicates at least one input is active");
        $display("Encoding: Highest priority active input -> binary output");
        $display();
        
        $finish;
    end

    // Monitor for unexpected behavior
    always @(test_data_in) begin
        if (test_data_in != 8'b00000000 && !valid_result) begin
            $display("WARNING: Valid should be high when inputs are active!");
        end
        if (test_data_in == 8'b00000000 && valid_result) begin
            $display("WARNING: Valid should be low when no inputs active!");
        end
    end

endmodule
```

Verilator Simulation Output:
=== 8-to-3 Priority Encoder Test ===

Test 1: No inputs active
Priority Encoder: Input=00000000 -> No valid input (Valid=0)


Test 2: Testing each input individually
Priority Encoder: Input=00000001 -> Encoded=000 (0), Valid=1

Priority Encoder: Input=00000001 -> Encoded=000 (0), Valid=1

Input bit 0 correctly encoded
Priority Encoder: Input=00000010 -> Encoded=001 (1), Valid=1

Priority Encoder: Input=00000010 -> Encoded=001 (1), Valid=1

Input bit 1 correctly encoded
Priority Encoder: Input=00000100 -> Encoded=010 (2), Valid=1

Priority Encoder: Input=00000100 -> Encoded=010 (2), Valid=1

Input bit 2 correctly encoded
Priority Encoder: Input=00001000 -> Encoded=011 (3), Valid=1

Priority Encoder: Input=00001000 -> Encoded=011 (3), Valid=1

Input bit 3 correctly encoded
Priority Encoder: Input=00010000 -> Encoded=100 (4), Valid=1

Priority Encoder: Input=00010000 -> Encoded=100 (4), Valid=1

Input bit 4 correctly encoded
Priority Encoder: Input=00100000 ->

0

#### Barrel Shifter
Combinational shifter that can shift/rotate data by variable amounts

In [32]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_6_examples/example_4__combinational_shifter/"
files = ["combinational_shifter.sv", "combinational_shifter_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// combinational_shifter.sv
module combinational_shifter (
    input  logic [7:0] data_in,     // 8-bit input data
    input  logic [2:0] shift_amt,   // 3-bit shift amount (0-7)
    input  logic [1:0] shift_op,    // 2-bit operation select
    output logic [7:0] data_out     // 8-bit shifted output
);

    // Operation encoding:
    // 00: Logical shift left (LSL)
    // 01: Logical shift right (LSR)
    // 10: Arithmetic shift right (ASR)
    // 11: Rotate right (ROR)

    always_comb begin
        case (shift_op)
            2'b00: begin // Logical shift left
                data_out = data_in << shift_amt;
            end
            
            2'b01: begin // Logical shift right
                data_out = data_in >> shift_amt;
            end
            
            2'b10: begin // Arithmetic shift right (sign extend)
                data_out = $signed(data_in) >>> shift_amt;
            end
            
            2'b11: begin // Rotate right
                case (shift_amt)
                    3'd0: data_out = data_in;
                    3'd1: data_out = {data_in[0], data_in[7:1]};
                    3'd2: data_out = {data_in[1:0], data_in[7:2]};
                    3'd3: data_out = {data_in[2:0], data_in[7:3]};
                    3'd4: data_out = {data_in[3:0], data_in[7:4]};
                    3'd5: data_out = {data_in[4:0], data_in[7:5]};
                    3'd6: data_out = {data_in[5:0], data_in[7:6]};
                    3'd7: data_out = {data_in[6:0], data_in[7]};
                endcase
            end
            
            default: data_out = data_in; // Pass through
        endcase
    end

    // Display shifter operation
    always_comb begin
        case (shift_op)
            2'b00: $display("Shifter: LSL %8b << %0d = %8b", 
                           data_in, shift_amt, data_out);
            2'b01: $display("Shifter: LSR %8b >> %0d = %8b", 
                           data_in, shift_amt, data_out);
            2'b10: $display("Shifter: ASR %8b >>> %0d = %8b", 
                           data_in, shift_amt, data_out);
            2'b11: $display("Shifter: ROR %8b rotate %0d = %8b", 
                           data_in, shift_amt, data_out);
        endcase
    end

endmodule
```

```systemverilog
// combinational_shifter_testbench.sv
module shifter_testbench;

    // Test signals
    logic [7:0] test_data;
    logic [2:0] test_shift_amt;
    logic [1:0] test_shift_op;
    logic [7:0] shifted_result;

    // Operation names for display
    string op_names[4] = '{"LSL", "LSR", "ASR", "ROR"};

    // Instantiate the design under test
    combinational_shifter DUT (
        .data_in(test_data),
        .shift_amt(test_shift_amt),
        .shift_op(test_shift_op),
        .data_out(shifted_result)
    );

    initial begin
        // Setup waveform dumping
        $dumpfile("shifter_testbench.vcd");
        $dumpvars(0, shifter_testbench);
        
        $display("=== Combinational Shifter Test ===");
        $display();
        
        // Test 1: Logical shift left (LSL)
        $display("Test 1: Logical Shift Left (LSL)");
        test_data = 8'b10110011;
        test_shift_op = 2'b00; // LSL
        
        for (int i = 0; i <= 4; i++) begin
            test_shift_amt = i[2:0];
            #10;
        end
        
        $display();
        
        // Test 2: Logical shift right (LSR)
        $display("Test 2: Logical Shift Right (LSR)");
        test_data = 8'b10110011;
        test_shift_op = 2'b01; // LSR
        
        for (int i = 0; i <= 4; i++) begin
            test_shift_amt = i[2:0];
            #10;
        end
        
        $display();
        
        // Test 3: Arithmetic shift right (ASR)
        $display("Test 3: Arithmetic Shift Right (ASR) - Negative number");
        test_data = 8'b11010110; // Negative in 2's complement
        test_shift_op = 2'b10; // ASR
        
        for (int i = 0; i <= 4; i++) begin
            test_shift_amt = i[2:0];
            #10;
        end
        
        $display();
        
        // Test 4: ASR with positive number
        $display("Test 4: Arithmetic Shift Right (ASR) - Positive number");
        test_data = 8'b01010110; // Positive number
        test_shift_op = 2'b10; // ASR
        
        for (int i = 0; i <= 3; i++) begin
            test_shift_amt = i[2:0];
            #10;
        end
        
        $display();
        
        // Test 5: Rotate right (ROR)
        $display("Test 5: Rotate Right (ROR)");
        test_data = 8'b10110001;
        test_shift_op = 2'b11; // ROR
        
        for (int i = 0; i <= 7; i++) begin
            test_shift_amt = i[2:0];
            #10;
        end
        
        $display();
        
        // Test 6: Edge cases
        $display("Test 6: Edge cases");
        
        // All zeros
        test_data = 8'b00000000;
        test_shift_op = 2'b00; // LSL
        test_shift_amt = 3'd3;
        #10;
        
        // All ones
        test_data = 8'b11111111;
        test_shift_op = 2'b01; // LSR
        test_shift_amt = 3'd2;
        #10;
        
        // Maximum shift
        test_data = 8'b10101010;
        test_shift_op = 2'b00; // LSL
        test_shift_amt = 3'd7;
        #10;
        
        $display();
        $display("=== Shifter Test Complete ===");
        $display("LSL: Logical shift left - zeros fill from right");
        $display("LSR: Logical shift right - zeros fill from left");
        $display("ASR: Arithmetic shift right - sign bit extends");
        $display("ROR: Rotate right - bits wrap around");
        $display();
        
        $finish;
    end

    // Function to verify LSL operation
    function logic [7:0] expected_lsl(logic [7:0] data, logic [2:0] amt);
        return data << amt;
    endfunction

    // Function to verify LSR operation
    function logic [7:0] expected_lsr(logic [7:0] data, logic [2:0] amt);
        return data >> amt;
    endfunction

    // Function to verify ASR operation
    function logic [7:0] expected_asr(logic [7:0] data, logic [2:0] amt);
        return $signed(data) >>> amt;
    endfunction

    // Function to verify ROR operation
    function logic [7:0] expected_ror(logic [7:0] data, logic [2:0] amt);
        case (amt)
            3'd0: return data;
            3'd1: return {data[0], data[7:1]};
            3'd2: return {data[1:0], data[7:2]};
            3'd3: return {data[2:0], data[7:3]};
            3'd4: return {data[3:0], data[7:4]};
            3'd5: return {data[4:0], data[7:5]};
            3'd6: return {data[5:0], data[7:6]};
            3'd7: return {data[6:0], data[7]};
        endcase
    endfunction

    // Monitor and verify all operations
    always @(shifted_result) begin
        #1; // Small delay for display
        case (test_shift_op)
            2'b00: begin // LSL verification
                if (shifted_result == expected_lsl(test_data, test_shift_amt)) begin
                    $display("LSL verification passed");
                end else begin
                    $display(
                        "LSL verification failed! Expected: %8b, Got: %8b", 
                        expected_lsl(test_data, test_shift_amt),
                        shifted_result);
                end
            end
            2'b01: begin // LSR verification
                if (shifted_result == expected_lsr(test_data, test_shift_amt)) begin
                    $display("LSR verification passed");
                end else begin
                    $display(
                        "LSR verification failed! Expected: %8b, Got: %8b", 
                        expected_lsr(test_data, test_shift_amt),
                        shifted_result);
                end
            end
            2'b10: begin // ASR verification
                if (shifted_result == expected_asr(test_data, test_shift_amt)) begin
                    $display("ASR verification passed");
                end else begin
                    $display(
                        "ASR verification failed! Expected: %8b, Got: %8b", 
                        expected_asr(test_data, test_shift_amt),
                        shifted_result);
                end
            end
            2'b11: begin // ROR verification
                if (shifted_result == expected_ror(test_data, test_shift_amt)) begin
                    $display("ROR verification passed");
                end else begin
                    $display(
                        "ROR verification failed! Expected: %8b, Got: %8b", 
                        expected_ror(test_data, test_shift_amt),
                        shifted_result);
                end
            end
            default: begin
                $display("Unknown operation: %2b", test_shift_op);
            end
        endcase
    end

endmodule
```

Verilator Simulation Output:
=== Combinational Shifter Test ===

Test 1: Logical Shift Left (LSL)
Shifter: LSL 10110011 << 0 = 10110011
Shifter: LSL 10110011 << 1 = 01100110
Shifter: LSL 10110011 << 1 = 01100110
LSL verification passed
Shifter: LSL 10110011 << 1 = 01100110
Shifter: LSL 10110011 << 1 = 01100110
Shifter: LSL 10110011 << 2 = 11001100
Shifter: LSL 10110011 << 2 = 11001100
LSL verification passed
Shifter: LSL 10110011 << 2 = 11001100
Shifter: LSL 10110011 << 2 = 11001100
Shifter: LSL 10110011 << 3 = 10011000
Shifter: LSL 10110011 << 3 = 10011000
LSL verification passed
Shifter: LSL 10110011 << 3 = 10011000
Shifter: LSL 10110011 << 3 = 10011000
Shifter: LSL 10110011 << 4 = 00110000
Shifter: LSL 10110011 << 4 = 00110000
LSL verification passed
Shifter: LSL 10110011 << 4 = 00110000
Shifter: LSL 10110011 << 4 = 00110000

Test 2: Logical Shift Right (LSR)
Shifter: LSR 10110011 >> 0 = 10110011
Shifter: LSR 10110011 >> 0 = 10110011
LSR verification passed
Shifter: LSR 10110011 >> 

0

#### Seven-Segment Display Driver
BCD to seven-segment decoder for numeric display applications

In [33]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_6_examples/example_5__bcd_seven_segment/"
files = ["bcd_seven_segment.sv", "bcd_seven_segment_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// bcd_seven_segment.sv
// BCD to Seven-Segment Display Decoder
// Converts 4-bit BCD input (0-9) to 7-segment display pattern

module bcd_seven_segment (
    input  logic [3:0] bcd_in,      // 4-bit BCD input (0-9)
    output logic [6:0] segments     // 7-segment output [a,b,c,d,e,f,g]
);

    // Seven-segment display layout:
    //  aaa
    // f   b
    // f   b
    //  ggg
    // e   c
    // e   c
    //  ddd
    
    // segments[6] = a, segments[5] = b, segments[4] = c, segments[3] = d
    // segments[2] = e, segments[1] = f, segments[0] = g
    // Active HIGH (1 = segment ON, 0 = segment OFF)

    always_comb begin
        case (bcd_in)
            4'h0: segments = 7'b1111110; // Display "0" - all except g
            4'h1: segments = 7'b0110000; // Display "1" - b,c only
            4'h2: segments = 7'b1101101; // Display "2" - a,b,d,e,g
            4'h3: segments = 7'b1111001; // Display "3" - a,b,c,d,g
            4'h4: segments = 7'b0110011; // Display "4" - b,c,f,g
            4'h5: segments = 7'b1011011; // Display "5" - a,c,d,f,g
            4'h6: segments = 7'b1011111; // Display "6" - a,c,d,e,f,g
            4'h7: segments = 7'b1110000; // Display "7" - a,b,c
            4'h8: segments = 7'b1111111; // Display "8" - all segments
            4'h9: segments = 7'b1111011; // Display "9" - a,b,c,d,f,g
            default: segments = 7'b0000000; // Blank display for invalid BCD
        endcase
    end

    // Display decoder operation for simulation
    always_comb begin
        case (bcd_in)
            4'h0: $display(
                "BCD Decoder: %h -> '0' (segments: %7b)", bcd_in, segments);
            4'h1: $display(
                "BCD Decoder: %h -> '1' (segments: %7b)", bcd_in, segments);
            4'h2: $display(
                "BCD Decoder: %h -> '2' (segments: %7b)", bcd_in, segments);
            4'h3: $display(
                "BCD Decoder: %h -> '3' (segments: %7b)", bcd_in, segments);
            4'h4: $display(
                "BCD Decoder: %h -> '4' (segments: %7b)", bcd_in, segments);
            4'h5: $display(
                "BCD Decoder: %h -> '5' (segments: %7b)", bcd_in, segments);
            4'h6: $display(
                "BCD Decoder: %h -> '6' (segments: %7b)", bcd_in, segments);
            4'h7: $display(
                "BCD Decoder: %h -> '7' (segments: %7b)", bcd_in, segments);
            4'h8: $display(
                "BCD Decoder: %h -> '8' (segments: %7b)", bcd_in, segments);
            4'h9: $display(
                "BCD Decoder: %h -> '9' (segments: %7b)", bcd_in, segments);
            default: $display(
                "BCD Decoder: %h -> BLANK (invalid BCD)", bcd_in);
        endcase
    end

endmodule
```

```systemverilog
// bcd_seven_segment_testbench.sv
module bcd_testbench;

    // Test signals
    logic [3:0] test_bcd;
    logic [6:0] display_segments;

    // Expected patterns for verification
    logic [6:0] expected_patterns [0:9] = '{
        7'b1111110, // 0
        7'b0110000, // 1
        7'b1101101, // 2
        7'b1111001, // 3
        7'b0110011, // 4
        7'b1011011, // 5
        7'b1011111, // 6
        7'b1110000, // 7
        7'b1111111, // 8
        7'b1111011  // 9
    };

    // Instantiate the design under test
    bcd_seven_segment DUT (
        .bcd_in(test_bcd),
        .segments(display_segments)
    );

    initial begin
        // Setup waveform dumping
        $dumpfile("bcd_testbench.vcd");
        $dumpvars(0, bcd_testbench);
        
        $display("=== BCD to Seven-Segment Decoder Test ===");
        $display("Segment order: [a,b,c,d,e,f,g] (MSB to LSB)");
        $display("Display layout:");
        $display("  aaa  ");
        $display(" f   b ");
        $display(" f   b ");
        $display("  ggg  ");
        $display(" e   c ");
        $display(" e   c ");
        $display("  ddd  ");
        $display();
        
        // Test all valid BCD inputs (0-9)
        $display("Testing valid BCD inputs (0-9):");
        for (int i = 0; i <= 9; i++) begin
            test_bcd = i[3:0];
            #10;
            
            // Verify the output
            if (display_segments == expected_patterns[i]) begin
                $display("âœ“ BCD %0d: PASS - Segments: %7b", i, display_segments);
            end else begin
                $display("âœ— BCD %0d: FAIL - Expected: %7b, Got: %7b", 
                        i, expected_patterns[i], display_segments);
            end
        end
        
        $display();
        
        // Test invalid BCD inputs (10-15)
        $display("Testing invalid BCD inputs (A-F):");
        for (int i = 10; i <= 15; i++) begin
            test_bcd = i[3:0];
            #10;
            
            if (display_segments == 7'b0000000) begin
                $display("âœ“ BCD %X: PASS - Blank display (segments: %7b)", i, display_segments);
            end else begin
                $display("âœ— BCD %X: FAIL - Expected blank, Got: %7b", i, display_segments);
            end
        end
        
        $display();
        
        // Visual representation test
        $display("Visual representation of each digit:");
        for (int i = 0; i <= 9; i++) begin
            test_bcd = i[3:0];
            #10;
            $display("Digit %0d:", i);
            display_seven_segment(display_segments);
            $display();
        end
        
        $display("=== BCD Decoder Test Complete ===");
        $finish;
    end

    // Function to display seven-segment pattern visually
    task display_seven_segment(logic [6:0] seg);
        string a, b, c, d, e, f, g;
        
        // Convert segment bits to display characters
        a = seg[6] ? "###" : "   ";
        b = seg[5] ? "#" : " ";
        c = seg[4] ? "#" : " ";
        d = seg[3] ? "###" : "   ";
        e = seg[2] ? "#" : " ";
        f = seg[1] ? "#" : " ";
        g = seg[0] ? "###" : "   ";
        
        // Display the seven-segment pattern
        $display(" %s ", a);
        $display("%s   %s", f, b);
        $display("%s   %s", f, b);
        $display(" %s ", g);
        $display("%s   %s", e, c);
        $display("%s   %s", e, c);
        $display(" %s ", d);
    endtask

    // Monitor for any changes
    always @(display_segments) begin
        #1; // Small delay for clean display
    end

endmodule
```

Verilator Simulation Output:
=== BCD to Seven-Segment Decoder Test ===
Segment order: [a,b,c,d,e,f,g] (MSB to LSB)
Display layout:
  aaa
 f   b
 f   b
  ggg
 e   c
 e   c
  ddd

Testing valid BCD inputs (0-9):
BCD Decoder: 0 -> '0' (segments: 1111110)
âœ“ BCD 0: PASS - Segments: 1111110
BCD Decoder: 1 -> '1' (segments: 0110000)
BCD Decoder: 1 -> '1' (segments: 0110000)
BCD Decoder: 1 -> '1' (segments: 0110000)
BCD Decoder: 1 -> '1' (segments: 0110000)
âœ“ BCD 1: PASS - Segments: 0110000
BCD Decoder: 2 -> '2' (segments: 1101101)
BCD Decoder: 2 -> '2' (segments: 1101101)
BCD Decoder: 2 -> '2' (segments: 1101101)
BCD Decoder: 2 -> '2' (segments: 1101101)
âœ“ BCD 2: PASS - Segments: 1101101
BCD Decoder: 3 -> '3' (segments: 1111001)
BCD Decoder: 3 -> '3' (segments: 1111001)
BCD Decoder: 3 -> '3' (segments: 1111001)
BCD Decoder: 3 -> '3' (segments: 1111001)
âœ“ BCD 3: PASS - Segments: 1111001
BCD Decoder: 4 -> '4' (segments: 0110011)
BCD Decoder: 4 -> '4' (segments: 0110011)
BCD Decoder: 4 -

0

#### Comparator Circuit
Multi-bit magnitude comparator with equal, greater-than, less-than outputs

In [34]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_6_examples/example_6__magnitude_comparator/"
files = ["magnitude_comparator.sv", "magnitude_comparator_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// magnitude_comparator.sv
// Multi-bit Magnitude Comparator
// Compares two N-bit unsigned numbers and generates comparison outputs

module magnitude_comparator #(
    parameter WIDTH = 8  // Bit width of inputs (default 8-bit)
) (
    input  logic [WIDTH-1:0] a_in,    // First number input
    input  logic [WIDTH-1:0] b_in,    // Second number input
    output logic             equal,   // Output: a == b
    output logic             greater, // Output: a > b
    output logic             less     // Output: a < b
);

    // Combinational comparison logic
    always_comb begin
        equal   = (a_in == b_in);
        greater = (a_in > b_in);
        less    = (a_in < b_in);
    end

    // Display comparison results for simulation
    always_comb begin
        if (equal)
            $display("Comparator: %0d == %0d (Equal)", a_in, b_in);
        else if (greater)
            $display("Comparator: %0d > %0d (Greater)", a_in, b_in);
        else if (less)
            $display("Comparator: %0d < %0d (Less)", a_in, b_in);
    end

    // Verification: exactly one output should be high
    always_comb begin
        assert (equal + greater + less == 1) 
        else $error("Comparator error: Multiple or no outputs active!");
    end

endmodule
```

```systemverilog
// magnitude_comparator_testbench.sv
module comparator_testbench;

    // Test parameters
    parameter WIDTH = 8;
    
    // Test signals
    logic [WIDTH-1:0] test_a;
    logic [WIDTH-1:0] test_b;
    logic eq_out, gt_out, lt_out;
    
    // Test counters
    int test_count = 0;
    int pass_count = 0;
    int fail_count = 0;

    // Instantiate the design under test
    magnitude_comparator #(.WIDTH(WIDTH)) DUT (
        .a_in(test_a),
        .b_in(test_b),
        .equal(eq_out),
        .greater(gt_out),
        .less(lt_out)
    );

    initial begin
        // Setup waveform dumping
        $dumpfile("comparator_testbench.vcd");
        $dumpvars(0, comparator_testbench);
        
        $display("=== %0d-bit Magnitude Comparator Test ===", WIDTH);
        $display("Testing A vs B with Equal, Greater, Less outputs");
        $display();
        
        // Test 1: Equal values
        $display("Test 1: Equal Values");
        test_equal_values();
        $display();
        
        // Test 2: A greater than B
        $display("Test 2: A > B Cases");
        test_greater_than();
        $display();
        
        // Test 3: A less than B
        $display("Test 3: A < B Cases");
        test_less_than();
        $display();
        
        // Test 4: Edge cases
        $display("Test 4: Edge Cases");
        test_edge_cases();
        $display();
        
        // Test 5: Random comprehensive test
        $display("Test 5: Random Comprehensive Test");
        test_random_values();
        $display();
        
        // Final summary
        $display("=== Test Summary ===");
        $display("Total tests: %0d", test_count);
        $display("Passed: %0d", pass_count);
        $display("Failed: %0d", fail_count);
        if (fail_count == 0) begin
            $display("ALL TESTS PASSED!");
        end else begin
            $display("%0d TESTS FAILED", fail_count);
        end
        
        $finish;
    end

    // Task to test equal values
    task automatic test_equal_values();
        logic [WIDTH-1:0] test_values [5];
        
        // Initialize test values
        test_values[0] = 0;
        test_values[1] = 1;
        test_values[2] = 15;
        test_values[3] = 128;
        test_values[4] = 255;
        
        foreach (test_values[i]) begin
            test_a = test_values[i];
            test_b = test_values[i];
            #10;
            verify_result(1, 0, 0, "Equal");
        end
    endtask

    // Task to test A > B cases
    task automatic test_greater_than();
        // Test arrays for A > B cases
        logic [WIDTH-1:0] test_a_vals [5] = '{10, 255, 128, 100, 200};
        logic [WIDTH-1:0] test_b_vals [5] = '{5, 254, 127, 50, 199};
        
        for (int i = 0; i < 5; i++) begin
            test_a = test_a_vals[i];
            test_b = test_b_vals[i];
            #10;
            verify_result(0, 1, 0, "Greater");
        end
    endtask

    // Task to test A < B cases
    task automatic test_less_than();
        // Test arrays for A < B cases
        logic [WIDTH-1:0] test_a_vals [5] = '{5, 0, 127, 50, 199};
        logic [WIDTH-1:0] test_b_vals [5] = '{10, 1, 128, 100, 200};
        
        for (int i = 0; i < 5; i++) begin
            test_a = test_a_vals[i];
            test_b = test_b_vals[i];
            #10;
            verify_result(0, 0, 1, "Less");
        end
    endtask

    // Task to test edge cases
    task automatic test_edge_cases();
        // Minimum values
        test_a = 0; test_b = 0;
        #10; verify_result(1, 0, 0, "Min Equal");
        
        // Maximum values
        test_a = {WIDTH{1'b1}}; test_b = {WIDTH{1'b1}};
        #10; verify_result(1, 0, 0, "Max Equal");
        
        // Min vs Max
        test_a = 0; test_b = {WIDTH{1'b1}};
        #10; verify_result(0, 0, 1, "Min < Max");
        
        // Max vs Min
        test_a = {WIDTH{1'b1}}; test_b = 0;
        #10; verify_result(0, 1, 0, "Max > Min");
        
        // Adjacent values
        test_a = 100; test_b = 101;
        #10; verify_result(0, 0, 1, "Adjacent Less");
        
        test_a = 101; test_b = 100;
        #10; verify_result(0, 1, 0, "Adjacent Greater");
    endtask

    // Task for random testing
    task automatic test_random_values();
        logic [31:0] rand_val;
        
        for (int i = 0; i < 20; i++) begin
            // Generate random values with proper bit width
            rand_val = $random();
            test_a = rand_val[WIDTH-1:0];
            rand_val = $random();
            test_b = rand_val[WIDTH-1:0];
            #10;
            
            // Determine expected result
            if (test_a == test_b)
                verify_result(1, 0, 0, "Random Equal");
            else if (test_a > test_b)
                verify_result(0, 1, 0, "Random Greater");
            else
                verify_result(0, 0, 1, "Random Less");
        end
    endtask

    // Verification task
    task automatic verify_result(logic exp_eq, logic exp_gt, logic exp_lt, string test_name);
        test_count++;
        
        if (eq_out == exp_eq && gt_out == exp_gt && lt_out == exp_lt) begin
            pass_count++;
            $display(
                "%s: A=%0d, B=%0d -> EQ=%b GT=%b LT=%b", 
                test_name, test_a, test_b, eq_out, gt_out, lt_out);
        end else begin
            fail_count++;
            $display(
                "%s: A=%0d, B=%0d -> Expected: EQ=%b GT=%b LT=%b, Got: EQ=%b GT=%b LT=%b", 
                test_name, test_a, test_b, exp_eq, exp_gt, exp_lt, eq_out, gt_out, lt_out);
        end
    endtask

    // Monitor for timing analysis
    always @(test_a or test_b) begin
        #1; // Small delay to let outputs settle
        // Check that exactly one output is active
        if ((eq_out + gt_out + lt_out) != 1) begin
            $display(
                "WARNING: Invalid output state at A=%0d, B=%0d",
                test_a, test_b);
        end
    end

    // Performance analysis
    initial begin
        #1000; // Wait for all tests to complete
        $display();
        $display("=== Performance Analysis ===");
        $display("Comparator width: %0d bits", WIDTH);
        $display("Maximum input value: %0d", (2**WIDTH)-1);
        $display("Total possible comparisons: %0d", (2**WIDTH) * (2**WIDTH));
        $display("Tests performed: %0d (%.2f%% coverage)", 
                test_count,
                (real'(test_count) / real'((2**WIDTH) * (2**WIDTH))) * 100.0);
    end

endmodule
```

Verilator Simulation Output:
=== 8-bit Magnitude Comparator Test ===
Testing A vs B with Equal, Greater, Less outputs

Test 1: Equal Values
Comparator: 0 == 0 (Equal)
Comparator: 0 == 0 (Equal)
Comparator: 0 == 0 (Equal)
Equal: A=0, B=0 -> EQ=1 GT=0 LT=0
Comparator: 1 == 1 (Equal)
Comparator: 1 == 1 (Equal)
Comparator: 1 == 1 (Equal)
Comparator: 1 == 1 (Equal)
Equal: A=1, B=1 -> EQ=1 GT=0 LT=0
Comparator: 15 == 15 (Equal)
Comparator: 15 == 15 (Equal)
Comparator: 15 == 15 (Equal)
Comparator: 15 == 15 (Equal)
Equal: A=15, B=15 -> EQ=1 GT=0 LT=0
Comparator: 128 == 128 (Equal)
Comparator: 128 == 128 (Equal)
Comparator: 128 == 128 (Equal)
Comparator: 128 == 128 (Equal)
Equal: A=128, B=128 -> EQ=1 GT=0 LT=0
Comparator: 255 == 255 (Equal)
Comparator: 255 == 255 (Equal)
Comparator: 255 == 255 (Equal)
Comparator: 255 == 255 (Equal)
Equal: A=255, B=255 -> EQ=1 GT=0 LT=0

Test 2: A > B Cases
Comparator: 10 > 5 (Greater)
Comparator: 10 > 5 (Greater)
Comparator: 10 > 5 (Greater)
Comparator: 10 > 5 

0

## always_ff for Sequential Logic

#### Ring Counter
Circular shift register where output feeds back to input

In [35]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_6_examples/example_8__ring_counter/"
files = ["ring_counter_design.sv", "ring_counter_design_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// ring_counter_design.sv
module ring_counter_4bit (
    input  logic clock,
    input  logic reset_n,
    input  logic enable,
    output logic [3:0] ring_output
);

    // 4-bit ring counter register
    logic [3:0] ring_register;
    
    always_ff @(posedge clock or negedge reset_n) begin
        if (!reset_n) begin
            // Initialize with one bit set (start pattern)
            ring_register <= 4'b0001;
        end
        else if (enable) begin
            // Circular shift: MSB feeds back to LSB
            ring_register <= {ring_register[2:0], ring_register[3]};
        end
    end
    
    // Output the ring register state
    assign ring_output = ring_register;
    
    // Display current state for debugging
    always @(posedge clock) begin
        if (enable && reset_n) begin
            $display("Ring Counter: %b", ring_register);
        end
    end

endmodule
```

```systemverilog
// ring_counter_design_testbench.sv
module ring_counter_testbench;
    
    // Testbench signals
    logic clock;
    logic reset_n;
    logic enable;
    logic [3:0] ring_output;
    
    // Instantiate the ring counter design
    ring_counter_4bit RING_COUNTER_INSTANCE (
        .clock(clock),
        .reset_n(reset_n),
        .enable(enable),
        .ring_output(ring_output)
    );
    
    // Clock generation (10ns period)
    initial begin
        clock = 0;
        forever #5 clock = ~clock;
    end
    
    // Test sequence
    initial begin
        // Setup waveform dumping
        $dumpfile("ring_counter_testbench.vcd");
        $dumpvars(0, ring_counter_testbench);
        
        // Display header
        $display();
        $display("=== 4-Bit Ring Counter Test ===");
        $display("Time | Reset | Enable | Output");
        $display("---------------------------------");
        
        // Initialize signals
        reset_n = 0;
        enable = 0;
        
        // Apply reset
        #10;
        reset_n = 1;
        $display(" %3t |   %b   |    %b   | %b", $time, reset_n, enable, ring_output);
        
        // Enable the ring counter and observe the circular shift
        #5;
        enable = 1;
        
        // Run for several clock cycles to see the ring pattern
        repeat(12) begin
            #10;
            $display(" %3t |   %b   |    %b   | %b", $time, reset_n, enable, ring_output);
        end
        
        // Test disable functionality
        $display();
        $display("Testing disable...");
        enable = 0;
        #20;
        $display(" %3t |   %b   |    %b   | %b (disabled)", $time, reset_n, enable, ring_output);
        
        // Re-enable
        enable = 1;
        #20;
        $display(" %3t |   %b   |    %b   | %b (re-enabled)", $time, reset_n, enable, ring_output);
        
        // Test reset during operation
        $display();
        $display("Testing reset during operation...");
        #10;
        reset_n = 0;
        #10;
        reset_n = 1;
        $display(" %3t |   %b   |    %b   | %b (after reset)", $time, reset_n, enable, ring_output);
        
        $display();
        $display("=== Ring Counter Test Complete ===");
        $display();
        
        #20;
        $finish;
    end

endmodule
```

Verilator Simulation Output:

=== 4-Bit Ring Counter Test ===
Time | Reset | Enable | Output
---------------------------------
  10 |   1   |    0   | 0001
Ring Counter: 0001
  25 |   1   |    1   | 0010
Ring Counter: 0010
  35 |   1   |    1   | 0100
Ring Counter: 0100
  45 |   1   |    1   | 1000
Ring Counter: 1000
  55 |   1   |    1   | 0001
Ring Counter: 0001
  65 |   1   |    1   | 0010
Ring Counter: 0010
  75 |   1   |    1   | 0100
Ring Counter: 0100
  85 |   1   |    1   | 1000
Ring Counter: 1000
  95 |   1   |    1   | 0001
Ring Counter: 0001
 105 |   1   |    1   | 0010
Ring Counter: 0010
 115 |   1   |    1   | 0100
Ring Counter: 0100
 125 |   1   |    1   | 1000
Ring Counter: 1000
 135 |   1   |    1   | 0001

Testing disable...
 155 |   1   |    0   | 0001 (disabled)
Ring Counter: 0001
Ring Counter: 0010
 175 |   1   |    1   | 0100 (re-enabled)

Testing reset during operation...
Ring Counter: 0100
 195 |   1   |    1   | 0001 (after reset)

=== Ring Counter Test Complete

0

#### LFSR (Linear Feedback Shift Register)
Pseudo-random number generator using XOR feedback taps

In [36]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_6_examples/example_9__lfsr/"
files = ["lfsr_generator_design.sv", "lfsr_generator_design_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// lfsr_generator_design.sv
module lfsr_4bit_generator (
    input  logic clock,
    input  logic reset_n,
    input  logic enable,
    output logic [3:0] lfsr_output,
    output logic random_bit
);

    // 4-bit LFSR register
    logic [3:0] lfsr_register;
    logic feedback_bit;
    
    // XOR feedback taps for 4-bit LFSR (polynomial: x^4 + x^3 + 1)
    // Taps at positions 4 and 3 (bits 3 and 2 in 0-indexed)
    assign feedback_bit = lfsr_register[3] ^ lfsr_register[2];
    
    always_ff @(posedge clock or negedge reset_n) begin
        if (!reset_n) begin
            // Initialize with non-zero seed (avoid all-zeros state)
            lfsr_register <= 4'b0001;
        end
        else if (enable) begin
            // Shift left and insert feedback bit at LSB
            lfsr_register <= {lfsr_register[2:0], feedback_bit};
        end
    end
    
    // Outputs
    assign lfsr_output = lfsr_register;
    assign random_bit = lfsr_register[3];  // MSB as random output
    
    // Display current state and feedback for debugging
    always @(posedge clock) begin
        if (enable && reset_n) begin
            $display("LFSR: %b, Feedback: %b, Random bit: %b", 
                     lfsr_register, feedback_bit, random_bit);
        end
    end

endmodule
```

```systemverilog
// lfsr_generator_design_testbench.sv
module lfsr_generator_testbench;
    
    // Testbench signals
    logic clock;
    logic reset_n;
    logic enable;
    logic [3:0] lfsr_output;
    logic random_bit;
    
    // Variables for sequence tracking
    logic [3:0] sequence_history [0:15];
    integer cycle_count;
    
    // Instantiate the LFSR generator design
    lfsr_4bit_generator LFSR_GENERATOR_INSTANCE (
        .clock(clock),
        .reset_n(reset_n),
        .enable(enable),
        .lfsr_output(lfsr_output),
        .random_bit(random_bit)
    );
    
    // Clock generation (10ns period)
    initial begin
        clock = 0;
        forever #5 clock = ~clock;
    end
    
    // Test sequence
    initial begin
        // Setup waveform dumping
        $dumpfile("lfsr_generator_testbench.vcd");
        $dumpvars(0, lfsr_generator_testbench);
        
        // Display header
        $display();
        $display("=== 4-Bit LFSR Pseudo-Random Generator Test ===");
        $display("Polynomial: x^4 + x^3 + 1 (taps at positions 4,3)");
        $display();
        $display("Cycle | LFSR State | Random Bit | Feedback");
        $display("------|------------|------------|----------");
        
        // Initialize signals and counters
        reset_n = 0;
        enable = 0;
        cycle_count = 0;
        
        // Apply reset
        #10;
        reset_n = 1;
        
        // Enable the LFSR and observe the pseudo-random sequence
        #5;
        enable = 1;
        
        // Run through complete sequence (2^4 - 1 = 15 states for 4-bit LFSR)
        repeat(16) begin
            #10;
            sequence_history[cycle_count] = lfsr_output;
            $display("  %2d  |    %b    |     %b      |    %b", 
                     cycle_count, lfsr_output, random_bit, 
                     lfsr_output[3] ^ lfsr_output[2]);
            cycle_count++;
        end
        
        // Check if sequence repeats (should return to initial state)
        $display();
        if (lfsr_output == 4'b0001) begin
            $display("âœ“ LFSR sequence completed full cycle (returned to initial state)");
        end else begin
            $display("âœ— LFSR did not complete full cycle");
        end
        
        // Test disable functionality
        $display();
        $display("Testing disable...");
        enable = 0;
        #20;
        $display("LFSR disabled - State frozen at: %b", lfsr_output);
        
        // Re-enable
        enable = 1;
        #20;
        $display("LFSR re-enabled - Continuing from: %b", lfsr_output);
        
        // Test reset during operation
        $display();
        $display("Testing reset during operation...");
        #10;
        reset_n = 0;
        #10;
        reset_n = 1;
        $display("After reset - LFSR state: %b", lfsr_output);
        
        // Show a few more random bits
        $display();
        $display("Generating random bit stream:");
        $write("Random bits: ");
        repeat(8) begin
            #10;
            $write("%b ", random_bit);
        end
        $display();
        
        $display();
        $display("=== LFSR Test Complete ===");
        $display("Maximum sequence length for 4-bit LFSR: %d states", 2**4 - 1);
        $display();
        
        #20;
        $finish;
    end

endmodule
```

Verilator Simulation Output:

=== 4-Bit LFSR Pseudo-Random Generator Test ===
Polynomial: x^4 + x^3 + 1 (taps at positions 4,3)

Cycle | LFSR State | Random Bit | Feedback
------|------------|------------|----------
LFSR: 0001, Feedback: 0, Random bit: 0
   0  |    0010    |     0      |    0
LFSR: 0010, Feedback: 0, Random bit: 0
   1  |    0100    |     0      |    1
LFSR: 0100, Feedback: 1, Random bit: 0
   2  |    1001    |     1      |    1
LFSR: 1001, Feedback: 1, Random bit: 1
   3  |    0011    |     0      |    0
LFSR: 0011, Feedback: 0, Random bit: 0
   4  |    0110    |     0      |    1
LFSR: 0110, Feedback: 1, Random bit: 0
   5  |    1101    |     1      |    0
LFSR: 1101, Feedback: 0, Random bit: 1
   6  |    1010    |     1      |    1
LFSR: 1010, Feedback: 1, Random bit: 1
   7  |    0101    |     0      |    1
LFSR: 0101, Feedback: 1, Random bit: 0
   8  |    1011    |     1      |    1
LFSR: 1011, Feedback: 1, Random bit: 1
   9  |    0111    |     0      |    1
LFSR

0

#### FIFO Buffer
First-in-first-out memory buffer with read/write pointers and status flags

In [37]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_6_examples/example_10__fifo_buffer/"
files = ["fifo_buffer_design.sv", "fifo_buffer_design_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// fifo_buffer_design.sv
module fifo_buffer_8x4 (
    input  logic       clock,
    input  logic       reset_n,
    input  logic       write_enable,
    input  logic       read_enable,
    input  logic [3:0] write_data,
    output logic [3:0] read_data,
    output logic       fifo_full,
    output logic       fifo_empty,
    output logic [2:0] data_count
);

    // FIFO parameters
    parameter FIFO_DEPTH = 8;
    parameter ADDR_WIDTH = 3;  // log2(8) = 3
    
    // FIFO memory array
    logic [3:0] fifo_memory [0:FIFO_DEPTH-1];
    
    // Read and write pointers
    logic [ADDR_WIDTH-1:0] write_pointer;
    logic [ADDR_WIDTH-1:0] read_pointer;
    logic [ADDR_WIDTH:0]   count;  // Extra bit to distinguish full/empty
    
    // FIFO control logic
    always_ff @(posedge clock or negedge reset_n) begin
        if (!reset_n) begin
            // Reset all pointers and count
            write_pointer <= 0;
            read_pointer <= 0;
            count <= 0;
        end
        else begin
            // Write operation
            if (write_enable && !fifo_full) begin
                fifo_memory[write_pointer] <= write_data;
                write_pointer <= write_pointer + 1;
                $display("WRITE: Data=%h at addr=%d", write_data, write_pointer);
            end
            
            // Read operation
            if (read_enable && !fifo_empty) begin
                read_pointer <= read_pointer + 1;
                $display("READ:  Data=%h from addr=%d", fifo_memory[read_pointer], read_pointer);
            end
            
            // Update count
            case ({write_enable && !fifo_full, read_enable && !fifo_empty})
                2'b10: count <= count + 1;  // Write only
                2'b01: count <= count - 1;  // Read only
                2'b11: count <= count;      // Both (no change)
                2'b00: count <= count;      // Neither (no change)
            endcase
        end
    end
    
    // Output assignments
    assign read_data = fifo_memory[read_pointer];
    assign fifo_full = (count == FIFO_DEPTH);
    assign fifo_empty = (count == 0);
    assign data_count = count[2:0];  // Lower 3 bits for count display
    
    // Status display
    always @(posedge clock) begin
        if (reset_n) begin
            $display("Status: Count=%d, WPtr=%d, RPtr=%d, Full=%b, Empty=%b", 
                     count, write_pointer, read_pointer, fifo_full, fifo_empty);
        end
    end

endmodule
```

```systemverilog
// fifo_buffer_design_testbench.sv
module fifo_buffer_testbench;
    
    // Testbench signals
    logic       clock;
    logic       reset_n;
    logic       write_enable;
    logic       read_enable;
    logic [3:0] write_data;
    logic [3:0] read_data;
    logic       fifo_full;
    logic       fifo_empty;
    logic [2:0] data_count;
    
    // Test data array
    logic [3:0] test_data [0:9] = '{4'hA, 4'hB, 4'hC, 4'hD, 4'hE, 
                                    4'hF, 4'h1, 4'h2, 4'h3, 4'h4};
    integer write_index;
    
    // Instantiate the FIFO buffer design
    fifo_buffer_8x4 FIFO_BUFFER_INSTANCE (
        .clock(clock),
        .reset_n(reset_n),
        .write_enable(write_enable),
        .read_enable(read_enable),
        .write_data(write_data),
        .read_data(read_data),
        .fifo_full(fifo_full),
        .fifo_empty(fifo_empty),
        .data_count(data_count)
    );
    
    // Clock generation (10ns period)
    initial begin
        clock = 0;
        forever #5 clock = ~clock;
    end
    
    // Test sequence
    initial begin
        // Setup waveform dumping
        $dumpfile("fifo_buffer_testbench.vcd");
        $dumpvars(0, fifo_buffer_testbench);
        
        // Display header
        $display();
        $display("=== 8x4 FIFO Buffer Test ===");
        $display("FIFO Depth: 8 locations, Data Width: 4 bits");
        $display();
        
        // Initialize signals
        reset_n = 0;
        write_enable = 0;
        read_enable = 0;
        write_data = 0;
        write_index = 0;
        
        // Apply reset
        #15;
        reset_n = 1;
        $display("=== RESET COMPLETE ===");
        #10;
        
        // Test 1: Fill FIFO completely
        $display();
        $display("=== TEST 1: Filling FIFO ===");
        for (int i = 0; i < 8; i++) begin
            write_data = test_data[i];
            write_enable = 1;
            #10;
            write_enable = 0;
            #10;
        end
        
        // Try to write when full
        $display();
        $display("=== Testing write when FULL ===");
        write_data = 4'h9;
        write_enable = 1;
        #10;
        write_enable = 0;
        #10;
        
        // Test 2: Read all data from FIFO
        $display();
        $display("=== TEST 2: Reading from FIFO ===");
        for (int i = 0; i < 8; i++) begin
            read_enable = 1;
            #10;
            read_enable = 0;
            #10;
        end
        
        // Try to read when empty
        $display();
        $display("=== Testing read when EMPTY ===");
        read_enable = 1;
        #10;
        read_enable = 0;
        #10;
        
        // Test 3: Simultaneous read/write operations
        $display();
        $display("=== TEST 3: Simultaneous Read/Write ===");
        
        // First, put some data in FIFO
        for (int i = 0; i < 4; i++) begin
            write_data = test_data[i];
            write_enable = 1;
            #10;
            write_enable = 0;
            #10;
        end
        
        // Now do simultaneous read/write
        $display("Performing simultaneous read/write operations:");
        for (int i = 0; i < 3; i++) begin
            write_data = test_data[i+4];
            write_enable = 1;
            read_enable = 1;
            #10;
            write_enable = 0;
            read_enable = 0;
            #10;
        end
        
        // Test 4: Verify FIFO ordering (First-In-First-Out)
        $display();
        $display("=== TEST 4: FIFO Ordering Verification ===");
        
        // Clear FIFO first
        while (!fifo_empty) begin
            read_enable = 1;
            #10;
            read_enable = 0;
            #10;
        end
        
        // Write known sequence
        $display("Writing sequence: A, B, C, D");
        for (int i = 0; i < 4; i++) begin
            write_data = 4'hA + i[3:0];  // A, B, C, D
            write_enable = 1;
            #10;
            write_enable = 0;
            #10;
        end
        
        // Read back and verify order
        $display("Reading back sequence (should be A, B, C, D):");
        for (int i = 0; i < 4; i++) begin
            logic [3:0] expected_data;
            read_enable = 1;
            #10;
            expected_data = 4'hA + i[3:0];
            $display("Expected: %h, Got: %h %s", 
                     expected_data, read_data, 
                     (read_data == expected_data) ? "âœ“" : "âœ—");
            read_enable = 0;
            #10;
        end
        
        $display();
        $display("=== FIFO Buffer Test Complete ===");
        $display();
        
        #20;
        $finish;
    end
    
    // Monitor FIFO status changes
    always @(posedge clock) begin
        if (reset_n && (fifo_full || fifo_empty)) begin
            if (fifo_full)
                $display("*** FIFO is FULL ***");
            if (fifo_empty)
                $display("*** FIFO is EMPTY ***");
        end
    end

endmodule
```

Verilator Simulation Output:

=== 8x4 FIFO Buffer Test ===
FIFO Depth: 8 locations, Data Width: 4 bits

=== RESET COMPLETE ===
*** FIFO is EMPTY ***
Status: Count= 0, WPtr=0, RPtr=0, Full=0, Empty=1

=== TEST 1: Filling FIFO ===
*** FIFO is EMPTY ***
Status: Count= 0, WPtr=0, RPtr=0, Full=0, Empty=1
WRITE: Data=a at addr=0
Status: Count= 1, WPtr=1, RPtr=0, Full=0, Empty=0
Status: Count= 1, WPtr=1, RPtr=0, Full=0, Empty=0
WRITE: Data=b at addr=1
Status: Count= 2, WPtr=2, RPtr=0, Full=0, Empty=0
Status: Count= 2, WPtr=2, RPtr=0, Full=0, Empty=0
WRITE: Data=c at addr=2
Status: Count= 3, WPtr=3, RPtr=0, Full=0, Empty=0
Status: Count= 3, WPtr=3, RPtr=0, Full=0, Empty=0
WRITE: Data=d at addr=3
Status: Count= 4, WPtr=4, RPtr=0, Full=0, Empty=0
Status: Count= 4, WPtr=4, RPtr=0, Full=0, Empty=0
WRITE: Data=e at addr=4
Status: Count= 5, WPtr=5, RPtr=0, Full=0, Empty=0
Status: Count= 5, WPtr=5, RPtr=0, Full=0, Empty=0
WRITE: Data=f at addr=5
Status: Count= 6, WPtr=6, RPtr=0, Full=0, Empty=0
Statu

0

#### PWM Generator
Pulse-width modulation generator with configurable duty cycle

In [38]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_6_examples/example_11__pwm_generator/"
files = ["pwm_generator_design.sv", "pwm_generator_design_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// pwm_generator_design.sv
module pwm_generator_8bit (
    input  logic       clock,
    input  logic       reset_n,
    input  logic       enable,
    input  logic [7:0] duty_cycle,    // 0-255 (0% to 100% duty cycle)
    output logic       pwm_output,
    output logic [7:0] counter_value
);

    // 8-bit counter for PWM period
    logic [7:0] pwm_counter;
    
    // PWM counter logic
    always_ff @(posedge clock or negedge reset_n) begin
        if (!reset_n) begin
            pwm_counter <= 8'b0;
        end
        else if (enable) begin
            pwm_counter <= pwm_counter + 1;  // Auto-wraps at 255->0
        end
    end
    
    // PWM output generation
    // Output is HIGH when counter < duty_cycle, LOW otherwise
    assign pwm_output = enable ? (pwm_counter < duty_cycle) : 1'b0;
    
    // Counter output for monitoring
    assign counter_value = pwm_counter;
    
    // Display PWM parameters when duty cycle changes
    always @(duty_cycle) begin
        if (reset_n) begin
            real duty_percent;
            duty_percent = (duty_cycle * 100.0) / 256.0;
            $display("PWM Config: Duty Cycle = %d/256 (%.1f percent)", duty_cycle, duty_percent);
        end
    end
    
    // Optional: Display counter rollover
    always @(posedge clock) begin
        if (enable && reset_n && (pwm_counter == 8'hFF)) begin
            $display("PWM Period Complete (Counter rolled over)");
        end
    end

endmodule
```

```systemverilog
// pwm_generator_design_testbench.sv
module pwm_generator_testbench;
    
    // Testbench signals
    logic       clock;
    logic       reset_n;
    logic       enable;
    logic [7:0] duty_cycle;
    logic       pwm_output;
    logic [7:0] counter_value;
    
    // Variables for duty cycle measurement
    integer high_count, total_count;
    real measured_duty;
    
    // Instantiate the PWM generator design
    pwm_generator_8bit PWM_GENERATOR_INSTANCE (
        .clock(clock),
        .reset_n(reset_n),
        .enable(enable),
        .duty_cycle(duty_cycle),
        .pwm_output(pwm_output),
        .counter_value(counter_value)
    );
    
    // Clock generation (10ns period = 100MHz)
    initial begin
        clock = 0;
        forever #5 clock = ~clock;
    end
    
    // Task to measure duty cycle over one complete period
    task measure_duty_cycle;
        input [7:0] expected_duty;
        begin
            high_count = 0;
            total_count = 0;
            
            // Wait for counter to start from 0
            while (counter_value != 0) @(posedge clock);
            
            // Measure for one complete period (256 clock cycles)
            repeat(256) begin
                @(posedge clock);
                if (pwm_output) high_count++;
                total_count++;
            end
            
            measured_duty = (high_count * 100.0) / total_count;
            $display("Expected: %d/256 (%.1f percent), Measured: %d/256 (%.1f percent) %s",
                     expected_duty, (expected_duty * 100.0)/256.0,
                     high_count, measured_duty,
                     (high_count == int'(expected_duty)) ? "âœ“" : "âœ—");
        end
    endtask
    
    // Test sequence
    initial begin
        // Setup waveform dumping
        $dumpfile("pwm_generator_testbench.vcd");
        $dumpvars(0, pwm_generator_testbench);
        
        // Display header
        $display();
        $display("=== 8-Bit PWM Generator Test ===");
        $display("PWM Frequency = Clock/256");
        $display("Resolution = 8 bits (256 steps)");
        $display();
        
        // Initialize signals
        reset_n = 0;
        enable = 0;
        duty_cycle = 0;
        
        // Apply reset
        #25;
        reset_n = 1;
        $display("=== RESET COMPLETE ===");
        #10;
        
        // Test 1: Different duty cycles
        $display();
        $display("=== TEST 1: Various Duty Cycles ===");
        
        enable = 1;
        
        // Test 0% duty cycle
        $display();
        $display("Testing 0 percent duty cycle:");
        duty_cycle = 8'd0;
        measure_duty_cycle(8'd0);
        
        // Test 25% duty cycle
        $display();
        $display("Testing 25 percent duty cycle:");
        duty_cycle = 8'd64;  // 64/256 = 25%
        measure_duty_cycle(8'd64);
        
        // Test 50% duty cycle
        $display();
        $display("Testing 50 percent duty cycle:");
        duty_cycle = 8'd128; // 128/256 = 50%
        measure_duty_cycle(8'd128);
        
        // Test 75% duty cycle
        $display();
        $display("Testing 75 percent duty cycle:");
        duty_cycle = 8'd192; // 192/256 = 75%
        measure_duty_cycle(8'd192);
        
        // Test 100% duty cycle
        $display();
        $display("Testing ~100 percent duty cycle:");
        duty_cycle = 8'd255; // 255/256 â‰ˆ 99.6%
        measure_duty_cycle(8'd255);
        
        // Test 2: Dynamic duty cycle changes
        $display();
        $display("=== TEST 2: Dynamic Duty Cycle Changes ===");
        
        // Show real-time changes
        $display("Changing duty cycle every 0.5 periods...");
        
        duty_cycle = 8'd32;   // 12.5 percent
        #1280;  // 0.5 period (128 clocks)
        
        duty_cycle = 8'd96;   // 37.5 percent
        #1280;
        
        duty_cycle = 8'd160;  // 62.5 percent
        #1280;
        
        duty_cycle = 8'd224;  // 87.5 percent
        #1280;
        
        // Test 3: Enable/Disable functionality
        $display();
        $display("=== TEST 3: Enable/Disable Test ===");
        
        duty_cycle = 8'd128;  // 50 percent
        $display("PWM Enabled (50 percent duty cycle):");
        #500;
        
        enable = 0;
        $display("PWM Disabled (output should be 0):");
        #500;
        
        enable = 1;
        $display("PWM Re-enabled:");
        #500;
        
        // Test 4: Edge cases
        $display();
        $display("=== TEST 4: Edge Cases ===");
        
        // Minimum non-zero duty cycle
        $display();
        $display("Testing minimum duty cycle (1/256):");
        duty_cycle = 8'd1;
        measure_duty_cycle(8'd1);
        
        // Maximum duty cycle
        $display();
        $display("Testing maximum duty cycle (255/256):");
        duty_cycle = 8'd255;
        measure_duty_cycle(8'd255);
        
        // Show a few complete PWM periods with 50% duty cycle
        $display();
        $display("=== Showing 2 complete PWM periods (50 percent duty) ===");
        duty_cycle = 8'd128;
        
        repeat(512) begin  // 2 complete periods
            @(posedge clock);
            if (counter_value == 0) begin
                $display("--- New PWM Period Starting ---");
            end
        end
        
        $display();
        $display("=== PWM Generator Test Complete ===");
        $display();
        
        #100;
        $finish;
    end
    
    // Monitor PWM output transitions
    always @(pwm_output) begin
        if (reset_n && enable) begin
            $display("PWM: %s at counter=%d", pwm_output ? "HIGH" : "LOW", counter_value);
        end
    end

endmodule
```

Verilator Simulation Output:

=== 8-Bit PWM Generator Test ===
PWM Frequency = Clock/256
Resolution = 8 bits (256 steps)

=== RESET COMPLETE ===

=== TEST 1: Various Duty Cycles ===

Testing 0 percent duty cycle:
PWM Period Complete (Counter rolled over)
Expected:   0/256 (0.0 percent), Measured:           0/256 (0.0 percent) âœ“

Testing 25 percent duty cycle:
PWM: HIGH at counter=  0
PWM Config: Duty Cycle =  64/256 (25.0 percent)
PWM:  LOW at counter= 64
PWM Period Complete (Counter rolled over)
PWM: HIGH at counter=  0
Expected:  64/256 (25.0 percent), Measured:          64/256 (25.0 percent) âœ“

Testing 50 percent duty cycle:
PWM Config: Duty Cycle = 128/256 (50.0 percent)
PWM:  LOW at counter=128
PWM Period Complete (Counter rolled over)
PWM: HIGH at counter=  0
Expected: 128/256 (50.0 percent), Measured:         128/256 (50.0 percent) âœ“

Testing 75 percent duty cycle:
PWM Config: Duty Cycle = 192/256 (75.0 percent)
PWM:  LOW at counter=192
PWM Period Complete (Counter rolled 

0

#### Traffic Light Controller
State machine controlling traffic light sequences with timing

In [39]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_6_examples/example_12__traffic_light_controller/"
files = [
    "traffic_light_controller.sv",
    "traffic_light_controller_testbench.sv",
]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// traffic_light_controller.sv
module traffic_light_controller (
    input  logic clk,
    input  logic reset,
    output logic red_light,
    output logic yellow_light,
    output logic green_light
);

    // State enumeration
    typedef enum logic [1:0] {
        RED    = 2'b00,
        GREEN  = 2'b01,
        YELLOW = 2'b10
    } traffic_state_t;

    traffic_state_t current_state, next_state;
    
    // Timer counter for state duration
    logic [3:0] timer_count;
    
    // State timing parameters (in clock cycles)
    parameter RED_TIME    = 4'd8;  // Red light duration
    parameter GREEN_TIME  = 4'd6;  // Green light duration  
    parameter YELLOW_TIME = 4'd2;  // Yellow light duration

    // State register
    always_ff @(posedge clk or posedge reset) begin
        if (reset) begin
            current_state <= RED;
            timer_count <= 4'd0;
        end else begin
            current_state <= next_state;
            if (next_state != current_state) begin
                timer_count <= 4'd0;  // Reset timer on state change
            end else begin
                timer_count <= timer_count + 1;
            end
        end
    end

    // Next state logic
    always_comb begin
        next_state = current_state;  // Default: stay in current state
        
        case (current_state)
            RED: begin
                if (timer_count >= RED_TIME - 1) begin
                    next_state = GREEN;
                end
            end
            
            GREEN: begin
                if (timer_count >= GREEN_TIME - 1) begin
                    next_state = YELLOW;
                end
            end
            
            YELLOW: begin
                if (timer_count >= YELLOW_TIME - 1) begin
                    next_state = RED;
                end
            end
            
            default: next_state = RED;
        endcase
    end

    // Output logic
    always_comb begin
        red_light    = (current_state == RED);
        green_light  = (current_state == GREEN);
        yellow_light = (current_state == YELLOW);
    end

    // Display state changes for debugging
    always_ff @(posedge clk) begin
        if (next_state != current_state) begin
            case (next_state)
                RED:    $display("Time %0t: Traffic Light -> RED", $time);
                GREEN:  $display("Time %0t: Traffic Light -> GREEN", $time);
                YELLOW: $display("Time %0t: Traffic Light -> YELLOW", $time);
                default: $display("Time %0t: Traffic Light -> UNKNOWN STATE", $time);
            endcase
        end
    end

endmodule
```

```systemverilog
// traffic_light_controller_testbench.sv
module traffic_light_testbench;

    // Testbench signals
    logic clk;
    logic reset;
    logic red_light;
    logic yellow_light;
    logic green_light;

    // Instantiate design under test
    traffic_light_controller TRAFFIC_CONTROLLER (
        .clk(clk),
        .reset(reset),
        .red_light(red_light),
        .yellow_light(yellow_light),
        .green_light(green_light)
    );

    // Clock generation
    initial begin
        clk = 0;
        forever #5 clk = ~clk;  // 10 time unit period
    end

    // Test sequence
    initial begin
        // Dump waves
        $dumpfile("traffic_light_testbench.vcd");
        $dumpvars(0, traffic_light_testbench);
        
        $display("=== Traffic Light Controller Test ===");
        $display("Time %0t: Starting simulation", $time);
        
        // Initialize
        reset = 1;
        #20;
        reset = 0;
        
        $display("Time %0t: Reset released - Traffic light should start in RED", $time);
        
        // Monitor light states
        forever begin
            @(posedge clk);
            $display("Time %0t: Lights - RED:%b GREEN:%b YELLOW:%b", 
                    $time, red_light, green_light, yellow_light);
            
            // Check that only one light is on at a time
            if ((red_light + green_light + yellow_light) != 1) begin
                $display("ERROR: Multiple or no lights active!");
                $finish;
            end
        end
    end

    // Test duration control
    initial begin
        #2000;  // Run for enough time to see multiple cycles
        $display("Time %0t: Test completed successfully", $time);
        $display("=== Traffic Light Controller Test Passed ===");
        $finish;
    end

    // Display light changes with timing
    always @(posedge clk) begin
        if ($changed(red_light) || $changed(green_light) || $changed(yellow_light)) begin
            if (red_light)    $display(">>> RED LIGHT ON    (Duration: 8 cycles)");
            if (green_light)  $display(">>> GREEN LIGHT ON  (Duration: 6 cycles)");
            if (yellow_light) $display(">>> YELLOW LIGHT ON (Duration: 2 cycles)");
        end
    end

endmodule
```

Verilator Simulation Output:
=== Traffic Light Controller Test ===
Time 0: Starting simulation
>>> RED LIGHT ON    (Duration: 8 cycles)
Time 20: Reset released - Traffic light should start in RED
Time 25: Lights - RED:1 GREEN:0 YELLOW:0
Time 35: Lights - RED:1 GREEN:0 YELLOW:0
Time 45: Lights - RED:1 GREEN:0 YELLOW:0
Time 55: Lights - RED:1 GREEN:0 YELLOW:0
Time 65: Lights - RED:1 GREEN:0 YELLOW:0
Time 75: Lights - RED:1 GREEN:0 YELLOW:0
Time 85: Lights - RED:1 GREEN:0 YELLOW:0
Time 95: Lights - RED:1 GREEN:0 YELLOW:0
Time 95: Traffic Light -> GREEN
Time 105: Lights - RED:0 GREEN:1 YELLOW:0
>>> GREEN LIGHT ON  (Duration: 6 cycles)
Time 115: Lights - RED:0 GREEN:1 YELLOW:0
Time 125: Lights - RED:0 GREEN:1 YELLOW:0
Time 135: Lights - RED:0 GREEN:1 YELLOW:0
Time 145: Lights - RED:0 GREEN:1 YELLOW:0
Time 155: Lights - RED:0 GREEN:1 YELLOW:0
Time 155: Traffic Light -> YELLOW
Time 165: Lights - RED:0 GREEN:0 YELLOW:1
>>> YELLOW LIGHT ON (Duration: 2 cycles)
Time 175: Lights - RED:0 GREEN:0 Y

0

#### UART Transmitter
Serial data transmitter with start/stop bits and baud rate control

In [40]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_6_examples/example_13__uart_transmitter/"
files = ["uart_transmitter.sv", "uart_transmitter_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// uart_transmitter.sv - Clean Version with Minimal Debug Output
module uart_transmitter (
    input  logic       clk,
    input  logic       reset,
    input  logic       tx_start,      // Start transmission
    input  logic [7:0] tx_data,       // Data to transmit
    output logic       tx_serial,     // Serial output
    output logic       tx_busy,       // Transmission in progress
    output logic       tx_done        // Transmission complete
);

    // UART frame states
    typedef enum logic [2:0] {
        IDLE       = 3'b000,
        START_BIT  = 3'b001,
        DATA_BITS  = 3'b010,
        STOP_BIT   = 3'b011
    } uart_state_t;

    uart_state_t current_state;
    
    // Internal registers
    logic [7:0] shift_register;    // Data to be transmitted
    logic [2:0] bit_counter;       // Count data bits (0-7)
    logic [3:0] baud_counter;      // Baud rate timing
    
    // Baud rate control (simplified for simulation)
    parameter BAUD_TICKS = 4'd8;   // Clock cycles per bit period
    
    // State register and counters
    always_ff @(posedge clk or posedge reset) begin
        if (reset) begin
            current_state <= IDLE;
            shift_register <= 8'h00;
            bit_counter <= 3'd0;
            baud_counter <= 4'd0;
        end else begin
            // State machine
            case (current_state)
                IDLE: begin
                    baud_counter <= 4'd0;
                    bit_counter <= 3'd0;
                    if (tx_start) begin
                        current_state <= START_BIT;
                        shift_register <= tx_data;
                        $display("Time %0t: Start transmission - Data: 0x%02h", $time, tx_data);
                    end
                end
                
                START_BIT: begin
                    if (baud_counter >= BAUD_TICKS - 1) begin
                        baud_counter <= 4'd0;
                        current_state <= DATA_BITS;
                    end else begin
                        baud_counter <= baud_counter + 1;
                    end
                end
                
                DATA_BITS: begin
                    if (baud_counter >= BAUD_TICKS - 1) begin
                        baud_counter <= 4'd0;
                        shift_register <= {1'b0, shift_register[7:1]};
                        bit_counter <= bit_counter + 1;
                        if (bit_counter >= 3'd7) begin
                            current_state <= STOP_BIT;
                        end
                    end else begin
                        baud_counter <= baud_counter + 1;
                    end
                end
                
                STOP_BIT: begin
                    if (baud_counter >= BAUD_TICKS - 1) begin
                        baud_counter <= 4'd0;
                        current_state <= IDLE;
                        $display("Time %0t: UART transmission complete!", $time);
                    end else begin
                        baud_counter <= baud_counter + 1;
                    end
                end
                
                default: current_state <= IDLE;
            endcase
        end
    end

    // Output logic
    always_comb begin
        case (current_state)
            IDLE:      tx_serial = 1'b1;              // Line idle high
            START_BIT: tx_serial = 1'b0;              // Start bit low
            DATA_BITS: tx_serial = shift_register[0]; // LSB first
            STOP_BIT:  tx_serial = 1'b1;              // Stop bit high
            default:   tx_serial = 1'b1;
        endcase
        
        tx_busy = (current_state != IDLE);
        tx_done = (current_state == STOP_BIT) && (baud_counter >= BAUD_TICKS - 1);
    end

    // Minimal debug output - only show key events
    // Remove or comment out this block for completely silent operation
    /*
    always_ff @(posedge clk) begin
        if (tx_start && current_state == IDLE) begin
            $display("Time %0t: Starting UART frame", $time);
        end
        if (tx_done) begin
            $display("Time %0t: UART frame complete", $time);
        end
    end
    */

endmodule
```

```systemverilog
// uart_transmitter_testbench.sv - Fixed Version
module uart_transmitter_testbench;

    // Testbench signals
    logic       clk;
    logic       reset;
    logic       tx_start;
    logic [7:0] tx_data;
    logic       tx_serial;
    logic       tx_busy;
    logic       tx_done;

    // Test data array
    logic [7:0] test_messages [0:3] = '{8'h41, 8'h42, 8'h43, 8'h0A}; // "ABC\n"

    // Instantiate design under test
    uart_transmitter UART_TX (
        .clk(clk),
        .reset(reset),
        .tx_start(tx_start),
        .tx_data(tx_data),
        .tx_serial(tx_serial),
        .tx_busy(tx_busy),
        .tx_done(tx_done)
    );

    // Clock generation
    initial begin
        clk = 0;
        forever #5 clk = ~clk;  // 10 time unit period
    end

    // Test sequence
    initial begin
        // Dump waves
        $dumpfile("uart_transmitter_testbench.vcd");
        $dumpvars(0, uart_transmitter_testbench);
        
        $display("=== UART Transmitter Test ===");
        $display("Time %0t: Starting simulation", $time);
        
        // Initialize
        reset = 1;
        tx_start = 0;
        tx_data = 8'h00;
        #50;
        reset = 0;
        #20;
        
        $display("Time %0t: Reset complete, UART ready", $time);
        
        // Send test messages
        for (int i = 0; i < 4; i++) begin
            // Wait for UART to be ready
            while (tx_busy) @(posedge clk);
            
            // Start transmission
            tx_data = test_messages[i];
            $display("Time %0t: Sending byte: 0x%02h ('%c')", $time, tx_data, 
                    (tx_data >= 32 && tx_data <= 126) ? tx_data : ".");
            
            @(posedge clk);  // Wait for clock edge
            tx_start = 1;
            @(posedge clk);  // Hold for one clock cycle
            tx_start = 0;
            
            // Wait for transmission to complete
            while (!tx_done) @(posedge clk);
            @(posedge clk);  // Extra cycle to see completion
            
            $display("Time %0t: Byte %0d transmitted", $time, i+1);
        end
        
        // Wait a bit more
        #100;
        $display("Time %0t: All transmissions completed successfully", $time);
        $display("=== UART Transmitter Test Passed ===");
        $finish;
    end

    // Remove verbose serial monitoring
    // (comment out or remove this block to reduce messages further)

    // Capture and display complete transmitted bytes
    logic [7:0] captured_byte;
    logic [2:0] bit_index;  // Changed from [3:0] to [2:0] since we only need 0-7
    logic prev_tx_busy;
    
    always @(posedge clk) begin
        if (reset) begin
            captured_byte <= 8'h00;
            bit_index <= 3'd0;  // Changed from 4'd0 to 3'd0
            prev_tx_busy <= 1'b0;
        end else begin
            prev_tx_busy <= tx_busy;
            
            // Detect start of transmission
            if (tx_busy && !prev_tx_busy) begin
                captured_byte <= 8'h00;
                bit_index <= 3'd0;  // Changed from 4'd0 to 3'd0
                $display("Time %0t: >>> Starting to capture frame", $time);
            end
            
            // Capture data bits
            if (UART_TX.current_state == UART_TX.DATA_BITS && UART_TX.baud_counter == 0) begin
                captured_byte[bit_index] <= tx_serial;
                bit_index <= bit_index + 1;
            end
            
            // Display complete byte
            if (tx_done) begin
                $display("Time %0t: >>> Complete byte received: 0x%02h ('%c')", 
                       $time, captured_byte, 
                       (captured_byte >= 32 && captured_byte <= 126) ? captured_byte : ".");
            end
        end
    end

    // Watchdog timer
    initial begin
        #5000;
        $display("Time %0t: Test completed normally", $time);
        $finish;
    end

endmodule
```

Verilator Simulation Output:
=== UART Transmitter Test ===
Time 0: Starting simulation
Time 70: Reset complete, UART ready
Time 70: Sending byte: 0x41 ('A')
Time 75: Start transmission - Data: 0x41
Time 85: >>> Starting to capture frame
Time 875: >>> Complete byte received: 0x41 ('A')
Time 875: UART transmission complete!
Time 885: Byte 1 transmitted
Time 885: Sending byte: 0x42 ('B')
Time 895: Start transmission - Data: 0x42
Time 905: >>> Starting to capture frame
Time 1695: >>> Complete byte received: 0x42 ('B')
Time 1695: UART transmission complete!
Time 1705: Byte 2 transmitted
Time 1705: Sending byte: 0x43 ('C')
Time 1715: Start transmission - Data: 0x43
Time 1725: >>> Starting to capture frame
Time 2515: >>> Complete byte received: 0x43 ('C')
Time 2515: UART transmission complete!
Time 2525: Byte 3 transmitted
Time 2525: Sending byte: 0x0a ('.')
Time 2535: Start transmission - Data: 0x0a
Time 2545: >>> Starting to capture frame
Time 3335: >>> Complete byte received: 0x0a ('.')
Ti

0

#### Memory Interface Controller
Controller for read/write operations to external memory with handshaking

In [41]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_6_examples/example_14__memory_interface_controller/"
files = [
    "memory_interface_controller.sv",
    "memory_interface_controller_testbench.sv",
]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// memory_interface_controller.sv
module memory_interface_controller (
    input  logic        clk,
    input  logic        rst_n,
    
    // CPU Interface
    input  logic        cpu_req,           // CPU request
    input  logic        cpu_wr_en,         // CPU write enable (1=write, 0=read)
    input  logic [7:0]  cpu_addr,          // CPU address
    input  logic [15:0] cpu_wr_data,       // CPU write data
    output logic [15:0] cpu_rd_data,       // CPU read data
    output logic        cpu_ready,         // CPU ready signal
    
    // External Memory Interface
    output logic        mem_req,           // Memory request
    output logic        mem_wr_en,         // Memory write enable
    output logic [7:0]  mem_addr,          // Memory address
    output logic [15:0] mem_wr_data,       // Memory write data
    input  logic [15:0] mem_rd_data,       // Memory read data
    input  logic        mem_ready          // Memory ready signal
);

    // State machine states
    typedef enum logic [1:0] {
        IDLE        = 2'b00,
        MEM_ACCESS  = 2'b01,
        WAIT_READY  = 2'b10
    } state_t;
    
    state_t current_state, next_state;
    
    // Internal registers
    logic [7:0]  addr_reg;
    logic [15:0] wr_data_reg;
    logic        wr_en_reg;
    
    // State machine: sequential logic
    always_ff @(posedge clk or negedge rst_n) begin
        if (!rst_n) begin
            current_state <= IDLE;
            addr_reg      <= 8'b0;
            wr_data_reg   <= 16'b0;
            wr_en_reg     <= 1'b0;
        end else begin
            current_state <= next_state;
            
            // Capture CPU request when transitioning to MEM_ACCESS
            if (current_state == IDLE && cpu_req) begin
                addr_reg    <= cpu_addr;
                wr_data_reg <= cpu_wr_data;
                wr_en_reg   <= cpu_wr_en;
            end
        end
    end
    
    // State machine: combinational logic
    always_comb begin
        // Default values
        next_state = current_state;
        mem_req    = 1'b0;
        cpu_ready  = 1'b0;
        
        case (current_state)
            IDLE: begin
                cpu_ready = 1'b1;  // Ready to accept new requests
                if (cpu_req) begin
                    next_state = MEM_ACCESS;
                end
            end
            
            MEM_ACCESS: begin
                mem_req = 1'b1;    // Send request to memory
                if (mem_ready) begin
                    cpu_ready  = 1'b1;  // Transaction complete
                    next_state = IDLE;
                end else begin
                    next_state = WAIT_READY;
                end
            end
            
            WAIT_READY: begin
                mem_req = 1'b1;    // Keep request active
                if (mem_ready) begin
                    cpu_ready  = 1'b1;  // Transaction complete
                    next_state = IDLE;
                end
            end
            
            default: begin
                next_state = IDLE;
            end
        endcase
    end
    
    // Memory interface outputs
    assign mem_addr    = addr_reg;
    assign mem_wr_data = wr_data_reg;
    assign mem_wr_en   = wr_en_reg;
    
    // CPU read data output
    assign cpu_rd_data = mem_rd_data;
    
    initial $display("Memory Interface Controller initialized");

endmodule
```

```systemverilog
// memory_interface_controller_testbench.sv
module memory_controller_testbench;

    // Clock and reset
    logic clk;
    logic rst_n;
    
    // CPU Interface signals
    logic        cpu_req;
    logic        cpu_wr_en;
    logic [7:0]  cpu_addr;
    logic [15:0] cpu_wr_data;
    logic [15:0] cpu_rd_data;
    logic        cpu_ready;
    
    // Memory Interface signals
    logic        mem_req;
    logic        mem_wr_en;
    logic [7:0]  mem_addr;
    logic [15:0] mem_wr_data;
    logic [15:0] mem_rd_data;
    logic        mem_ready;
    
    // Simple memory model
    logic [15:0] memory_array [0:255];
    
    // Instantiate the memory interface controller
    memory_interface_controller MEMORY_CONTROLLER_INST (
        .clk(clk),
        .rst_n(rst_n),
        .cpu_req(cpu_req),
        .cpu_wr_en(cpu_wr_en),
        .cpu_addr(cpu_addr),
        .cpu_wr_data(cpu_wr_data),
        .cpu_rd_data(cpu_rd_data),
        .cpu_ready(cpu_ready),
        .mem_req(mem_req),
        .mem_wr_en(mem_wr_en),
        .mem_addr(mem_addr),
        .mem_wr_data(mem_wr_data),
        .mem_rd_data(mem_rd_data),
        .mem_ready(mem_ready)
    );
    
    // Clock generation
    initial begin
        clk = 0;
        forever #5 clk = ~clk;  // 10ns clock period
    end
    
    // Simple memory model behavior
    always_ff @(posedge clk) begin
        if (mem_req) begin
            if (mem_wr_en) begin
                // Write operation
                memory_array[mem_addr] <= mem_wr_data;
                $display("Time %0t: Memory WRITE - Addr: 0x%02h, Data: 0x%04h", 
                         $time, mem_addr, mem_wr_data);
            end else begin
                // Read operation
                $display("Time %0t: Memory READ  - Addr: 0x%02h, Data: 0x%04h", 
                         $time, mem_addr, memory_array[mem_addr]);
            end
        end
    end
    
    // Memory ready signal (simulate memory delay)
    always_ff @(posedge clk or negedge rst_n) begin
        if (!rst_n) begin
            mem_ready <= 1'b0;
        end else begin
            mem_ready <= mem_req;  // Ready one cycle after request
        end
    end
    
    // Memory read data
    assign mem_rd_data = memory_array[mem_addr];
    
    // Test sequence
    initial begin
        // Initialize
        rst_n = 0;
        cpu_req = 0;
        cpu_wr_en = 0;
        cpu_addr = 0;
        cpu_wr_data = 0;
        
        // Initialize memory with some values
        memory_array[8'h10] = 16'hABCD;
        memory_array[8'h20] = 16'h1234;
        memory_array[8'h30] = 16'h5678;
        
        // Dump waves
        $dumpfile("memory_controller_testbench.vcd");
        $dumpvars(0, memory_controller_testbench);
        
        $display();
        $display("=== Memory Interface Controller Test ===");
        $display();
        
        // Release reset
        #20 rst_n = 1;
        #10;
        
        // Test 1: Write operation
        $display("Test 1: Write 0xDEAD to address 0x10");
        wait(cpu_ready);
        cpu_req = 1;
        cpu_wr_en = 1;
        cpu_addr = 8'h10;
        cpu_wr_data = 16'hDEAD;
        #10;
        cpu_req = 0;
        wait(cpu_ready);
        #20;
        
        // Test 2: Read operation
        $display("Test 2: Read from address 0x10");
        wait(cpu_ready);
        cpu_req = 1;
        cpu_wr_en = 0;
        cpu_addr = 8'h10;
        #10;
        cpu_req = 0;
        wait(cpu_ready);
        $display("Read data: 0x%04h", cpu_rd_data);
        #20;
        
        // Test 3: Write to different address
        $display("Test 3: Write 0xBEEF to address 0x20");
        wait(cpu_ready);
        cpu_req = 1;
        cpu_wr_en = 1;
        cpu_addr = 8'h20;
        cpu_wr_data = 16'hBEEF;
        #10;
        cpu_req = 0;
        wait(cpu_ready);
        #20;
        
        // Test 4: Read from different address
        $display("Test 4: Read from address 0x20");
        wait(cpu_ready);
        cpu_req = 1;
        cpu_wr_en = 0;
        cpu_addr = 8'h20;
        #10;
        cpu_req = 0;
        wait(cpu_ready);
        $display("Read data: 0x%04h", cpu_rd_data);
        #20;
        
        // Test 5: Multiple operations
        $display("Test 5: Multiple operations");
        for (int i = 0; i < 3; i++) begin
            // Write
            wait(cpu_ready);
            cpu_req = 1;
            cpu_wr_en = 1;
            cpu_addr = 8'h30 + 8'(i);
            cpu_wr_data = 16'h1000 + 16'(i);
            #10;
            cpu_req = 0;
            wait(cpu_ready);
            
            // Read back
            wait(cpu_ready);
            cpu_req = 1;
            cpu_wr_en = 0;
            cpu_addr = 8'h30 + 8'(i);
            #10;
            cpu_req = 0;
            wait(cpu_ready);
            $display("Address 0x%02h: Written 0x%04h, Read 0x%04h", 
                     8'h30 + 8'(i), 16'h1000 + 16'(i), cpu_rd_data);
            #10;
        end
        
        $display();
        $display("=== All tests completed ===");
        $display();
        
        #50 $finish;
    end

endmodule
```

Verilator Simulation Output:
Memory Interface Controller initialized

=== Memory Interface Controller Test ===

Test 1: Write 0xDEAD to address 0x10
Time 45: Memory WRITE - Addr: 0x10, Data: 0xdead
Time 55: Memory WRITE - Addr: 0x10, Data: 0xdead
Test 2: Read from address 0x10
Time 75: Memory READ  - Addr: 0x10, Data: 0xdead
Read data: 0xdead
Time 85: Memory READ  - Addr: 0x10, Data: 0xdead
Test 3: Write 0xBEEF to address 0x20
Time 105: Memory WRITE - Addr: 0x20, Data: 0xbeef
Time 115: Memory WRITE - Addr: 0x20, Data: 0xbeef
Test 4: Read from address 0x20
Time 135: Memory READ  - Addr: 0x20, Data: 0xbeef
Read data: 0xbeef
Time 145: Memory READ  - Addr: 0x20, Data: 0xbeef
Test 5: Multiple operations
Time 165: Memory WRITE - Addr: 0x30, Data: 0x1000
Address 0x30: Written 0x1000, Read 0x1000
Time 175: Memory WRITE - Addr: 0x30, Data: 0x1000
Time 195: Memory WRITE - Addr: 0x31, Data: 0x1001
Address 0x31: Written 0x1001, Read 0x1001
Time 205: Memory WRITE - Addr: 0x31, Data: 0x1001
Time 225

0

## always_latch for Latches

#### Address Latch
Transparent latch for holding address during memory access cycles

In [42]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_6_examples/example_15__address_latch/"
files = ["address_latch.sv", "address_latch_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// address_latch.sv
module address_latch (
    input  logic        clk,
    input  logic        rst_n,
    
    // Input address from CPU/Controller
    input  logic [15:0] addr_in,           // Input address
    input  logic        addr_valid,        // Address valid signal
    
    // Memory access control
    input  logic        mem_cycle_start,   // Start of memory cycle
    input  logic        mem_cycle_end,     // End of memory cycle
    
    // Latched address output
    output logic [15:0] addr_out,          // Latched address output
    output logic        addr_latched       // Address is currently latched
);

    // Internal latch enable signal
    logic latch_enable;
    logic cycle_active;
    
    // Memory cycle tracking
    always_ff @(posedge clk or negedge rst_n) begin
        if (!rst_n) begin
            cycle_active <= 1'b0;
        end else begin
            if (mem_cycle_start) begin
                cycle_active <= 1'b1;
            end else if (mem_cycle_end) begin
                cycle_active <= 1'b0;
            end
        end
    end
    
    // Latch enable logic: transparent when not in memory cycle and address is valid
    assign latch_enable = !cycle_active && addr_valid;
    
    // Transparent latch behavior
    always_latch begin
        if (latch_enable) begin
            addr_out = addr_in;
        end
    end
    
    // Address latched status
    assign addr_latched = cycle_active;
    
    // Display messages for debugging
    always @(posedge mem_cycle_start) begin
        $display("Time %0t: Address latch - Memory cycle started, address 0x%04h latched", 
                 $time, addr_out);
    end
    
    always @(posedge mem_cycle_end) begin
        $display("Time %0t: Address latch - Memory cycle ended, latch now transparent", 
                 $time);
    end
    
    initial $display("Address Latch initialized - Transparent latch for memory access");

endmodule
```

```systemverilog
// address_latch_testbench.sv
module address_latch_testbench;

    // Clock and reset
    logic clk;
    logic rst_n;
    
    // Input signals
    logic [15:0] addr_in;
    logic        addr_valid;
    logic        mem_cycle_start;
    logic        mem_cycle_end;
    
    // Output signals
    logic [15:0] addr_out;
    logic        addr_latched;
    
    // Instantiate the address latch
    address_latch ADDRESS_LATCH_INST (
        .clk(clk),
        .rst_n(rst_n),
        .addr_in(addr_in),
        .addr_valid(addr_valid),
        .mem_cycle_start(mem_cycle_start),
        .mem_cycle_end(mem_cycle_end),
        .addr_out(addr_out),
        .addr_latched(addr_latched)
    );
    
    // Clock generation
    initial begin
        clk = 0;
        forever #5 clk = ~clk;  // 10ns clock period
    end
    
    // Test sequence
    initial begin
        // Initialize signals
        rst_n = 0;
        addr_in = 16'h0000;
        addr_valid = 0;
        mem_cycle_start = 0;
        mem_cycle_end = 0;
        
        // Dump waves
        $dumpfile("address_latch_testbench.vcd");
        $dumpvars(0, address_latch_testbench);
        
        $display();
        $display("=== Address Latch Test ===");
        $display();
        
        // Release reset
        #20 rst_n = 1;
        #10;
        
        // Test 1: Basic transparent operation
        $display("Test 1: Transparent latch operation");
        addr_in = 16'hA000;
        addr_valid = 1;
        #10;
        $display("Input: 0x%04h, Output: 0x%04h, Latched: %b", addr_in, addr_out, addr_latched);
        
        // Change address while transparent
        addr_in = 16'hB000;
        #10;
        $display("Input: 0x%04h, Output: 0x%04h, Latched: %b", addr_in, addr_out, addr_latched);
        #10;
        
        // Test 2: Start memory cycle (latch should hold address)
        $display("Test 2: Memory cycle - address should be latched");
        mem_cycle_start = 1;
        #10;
        mem_cycle_start = 0;
        $display("Memory cycle started - Input: 0x%04h, Output: 0x%04h, Latched: %b", 
                 addr_in, addr_out, addr_latched);
        
        // Try changing input address during memory cycle
        addr_in = 16'hC000;
        #10;
        $display("Changed input during cycle - Input: 0x%04h, Output: 0x%04h, Latched: %b", 
                 addr_in, addr_out, addr_latched);
        
        addr_in = 16'hD000;
        #10;
        $display("Changed input again - Input: 0x%04h, Output: 0x%04h, Latched: %b", 
                 addr_in, addr_out, addr_latched);
        
        // End memory cycle
        mem_cycle_end = 1;
        #10;
        mem_cycle_end = 0;
        $display("Memory cycle ended - Input: 0x%04h, Output: 0x%04h, Latched: %b", 
                 addr_in, addr_out, addr_latched);
        #10;
        
        // Test 3: Address invalid during transparent mode
        $display("Test 3: Address invalid - latch should not change");
        addr_valid = 0;
        addr_in = 16'hE000;
        #10;
        $display("Address invalid - Input: 0x%04h, Output: 0x%04h, Latched: %b", 
                 addr_in, addr_out, addr_latched);
        
        // Make address valid again
        addr_valid = 1;
        #10;
        $display("Address valid again - Input: 0x%04h, Output: 0x%04h, Latched: %b", 
                 addr_in, addr_out, addr_latched);
        #10;
        
        // Test 4: Multiple memory cycles
        $display("Test 4: Multiple memory access cycles");
        for (int i = 0; i < 3; i++) begin
            // Set new address
            addr_in = 16'h1000 + 16'(i * 16'h100);
            #10;
            $display("Cycle %0d: Set address 0x%04h", i+1, addr_in);
            
            // Start memory cycle
            mem_cycle_start = 1;
            #10;
            mem_cycle_start = 0;
            $display("  Memory cycle started - Output: 0x%04h, Latched: %b", addr_out, addr_latched);
            
            // Simulate memory access time
            #30;
            
            // End memory cycle
            mem_cycle_end = 1;
            #10;
            mem_cycle_end = 0;
            $display("  Memory cycle ended - Output: 0x%04h, Latched: %b", addr_out, addr_latched);
            #10;
        end
        
        // Test 5: Address changes during non-valid periods
        $display("Test 5: Address changes when addr_valid is low");
        addr_valid = 0;
        for (int i = 0; i < 3; i++) begin
            addr_in = 16'hF000 + 16'(i * 16'h10);
            #10;
            $display("Invalid addr change %0d: Input: 0x%04h, Output: 0x%04h", 
                     i+1, addr_in, addr_out);
        end
        
        // Make valid and check final value
        addr_valid = 1;
        #10;
        $display("Made valid: Input: 0x%04h, Output: 0x%04h", addr_in, addr_out);
        
        $display();
        $display("=== All tests completed ===");
        $display();
        
        #50 $finish;
    end
    
    // Monitor for debugging
    always @(addr_out) begin
        if ($time > 0) begin
            $display("Time %0t: Address output changed to 0x%04h", $time, addr_out);
        end
    end

endmodule
```

Verilator Simulation Output:
Address Latch initialized - Transparent latch for memory access

=== Address Latch Test ===

Test 1: Transparent latch operation
Time 30: Address output changed to 0xa000
Input: 0xa000, Output: 0xa000, Latched: 0
Time 40: Address output changed to 0xb000
Input: 0xb000, Output: 0xb000, Latched: 0
Test 2: Memory cycle - address should be latched
Time 60: Address latch - Memory cycle started, address 0xb000 latched
Memory cycle started - Input: 0xb000, Output: 0xb000, Latched: 1
Changed input during cycle - Input: 0xc000, Output: 0xb000, Latched: 1
Changed input again - Input: 0xd000, Output: 0xb000, Latched: 1
Time 90: Address latch - Memory cycle ended, latch now transparent
Time 95: Address output changed to 0xd000
Memory cycle ended - Input: 0xd000, Output: 0xd000, Latched: 0
Test 3: Address invalid - latch should not change
Address invalid - Input: 0xe000, Output: 0xd000, Latched: 0
Time 120: Address output changed to 0xe000
Address valid again - Input: 0

0

#### Data Bus Latch
Bidirectional data latch for bus isolation and timing control

In [43]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_6_examples/example_16__data_bus_latch/"
files = ["data_bus_latch.sv", "data_bus_latch_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// data_bus_latch.sv - Fixed version without circular logic
module data_bus_latch (
    input  logic        clk,
    input  logic        rst_n,
    
    // Control signals
    input  logic        latch_enable,      // Enable latch operation
    input  logic        cpu_to_bus_dir,    // Direction: 1=CPU->Bus, 0=Bus->CPU
    input  logic        bus_output_enable, // Enable bus output drivers
    
    // CPU side interface
    inout  logic [7:0]  cpu_data,          // Bidirectional CPU data bus
    
    // External bus interface  
    inout  logic [7:0]  ext_bus_data,      // Bidirectional external data bus
    
    // Status outputs
    output logic        latch_active,      // Latch is currently active
    output logic        bus_isolated       // Bus is isolated (high-Z)
);

    // Internal latched data storage
    logic [7:0] cpu_to_bus_latch;    // Latch for CPU->Bus direction
    logic [7:0] bus_to_cpu_latch;    // Latch for Bus->CPU direction
    
    // Previous latch enable states for edge detection
    logic prev_latch_enable;
    logic prev_cpu_to_bus_dir;
    
    // Control signals for driving the buses
    logic drive_cpu_bus;
    logic drive_ext_bus;
    
    // Update previous states on clock edge
    always_ff @(posedge clk or negedge rst_n) begin
        if (!rst_n) begin
            prev_latch_enable <= 1'b0;
            prev_cpu_to_bus_dir <= 1'b0;
        end else begin
            prev_latch_enable <= latch_enable;
            prev_cpu_to_bus_dir <= cpu_to_bus_dir;
        end
    end
    
    // Latch data on the rising edge of latch_enable
    always_ff @(posedge clk or negedge rst_n) begin
        if (!rst_n) begin
            cpu_to_bus_latch <= 8'h00;
            bus_to_cpu_latch <= 8'h00;
        end else begin
            // Latch on rising edge of latch_enable
            if (latch_enable && !prev_latch_enable) begin
                if (cpu_to_bus_dir) begin
                    // CPU->Bus: latch CPU data
                    cpu_to_bus_latch <= cpu_data;
                    $display(
                        "Time %0t: Data Bus Latch - CPU->Bus direction, latching CPU data: 0x%02h", 
                        $time, cpu_data);
                end else begin
                    // Bus->CPU: latch external bus data
                    bus_to_cpu_latch <= ext_bus_data;
                    $display(
                        "Time %0t: Data Bus Latch - Bus->CPU direction, latching Bus data: 0x%02h", 
                        $time, ext_bus_data);
                end
            end
        end
    end
    
    // Drive control logic - only drive when latch is active and output is enabled
    assign drive_ext_bus = latch_enable && cpu_to_bus_dir && bus_output_enable;
    assign drive_cpu_bus = latch_enable && !cpu_to_bus_dir && bus_output_enable;
    
    // Bidirectional bus control
    assign cpu_data = drive_cpu_bus ? bus_to_cpu_latch : 8'bz;
    assign ext_bus_data = drive_ext_bus ? cpu_to_bus_latch : 8'bz;
    
    // Status signals
    assign latch_active = latch_enable;
    assign bus_isolated = !bus_output_enable;
    
    // Debug output for bus driving
    always @(posedge bus_output_enable) begin
        if (latch_enable) begin
            if (cpu_to_bus_dir) begin
                $display(
                    "Time %0t: Data Bus Latch - Driving ext_bus with latched data: 0x%02h", 
                    $time, cpu_to_bus_latch);
            end else begin
                $display(
                    "Time %0t: Data Bus Latch - Driving cpu_data with latched data: 0x%02h", 
                    $time, bus_to_cpu_latch);
            end
        end
    end
    
    always @(negedge bus_output_enable) begin
        $display(
            "Time %0t: Data Bus Latch - Output disabled, buses isolated",
            $time);
    end
    
    initial $display(
        "Data Bus Latch initialized - Bidirectional data isolation and timing control");

endmodule
```

```systemverilog
// data_bus_latch_testbench.sv
module data_bus_latch_testbench;

    // Clock and reset
    logic clk;
    logic rst_n;
    
    // Control signals
    logic latch_enable;
    logic cpu_to_bus_dir;
    logic bus_output_enable;
    
    // Data buses
    wire [7:0] cpu_data;
    wire [7:0] ext_bus_data;
    
    // Status signals
    logic latch_active;
    logic bus_isolated;
    
    // Test drivers
    logic [7:0] cpu_data_driver;
    logic [7:0] ext_bus_data_driver;
    logic       cpu_drive_enable;
    logic       ext_bus_drive_enable;
    
    // Drive the bidirectional buses
    assign cpu_data = cpu_drive_enable ? cpu_data_driver : 8'bz;
    assign ext_bus_data = ext_bus_drive_enable ? ext_bus_data_driver : 8'bz;
    
    // Instantiate the data bus latch
    data_bus_latch DATA_BUS_LATCH_INST (
        .clk(clk),
        .rst_n(rst_n),
        .latch_enable(latch_enable),
        .cpu_to_bus_dir(cpu_to_bus_dir),
        .bus_output_enable(bus_output_enable),
        .cpu_data(cpu_data),
        .ext_bus_data(ext_bus_data),
        .latch_active(latch_active),
        .bus_isolated(bus_isolated)
    );
    
    // Clock generation
    initial begin
        clk = 0;
        forever #5 clk = ~clk;  // 10ns clock period
    end
    
    // Helper task to display bus states
    task display_bus_state(string test_name);
        $display("%s - CPU: 0x%02h, ExtBus: 0x%02h, LatchActive: %b, Isolated: %b", 
                 test_name, cpu_data, ext_bus_data, latch_active, bus_isolated);
    endtask
    
    // Test sequence
    initial begin
        // Initialize signals
        rst_n = 0;
        latch_enable = 0;
        cpu_to_bus_dir = 0;
        bus_output_enable = 0;
        cpu_data_driver = 8'h00;
        ext_bus_data_driver = 8'h00;
        cpu_drive_enable = 0;
        ext_bus_drive_enable = 0;
        
        // Dump waves
        $dumpfile("data_bus_latch_testbench.vcd");
        $dumpvars(0, data_bus_latch_testbench);
        
        $display();
        $display("=== Data Bus Latch Test ===");
        $display();
        
        // Release reset
        #20 rst_n = 1;
        #10;
        
        // Test 1: CPU to Bus transfer
        $display("Test 1: CPU to Bus data transfer");
        cpu_data_driver = 8'hAA;
        cpu_drive_enable = 1;
        cpu_to_bus_dir = 1;  // CPU->Bus direction
        #10;
        display_bus_state("  Initial state");
        
        // Enable latch (should latch CPU data)
        latch_enable = 1;
        #10;
        display_bus_state("  Latch enabled");
        
        // Change CPU data (should not affect latched value)
        cpu_data_driver = 8'hFF;
        #10;
        display_bus_state("  CPU data changed");
        
        // Enable bus output (should drive external bus)
        bus_output_enable = 1;
        #10;
        display_bus_state("  Bus output enabled");
        
        // Disable CPU driver to see latch driving bus
        cpu_drive_enable = 0;
        #10;
        display_bus_state("  CPU driver disabled");
        
        // Cleanup
        bus_output_enable = 0;
        latch_enable = 0;
        #20;
        
        // Test 2: Bus to CPU transfer
        $display("Test 2: Bus to CPU data transfer");
        ext_bus_data_driver = 8'h55;
        ext_bus_drive_enable = 1;
        cpu_to_bus_dir = 0;  // Bus->CPU direction
        #10;
        display_bus_state("  Initial state");
        
        // Enable latch (should latch bus data)
        latch_enable = 1;
        #10;
        display_bus_state("  Latch enabled");
        
        // Change bus data (should not affect latched value)
        ext_bus_data_driver = 8'h33;
        #10;
        display_bus_state("  Bus data changed");
        
        // Enable bus output (should drive CPU bus)
        bus_output_enable = 1;
        #10;
        display_bus_state("  Bus output enabled");
        
        // Disable external bus driver to see latch driving CPU
        ext_bus_drive_enable = 0;
        #10;
        display_bus_state("  Ext bus driver disabled");
        
        // Cleanup
        bus_output_enable = 0;
        latch_enable = 0;
        #20;
        
        // Test 3: Bus isolation
        $display("Test 3: Bus isolation test");
        cpu_data_driver = 8'hCC;
        ext_bus_data_driver = 8'h99;
        cpu_drive_enable = 1;
        ext_bus_drive_enable = 1;
        #10;
        display_bus_state("  Both buses driven");
        
        // Enable latch but keep output disabled (isolation)
        latch_enable = 1;
        cpu_to_bus_dir = 1;
        #10;
        display_bus_state("  Latch enabled, output disabled");
        
        // Disable external drivers
        cpu_drive_enable = 0;
        ext_bus_drive_enable = 0;
        #10;
        display_bus_state("  All drivers disabled (isolated)");
        
        // Enable output
        bus_output_enable = 1;
        #10;
        display_bus_state("  Output enabled");
        
        // Cleanup
        bus_output_enable = 0;
        latch_enable = 0;
        #20;
        
        // Test 4: Multiple transfers with different data
        $display("Test 4: Multiple transfers");
        for (int i = 0; i < 4; i++) begin
            logic [7:0] test_data = 8'h10 + 8'(i * 8'h11);
            
            $display("  Transfer %0d: Data 0x%02h", i+1, test_data);
            
            // Setup data
            if (i % 2 == 0) begin
                // CPU to Bus
                cpu_data_driver = test_data;
                cpu_drive_enable = 1;
                ext_bus_drive_enable = 0;
                cpu_to_bus_dir = 1;
            end else begin
                // Bus to CPU
                ext_bus_data_driver = test_data;
                ext_bus_drive_enable = 1;
                cpu_drive_enable = 0;
                cpu_to_bus_dir = 0;
            end
            
            #10;
            
            // Latch and transfer
            latch_enable = 1;
            #10;
            bus_output_enable = 1;
            #10;
            display_bus_state("    Transfer complete");
            
            // Cleanup
            bus_output_enable = 0;
            latch_enable = 0;
            cpu_drive_enable = 0;
            ext_bus_drive_enable = 0;
            #10;
        end
        
        // Test 5: Timing control demonstration
        $display("Test 5: Timing control demonstration");
        cpu_data_driver = 8'hDE;
        cpu_drive_enable = 1;
        cpu_to_bus_dir = 1;
        
        $display("  Phase 1: Setup data");
        #10;
        display_bus_state("    Data setup");
        
        $display("  Phase 2: Latch data");
        latch_enable = 1;
        #10;
        display_bus_state("    Data latched");
        
        $display("  Phase 3: Remove source data");
        cpu_drive_enable = 0;
        #10;
        display_bus_state("    Source removed");
        
        $display("  Phase 4: Drive output");
        bus_output_enable = 1;
        #10;
        display_bus_state("    Output driving");
        
        $display("  Phase 5: Complete transfer");
        bus_output_enable = 0;
        latch_enable = 0;
        #10;
        display_bus_state("    Transfer complete");
        
        $display();
        $display("=== All tests completed ===");
        $display();
        
        #50 $finish;
    end

endmodule
```

Verilator Simulation Output:
Data Bus Latch initialized - Bidirectional data isolation and timing control

=== Data Bus Latch Test ===

Test 1: CPU to Bus data transfer
  Initial state - CPU: 0xaa, ExtBus: 0x00, LatchActive: 0, Isolated: 1
Time 45: Data Bus Latch - CPU->Bus direction, latching CPU data: 0xaa
  Latch enabled - CPU: 0xaa, ExtBus: 0x00, LatchActive: 1, Isolated: 1
  CPU data changed - CPU: 0xff, ExtBus: 0x00, LatchActive: 1, Isolated: 1
Time 60: Data Bus Latch - Driving ext_bus with latched data: 0xaa
  Bus output enabled - CPU: 0xff, ExtBus: 0xaa, LatchActive: 1, Isolated: 0
  CPU driver disabled - CPU: 0x00, ExtBus: 0xaa, LatchActive: 1, Isolated: 0
Time 80: Data Bus Latch - Output disabled, buses isolated
Test 2: Bus to CPU data transfer
  Initial state - CPU: 0x00, ExtBus: 0x55, LatchActive: 0, Isolated: 1
Time 115: Data Bus Latch - Bus->CPU direction, latching Bus data: 0x55
  Latch enabled - CPU: 0x00, ExtBus: 0x55, LatchActive: 1, Isolated: 1
  Bus data changed - C

0

#### Clock Gating Latch
Level-sensitive latch used in clock gating circuits (with warnings about usage)

In [44]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_6_examples/example_17__clock_gating_latch/"
files = ["clock_gating_latch.sv", "clock_gating_latch_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// clock_gating_latch.sv
// WARNING: Clock gating latches are advanced power management techniques
// that require careful analysis and should only be used by experienced designers
// Improper use can cause timing violations, glitches, and functional failures

module clock_gating_latch (
    input  logic clk,           // Original clock signal
    input  logic enable,        // Clock enable signal
    input  logic test_mode,     // Test mode bypass (for DFT)
    output logic gated_clk      // Gated clock output
);

    // Level-sensitive latch for clock gating
    // The latch is transparent when clk is LOW
    logic enable_latched;
    
    // Clock gating latch - enables clock when enable is high
    // WARNING: This creates a level-sensitive latch which can cause:
    // 1. Timing violations if enable changes during clock high period
    // 2. Glitches on the gated clock
    // 3. Difficulty in static timing analysis
    
    /* verilator lint_off LATCH */
    // This is an intentional latch for clock gating purposes
    always_latch begin
        if (~clk)  // Latch is transparent when clock is low
            enable_latched = enable | test_mode;  // Include test mode for DFT
        // When clk is high, enable_latched holds its previous value (latch behavior)
    end
    /* verilator lint_on LATCH */
    
    // Generate gated clock
    // WARNING: This AND gate can create glitches if enable_latched
    // transitions during clk high period
    assign gated_clk = clk & enable_latched;
    
    // Alternative safer implementation would use dedicated clock gating cells
    // provided by the technology library, which include built-in glitch protection
    
    initial begin
        $display("=== CLOCK GATING LATCH WARNINGS ===");
        $display("1. This is a simplified educational example");
        $display("2. Real designs should use library-provided clock gating cells");
        $display("3. Requires careful static timing analysis");
        $display("4. Enable signal must be stable during clock high period");
        $display("5. Can cause functional and timing issues if misused");
        $display("=====================================");
    end

endmodule
```

```systemverilog
// clock_gating_latch_testbench.sv
module clock_gating_testbench;

    // Testbench signals
    logic clk;
    logic enable;
    logic test_mode;
    logic gated_clk;
    
    // Instantiate the clock gating latch
    clock_gating_latch CLOCK_GATE_INSTANCE (
        .clk(clk),
        .enable(enable),
        .test_mode(test_mode),
        .gated_clk(gated_clk)
    );
    
    // Clock generation - 10ns period (100MHz)
    initial begin
        clk = 0;
        forever #5 clk = ~clk;
    end
    
    // Test sequence
    initial begin
        // Dump waves for analysis
        $dumpfile("clock_gating_testbench.vcd");
        $dumpvars(0, clock_gating_testbench);
        
        // Initialize signals
        enable = 0;
        test_mode = 0;
        
        $display("\n=== Clock Gating Latch Test ===");
        
        // Test 1: Clock gating disabled (enable = 0)
        $display("\nTest 1: Clock gating - enable = 0");
        #20;
        $display("Time %0t: clk=%b, enable=%b, gated_clk=%b", $time, clk, enable, gated_clk);
        
        // Test 2: Enable clock gating (enable = 1)
        $display("\nTest 2: Clock gating - enable = 1");
        @(negedge clk);  // Change enable during clock low (safe)
        enable = 1;
        #30;
        $display("Time %0t: clk=%b, enable=%b, gated_clk=%b", $time, clk, enable, gated_clk);
        
        // Test 3: Disable clock gating again
        $display("\nTest 3: Clock gating - enable = 0 again");
        @(negedge clk);  // Change enable during clock low (safe)
        enable = 0;
        #20;
        $display("Time %0t: clk=%b, enable=%b, gated_clk=%b", $time, clk, enable, gated_clk);
        
        // Test 4: Test mode bypass
        $display("\nTest 4: Test mode bypass - test_mode = 1");
        @(negedge clk);
        test_mode = 1;
        #20;
        $display("Time %0t: clk=%b, enable=%b, test_mode=%b, gated_clk=%b", 
                 $time, clk, enable, test_mode, gated_clk);
        
        // Test 5: Demonstrate DANGEROUS behavior - changing enable during clock high
        $display("\nTest 5: enable during clock HIGH period");
        @(negedge clk);
        test_mode = 0;
        enable = 1;
        #7;  // Wait until middle of clock high period
        if (clk) begin
            $display("WARNING: Changing enable during clock HIGH - can cause glitches!");
            enable = 0;  // This is dangerous and can cause glitches
            #1;
            $display("Time %0t: clk=%b, enable=%b, gated_clk=%b (POTENTIAL GLITCH)", 
                     $time, clk, enable, gated_clk);
        end
        
        #30;
        
        $display("\n=== Test Complete ===");
        $display("Check VCD file for detailed timing analysis");
        $display("Notice how gated_clk follows the enable signal properly");
        $display("when enable changes during clock LOW period (safe)");
        $finish;
    end
    
    // Monitor for educational purposes
    always @(posedge gated_clk) begin
        $display("Time %0t: Gated clock rising edge detected", $time);
    end

endmodule
```

Verilator Simulation Output:
=== CLOCK GATING LATCH WARNINGS ===
1. This is a simplified educational example
2. Real designs should use library-provided clock gating cells
3. Requires careful static timing analysis
4. Enable signal must be stable during clock high period
5. Can cause functional and timing issues if misused

=== Clock Gating Latch Test ===

Test 1: Clock gating - enable = 0
Time 20: clk=1, enable=0, gated_clk=0

Test 2: Clock gating - enable = 1
Time 35: Gated clock rising edge detected
Time 45: Gated clock rising edge detected
Time 55: Gated clock rising edge detected
Time 60: clk=1, enable=1, gated_clk=1

Test 3: Clock gating - enable = 0 again
Time 65: Gated clock rising edge detected
Time 90: clk=1, enable=0, gated_clk=0

Test 4: Test mode bypass - test_mode = 1
Time 105: Gated clock rising edge detected
Time 115: Gated clock rising edge detected
Time 120: clk=1, enable=0, test_mode=1, gated_clk=1

Test 5: enable during clock HIGH period
Time 125: Gated clock rising

0

## Blocking vs. Non-Blocking Assignments

#### Multi Stage Pipeline
Multi-stage pipeline showing correct vs. incorrect assignment usage

In [45]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_6_examples/example_18__multi_stage_pipeline/"
files = ["multi_stage_pipeline.sv", "multi_stage_pipeline_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// multi_stage_pipeline.sv
// Demonstrates correct vs incorrect assignment usage in multi-stage pipelines
// This example shows a 4-stage arithmetic pipeline: INPUT -> ADD -> MULTIPLY -> OUTPUT

module multi_stage_pipeline (
    input  logic        clk,
    input  logic        reset_n,
    input  logic        enable,
    input  logic [7:0]  data_in_a,
    input  logic [7:0]  data_in_b,
    input  logic [7:0]  data_in_c,
    output logic [15:0] result_correct,
    output logic [15:0] result_incorrect,
    output logic        valid_out
);

    // ========================================
    // CORRECT PIPELINE IMPLEMENTATION
    // Using non-blocking assignments (<=)
    // ========================================
    
    // Stage 1: Input registers
    logic [7:0] stage1_a, stage1_b, stage1_c;
    logic       stage1_valid;
    
    // Stage 2: Addition stage
    logic [8:0] stage2_sum;        // a + b (9 bits to handle overflow)
    logic [7:0] stage2_c;
    logic       stage2_valid;
    
    // Stage 3: Multiplication stage  
    logic [15:0] stage3_product;   // sum * c
    logic        stage3_valid;
    
    // CORRECT: Non-blocking assignments for sequential logic
    always_ff @(posedge clk or negedge reset_n) begin
        if (~reset_n) begin
            // Reset all pipeline stages
            stage1_a <= 8'b0;
            stage1_b <= 8'b0;
            stage1_c <= 8'b0;
            stage1_valid <= 1'b0;
            
            stage2_sum <= 9'b0;
            stage2_c <= 8'b0;
            stage2_valid <= 1'b0;
            
            stage3_product <= 16'b0;
            stage3_valid <= 1'b0;
            
            result_correct <= 16'b0;
            valid_out <= 1'b0;
        end else if (enable) begin
            // Stage 1: Input capture
            stage1_a <= data_in_a;
            stage1_b <= data_in_b;
            stage1_c <= data_in_c;
            stage1_valid <= 1'b1;
            
            // Stage 2: Addition (a + b)
            stage2_sum <= stage1_a + stage1_b;
            stage2_c <= stage1_c;
            stage2_valid <= stage1_valid;
            
            // Stage 3: Multiplication (sum * c)
            stage3_product <= stage2_sum * stage2_c;
            stage3_valid <= stage2_valid;
            
            // Stage 4: Output
            result_correct <= stage3_product;
            valid_out <= stage3_valid;
        end
    end

    // ========================================
    // INCORRECT PIPELINE IMPLEMENTATION
    // Using blocking assignments (=) - DEMONSTRATES PROBLEMS
    // ========================================
    
    // Incorrect pipeline registers
    logic [7:0] bad_stage1_a, bad_stage1_b, bad_stage1_c;
    logic [8:0] bad_stage2_sum;
    logic [7:0] bad_stage2_c;
    logic [15:0] bad_stage3_product;
    
    // INCORRECT: Blocking assignments cause race conditions and incorrect behavior
    always_ff @(posedge clk or negedge reset_n) begin
        if (~reset_n) begin
            bad_stage1_a = 8'b0;
            bad_stage1_b = 8'b0;
            bad_stage1_c = 8'b0;
            bad_stage2_sum = 9'b0;
            bad_stage2_c = 8'b0;
            bad_stage3_product = 16'b0;
            result_incorrect = 16'b0;
        end else if (enable) begin
            // PROBLEM: All assignments execute immediately in order
            // This creates a combinational path through all stages!
            bad_stage1_a = data_in_a;           // Executes first
            bad_stage1_b = data_in_b;           // Executes second  
            bad_stage1_c = data_in_c;           // Executes third
            
            bad_stage2_sum = bad_stage1_a + bad_stage1_b;  // Uses NEW values immediately!
            bad_stage2_c = bad_stage1_c;                   // Uses NEW value immediately!
            
            bad_stage3_product = bad_stage2_sum * bad_stage2_c;  // Uses NEW values immediately!
            
            result_incorrect = bad_stage3_product;         // Final result in same clock!
        end
    end
    
    // Display warnings about the incorrect implementation
    initial begin
        $display("=== PIPELINE ASSIGNMENT USAGE DEMO ===");
        $display("CORRECT:   Uses non-blocking assignments (<=) for proper pipeline behavior");
        $display("INCORRECT: Uses blocking assignments (=) causing combinational behavior");
        $display("=> Compare result_correct vs result_incorrect in simulation");
        $display("=> Incorrect version computes result in single clock cycle!");
        $display("=> Correct version takes 4 clock cycles for first result");
        $display("=======================================");
    end

endmodule
```

```systemverilog
// multi_stage_pipeline_testbench.sv
module pipeline_testbench;

    // Testbench signals
    logic        clk;
    logic        reset_n;
    logic        enable;
    logic [7:0]  data_in_a;
    logic [7:0]  data_in_b;
    logic [7:0]  data_in_c;
    logic [15:0] result_correct;
    logic [15:0] result_incorrect;
    logic        valid_out;
    
    // Test data
    logic [7:0] test_a [0:4] = '{5, 10, 3, 8, 12};
    logic [7:0] test_b [0:4] = '{3, 2,  7, 4, 1};
    logic [7:0] test_c [0:4] = '{2, 4,  3, 5, 6};
    
    // Expected results: (a + b) * c
    logic [15:0] expected [0:4] = '{16, 48, 30, 60, 78}; // (5+3)*2, (10+2)*4, (3+7)*3, (8+4)*5, (12+1)*6
    
    int test_index = 0;
    int cycle_count = 0;
    
    // Instantiate the multi-stage pipeline
    multi_stage_pipeline PIPELINE_INSTANCE (
        .clk(clk),
        .reset_n(reset_n),
        .enable(enable),
        .data_in_a(data_in_a),
        .data_in_b(data_in_b),
        .data_in_c(data_in_c),
        .result_correct(result_correct),
        .result_incorrect(result_incorrect),
        .valid_out(valid_out)
    );
    
    // Clock generation - 10ns period
    initial begin
        clk = 0;
        forever #5 clk = ~clk;
    end
    
    // Test sequence
    initial begin
        // Dump waves for analysis
        $dumpfile("pipeline_testbench.vcd");
        $dumpvars(0, pipeline_testbench);
        
        // Initialize
        reset_n = 0;
        enable = 0;
        data_in_a = 0;
        data_in_b = 0;
        data_in_c = 0;
        
        $display("\n=== Multi-Stage Pipeline Test ===");
        $display("Pipeline: INPUT -> ADD -> MULTIPLY -> OUTPUT");
        $display("Operation: result = (a + b) * c");
        
        // Reset
        repeat(2) @(posedge clk);
        reset_n = 1;
        enable = 1;
        
        $display("\n--- Feeding Test Data ---");
        
        // Feed test data into pipeline
        for (int i = 0; i < 5; i++) begin
            @(posedge clk);
            data_in_a = test_a[i];
            data_in_b = test_b[i];
            data_in_c = test_c[i];
            cycle_count++;
            
            $display("Cycle %0d: Input a=%0d, b=%0d, c=%0d (expected result = %0d)", 
                     cycle_count, data_in_a, data_in_b, data_in_c, expected[i]);
            $display("         Immediate results: correct=%0d, incorrect=%0d", 
                     result_correct, result_incorrect);
        end
        
        // Continue running to see pipeline results
        $display("\n--- Pipeline Results ---");
        for (int i = 0; i < 8; i++) begin
            @(posedge clk);
            cycle_count++;
            
            if (valid_out) begin
                $display("Cycle %0d: VALID OUTPUT -> correct=%0d, incorrect=%0d", 
                         cycle_count, result_correct, result_incorrect);
                
                // Check if correct result matches expected
                if (test_index < 5 && result_correct == expected[test_index]) begin
                    $display("         âœ“ Correct pipeline result matches expected %0d", expected[test_index]);
                end else if (test_index < 5) begin
                    $display("         âœ— Correct pipeline result %0d != expected %0d", result_correct, expected[test_index]);
                end
                test_index++;
            end else begin
                $display("Cycle %0d: Pipeline filling... correct=%0d, incorrect=%0d", 
                         cycle_count, result_correct, result_incorrect);
            end
        end
        
        $display("\n--- Analysis Summary ---");
        $display("INCORRECT Implementation (blocking =):");
        $display("- All computation happens in same clock cycle");
        $display("- Creates combinational path through entire pipeline");
        $display("- No pipeline latency - result appears immediately");
        $display("- Defeats the purpose of pipelining");
        
        $display("\nCORRECT Implementation (non-blocking <=):");
        $display("- Each stage operates independently");
        $display("- Takes 4 clock cycles for first result (pipeline latency)");
        $display("- Subsequent results appear every clock cycle (throughput)");
        $display("- Proper pipeline behavior for high-performance designs");
        
        $display("\n=== Test Complete ===");
        $finish;
    end
    
    // Monitor pipeline stages for educational purposes
    always @(posedge clk) begin
        if (reset_n && enable) begin
            $display("Stage Values: S1_a=%0d S1_b=%0d S1_c=%0d | S2_sum=%0d S2_c=%0d | S3_prod=%0d", 
                     PIPELINE_INSTANCE.stage1_a, PIPELINE_INSTANCE.stage1_b, PIPELINE_INSTANCE.stage1_c,
                     PIPELINE_INSTANCE.stage2_sum, PIPELINE_INSTANCE.stage2_c, 
                     PIPELINE_INSTANCE.stage3_product);
        end
    end

endmodule
```

Verilator Simulation Output:
=== PIPELINE ASSIGNMENT USAGE DEMO ===
CORRECT:   Uses non-blocking assignments (<=) for proper pipeline behavior
INCORRECT: Uses blocking assignments (=) causing combinational behavior
=> Compare result_correct vs result_incorrect in simulation
=> Incorrect version computes result in single clock cycle!
=> Correct version takes 4 clock cycles for first result

=== Multi-Stage Pipeline Test ===
Pipeline: INPUT -> ADD -> MULTIPLY -> OUTPUT
Operation: result = (a + b) * c

--- Feeding Test Data ---
Stage Values: S1_a=0 S1_b=0 S1_c=0 | S2_sum=0 S2_c=0 | S3_prod=0
Cycle 1: Input a=5, b=3, c=2 (expected result = 16)
         Immediate results: correct=0, incorrect=0
Stage Values: S1_a=0 S1_b=0 S1_c=0 | S2_sum=0 S2_c=0 | S3_prod=0
Cycle 2: Input a=10, b=2, c=4 (expected result = 48)
         Immediate results: correct=0, incorrect=16
Stage Values: S1_a=5 S1_b=3 S1_c=2 | S2_sum=0 S2_c=0 | S3_prod=0
Cycle 3: Input a=3, b=7, c=3 (expected result = 30)
         Immed

0

#### Shift Register Comparison
Side-by-side comparison of blocking vs. non-blocking in shift registers

In [46]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_6_examples/example_19__shift_register_comparison/"
files = [
    "shift_register_comparison.sv",
    "shift_register_comparison_testbench.sv",
]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// shift_register_comparison.sv
// Demonstrates critical differences between blocking and non-blocking assignments
// in shift register implementations - side-by-side comparison

module shift_register_comparison (
    input  logic       clk,
    input  logic       reset_n,
    input  logic       shift_enable,
    input  logic       serial_in,
    output logic [7:0] parallel_out_correct,
    output logic [7:0] parallel_out_incorrect,
    output logic       serial_out_correct,
    output logic       serial_out_incorrect
);

    // ========================================
    // CORRECT SHIFT REGISTER IMPLEMENTATION
    // Using non-blocking assignments (<=)
    // ========================================
    
    logic [7:0] shift_reg_correct;
    
    // CORRECT: Non-blocking assignments create proper shift register
    always_ff @(posedge clk or negedge reset_n) begin
        if (~reset_n) begin
            shift_reg_correct <= 8'b0;
        end else if (shift_enable) begin
            // All assignments happen simultaneously at clock edge
            shift_reg_correct[7] <= serial_in;           // New bit in
            shift_reg_correct[6] <= shift_reg_correct[7]; // Bit 7 -> 6
            shift_reg_correct[5] <= shift_reg_correct[6]; // Bit 6 -> 5
            shift_reg_correct[4] <= shift_reg_correct[5]; // Bit 5 -> 4
            shift_reg_correct[3] <= shift_reg_correct[4]; // Bit 4 -> 3
            shift_reg_correct[2] <= shift_reg_correct[3]; // Bit 3 -> 2
            shift_reg_correct[1] <= shift_reg_correct[2]; // Bit 2 -> 1
            shift_reg_correct[0] <= shift_reg_correct[1]; // Bit 1 -> 0
        end
    end
    
    // Outputs for correct implementation
    assign parallel_out_correct = shift_reg_correct;
    assign serial_out_correct = shift_reg_correct[0];

    // ========================================
    // INCORRECT SHIFT REGISTER IMPLEMENTATION  
    // Using blocking assignments (=)
    // ========================================
    
    logic [7:0] shift_reg_incorrect;
    
    // INCORRECT: Blocking assignments create wrong behavior
    always_ff @(posedge clk or negedge reset_n) begin
        if (~reset_n) begin
            shift_reg_incorrect = 8'b0;
        end else if (shift_enable) begin
            // PROBLEM: Assignments execute sequentially!
            // This creates a "shift-through" effect in one clock cycle
            shift_reg_incorrect[7] = serial_in;              // Executes first
            shift_reg_incorrect[6] = shift_reg_incorrect[7]; // Uses NEW value of [7]!
            shift_reg_incorrect[5] = shift_reg_incorrect[6]; // Uses NEW value of [6]!
            shift_reg_incorrect[4] = shift_reg_incorrect[5]; // Uses NEW value of [5]!
            shift_reg_incorrect[3] = shift_reg_incorrect[4]; // Uses NEW value of [4]!
            shift_reg_incorrect[2] = shift_reg_incorrect[3]; // Uses NEW value of [3]!
            shift_reg_incorrect[1] = shift_reg_incorrect[2]; // Uses NEW value of [2]!
            shift_reg_incorrect[0] = shift_reg_incorrect[1]; // Uses NEW value of [1]!
            // Result: serial_in propagates through ALL bits in ONE clock cycle!
        end
    end
    
    // Outputs for incorrect implementation
    assign parallel_out_incorrect = shift_reg_incorrect;
    assign serial_out_incorrect = shift_reg_incorrect[0];
    
    // Alternative correct implementation using concatenation (for reference)
    logic [7:0] shift_reg_concat;
    always_ff @(posedge clk or negedge reset_n) begin
        if (~reset_n) begin
            shift_reg_concat <= 8'b0;
        end else if (shift_enable) begin
            // This is also correct - concatenation with non-blocking assignment
            shift_reg_concat <= {serial_in, shift_reg_concat[7:1]};
        end
    end
    
    // Display educational information
    initial begin
        $display("=== SHIFT REGISTER ASSIGNMENT COMPARISON ===");
        $display("CORRECT (<=):   Each bit shifts one position per clock");
        $display("INCORRECT (=):  Input propagates through ALL bits in one clock");
        $display("=> Watch how serial_in='1' behaves differently:");
        $display("   - Correct: Takes 8 clocks to reach serial_out");
        $display("   - Incorrect: Reaches serial_out in 1 clock!");
        $display("===========================================");
    end

endmodule
```

```systemverilog
// shift_register_comparison_testbench.sv
module shift_register_testbench;

    // Testbench signals
    logic       clk;
    logic       reset_n;
    logic       shift_enable;
    logic       serial_in;
    logic [7:0] parallel_out_correct;
    logic [7:0] parallel_out_incorrect;
    logic       serial_out_correct;
    logic       serial_out_incorrect;
    
    // Test pattern for serial input
    logic test_pattern [0:15] = '{1,0,1,1,0,0,1,0,1,1,1,0,0,1,1,0};
    int pattern_index = 0;
    int cycle_count = 0;
    
    // Instantiate the shift register comparison
    shift_register_comparison SHIFT_REG_INSTANCE (
        .clk(clk),
        .reset_n(reset_n),
        .shift_enable(shift_enable),
        .serial_in(serial_in),
        .parallel_out_correct(parallel_out_correct),
        .parallel_out_incorrect(parallel_out_incorrect),
        .serial_out_correct(serial_out_correct),
        .serial_out_incorrect(serial_out_incorrect)
    );
    
    // Clock generation - 10ns period
    initial begin
        clk = 0;
        forever #5 clk = ~clk;
    end
    
    // Test sequence
    initial begin
        // Dump waves for detailed analysis
        $dumpfile("shift_register_testbench.vcd");
        $dumpvars(0, shift_register_testbench);
        
        // Initialize signals
        reset_n = 0;
        shift_enable = 0;
        serial_in = 0;
        
        $display("\n=== Shift Register Assignment Comparison ===");
        $display("8-bit shift register: MSB <- serial_in ... LSB -> serial_out");
        
        // Reset sequence
        repeat(2) @(posedge clk);
        reset_n = 1;
        shift_enable = 1;
        
        $display("\n--- Test 1: Single '1' bit propagation ---");
        
        // Test 1: Send single '1' bit and observe propagation
        @(posedge clk);
        serial_in = 1;
        cycle_count++;
        $display("Cycle %0d: Input='1' -> Correct=%8b (%0d), Incorrect=%8b (%0d)", 
                 cycle_count, parallel_out_correct, parallel_out_correct, 
                 parallel_out_incorrect, parallel_out_incorrect);
        
        // Follow with zeros to see the '1' propagate
        for (int i = 0; i < 10; i++) begin
            @(posedge clk);
            serial_in = 0;
            cycle_count++;
            $display("Cycle %0d: Input='0' -> Correct=%8b (%0d), Incorrect=%8b (%0d)", 
                     cycle_count, parallel_out_correct, parallel_out_correct, 
                     parallel_out_incorrect, parallel_out_incorrect);
        end
        
        $display("\n--- Test 2: Pattern shifting ---");
        
        // Reset for pattern test
        @(posedge clk);
        reset_n = 0;
        @(posedge clk);
        reset_n = 1;
        cycle_count = 0;
        
        // Shift in test pattern
        for (int i = 0; i < 16; i++) begin
            @(posedge clk);
            serial_in = test_pattern[i];
            cycle_count++;
            
            $display("Cycle %0d: Input='%0d' -> Correct=%8b, Incorrect=%8b", 
                     cycle_count, serial_in, parallel_out_correct, parallel_out_incorrect);
            $display("         Serial Out: Correct='%0d', Incorrect='%0d'", 
                     serial_out_correct, serial_out_incorrect);
        end
        
        $display("\n--- Test 3: Demonstrating the key difference ---");
        
        // Reset and show the critical difference
        @(posedge clk);
        reset_n = 0;
        @(posedge clk);
        reset_n = 1;
        
        // Show what happens when we input '1' after reset
        $display("\nAfter reset, both registers are 00000000");
        $display("Now inputting '1'...");
        
        @(posedge clk);
        serial_in = 1;
        $display("After 1 clock with input='1':");
        $display("  CORRECT:   %8b (1 shifted to bit 7 only)", parallel_out_correct);
        $display("  INCORRECT: %8b (1 propagated through ALL bits!)", parallel_out_incorrect);
        
        @(posedge clk);
        serial_in = 0;
        $display("After 2nd clock with input='0':");
        $display("  CORRECT:   %8b (1 shifted from bit 7 to bit 6)", parallel_out_correct);
        $display("  INCORRECT: %8b (all 0s because 1 'fell out' immediately)", parallel_out_incorrect);
        
        $display("\n--- Analysis Summary ---");
        $display("BLOCKING ASSIGNMENTS (=) PROBLEM:");
        $display("- Execute sequentially within same clock cycle");
        $display("- Each assignment uses the NEW value from previous assignment");
        $display("- Creates 'shift-through' effect - data propagates through ALL stages");
        $display("- Serial input appears at serial output in SAME clock cycle!");
        $display("- NOT a real shift register - just a wire with delay");
        
        $display("\nNON-BLOCKING ASSIGNMENTS (<=) CORRECT:");
        $display("- All assignments scheduled simultaneously");
        $display("- Each assignment uses OLD values from previous clock cycle");
        $display("- Creates proper shift register behavior");
        $display("- Takes 8 clock cycles for serial input to reach serial output");
        $display("- Each bit position stores data for exactly one clock cycle");
        
        $display("\n=== Test Complete ===");
        $display("Examine VCD file to see detailed timing behavior");
        $finish;
    end
    
    // Monitor for educational purposes
    always @(posedge clk) begin
        if (reset_n && shift_enable) begin
            // Show internal register state
            $display("  Internal: Correct[7:0]=%b, Incorrect[7:0]=%b", 
                     SHIFT_REG_INSTANCE.shift_reg_correct, 
                     SHIFT_REG_INSTANCE.shift_reg_incorrect);
        end
    end

endmodule
```

Verilator Simulation Output:
=== SHIFT REGISTER ASSIGNMENT COMPARISON ===
CORRECT (<=):   Each bit shifts one position per clock
INCORRECT (=):  Input propagates through ALL bits in one clock
=> Watch how serial_in='1' behaves differently:
   - Correct: Takes 8 clocks to reach serial_out
   - Incorrect: Reaches serial_out in 1 clock!

=== Shift Register Assignment Comparison ===
8-bit shift register: MSB <- serial_in ... LSB -> serial_out

--- Test 1: Single '1' bit propagation ---
  Internal: Correct[7:0]=00000000, Incorrect[7:0]=00000000
Cycle 1: Input='1' -> Correct=00000000 (0), Incorrect=00000000 (0)
  Internal: Correct[7:0]=00000000, Incorrect[7:0]=00000000
Cycle 2: Input='0' -> Correct=10000000 (128), Incorrect=11111111 (255)
  Internal: Correct[7:0]=10000000, Incorrect[7:0]=11111111
Cycle 3: Input='0' -> Correct=01000000 (64), Incorrect=00000000 (0)
  Internal: Correct[7:0]=01000000, Incorrect[7:0]=00000000
Cycle 4: Input='0' -> Correct=00100000 (32), Incorrect=00000000 (0)
  I

0

#### Multi Level Combinational
Multi-level combinational logic showing assignment timing effects

In [47]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_6_examples/example_20__multi_level_combinational/"
files = [
    "multi_level_combinational.sv",
    "multi_level_combinational_testbench.sv",
]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// multi_level_combinational.sv
// Demonstrates assignment timing effects in multi-level combinational logic
// Shows how blocking vs non-blocking assignments affect intermediate signal propagation

module multi_level_combinational (
    input  logic a,
    input  logic b,
    input  logic c,
    input  logic d,
    output logic result_blocking,
    output logic result_nonblocking,
    output logic result_continuous
);

    // Intermediate signals for multi-level logic
    // Circuit implements: result = ((a & b) | (c & d)) ^ (a | c)
    
    // ========================================
    // METHOD 1: BLOCKING ASSIGNMENTS (=)
    // Sequential execution within always block
    // ========================================
    
    logic level1_and1_block, level1_and2_block, level1_or_block;
    logic level2_or_block;
    logic level3_xor_block;
    
    always_comb begin
        // Level 1: Basic gates (execute in order)
        level1_and1_block = a & b;        // Executes first
        level1_and2_block = c & d;        // Executes second  
        level1_or_block = a | c;          // Executes third
        
        // Level 2: OR of the AND results (uses NEW values immediately)
        level2_or_block = level1_and1_block | level1_and2_block;
        
        // Level 3: Final XOR (uses NEW value immediately)
        level3_xor_block = level2_or_block ^ level1_or_block;
        
        // Final result
        result_blocking = level3_xor_block;
    end

    // ========================================
    // METHOD 2: NON-BLOCKING ASSIGNMENTS (<=)
    // WARNING: This is incorrect for combinational logic!
    // COMMENTED OUT to avoid warnings - demonstrates bad practice
    // ========================================
    
    logic level1_and1_nonblock, level1_and2_nonblock, level1_or_nonblock;
    logic level2_or_nonblock;
    logic level3_xor_nonblock;
    
    // COMMENTED OUT - This code generates warnings because it's incorrect practice
    /*
    always_comb begin
        // INCORRECT: Non-blocking in combinational logic causes issues
        level1_and1_nonblock <= a & b;        // Scheduled for later
        level1_and2_nonblock <= c & d;        // Scheduled for later
        level1_or_nonblock <= a | c;          // Scheduled for later
        
        // PROBLEM: These use OLD values from previous evaluation!
        level2_or_nonblock <= level1_and1_nonblock | level1_and2_nonblock;
        level3_xor_nonblock <= level2_or_nonblock ^ level1_or_nonblock;
        
        result_nonblocking <= level3_xor_nonblock;
    end
    */
    
    // For demonstration purposes, tie nonblocking result to 0
    // In real design, you would never use non-blocking for combinational logic
    assign result_nonblocking = 1'b0;  // Shows this method is "broken"

    // ========================================
    // METHOD 3: CONTINUOUS ASSIGNMENTS (assign)
    // Best practice for combinational logic
    // ========================================
    
    logic level1_and1_cont, level1_and2_cont, level1_or_cont;
    logic level2_or_cont;
    
    // Level 1: Basic operations
    assign level1_and1_cont = a & b;
    assign level1_and2_cont = c & d;
    assign level1_or_cont = a | c;
    
    // Level 2: Combine level 1 results
    assign level2_or_cont = level1_and1_cont | level1_and2_cont;
    
    // Level 3: Final result
    assign result_continuous = level2_or_cont ^ level1_or_cont;
    
    // Alternative: Single continuous assignment (also correct)
    // assign result_continuous = ((a & b) | (c & d)) ^ (a | c);

    // ========================================
    // TIMING ANALYSIS HELPERS
    // ========================================
    
    // Monitor intermediate signals for educational purposes
    always @(*) begin
        $display("Input change detected: a=%b, b=%b, c=%b, d=%b", a, b, c, d);
        $display("  Level 1 - AND1: block=%b, nonblock=%b, cont=%b", 
                 level1_and1_block, level1_and1_nonblock, level1_and1_cont);
        $display("  Level 1 - AND2: block=%b, nonblock=%b, cont=%b", 
                 level1_and2_block, level1_and2_nonblock, level1_and2_cont);
        $display("  Level 1 - OR:   block=%b, nonblock=%b, cont=%b", 
                 level1_or_block, level1_or_nonblock, level1_or_cont);
        $display("  Level 2 - OR:   block=%b, nonblock=%b, cont=%b", 
                 level2_or_block, level2_or_nonblock, level2_or_cont);
        $display("  Final Result:   block=%b, nonblock=%b, cont=%b", 
                 result_blocking, result_nonblocking, result_continuous);
        $display("  Expected Result: %b", ((a & b) | (c & d)) ^ (a | c));
        $display("---");
    end
    
    // Display educational information
    initial begin
        $display("=== MULTI-LEVEL COMBINATIONAL LOGIC TIMING ===");
        $display("Circuit: result = ((a & b) | (c & d)) ^ (a | c)");
        $display("BLOCKING (=):     Correct - sequential execution updates intermediates");
        $display("NON-BLOCKING (<=): Commented out - incorrect for combinational logic");
        $display("CONTINUOUS (assign): Best - pure combinational behavior");
        $display("============================================");
    end

endmodule
```

```systemverilog
// multi_level_combinational_testbench.sv
module multilevel_comb_testbench;

    // Testbench signals
    logic a, b, c, d;
    logic result_blocking;
    logic result_nonblocking; 
    logic result_continuous;
    
    // Expected result calculation
    logic expected_result;
    assign expected_result = ((a & b) | (c & d)) ^ (a | c);
    
    // Test vectors - all 16 possible combinations
    logic [3:0] test_vectors [0:15];
    
    // Instantiate the multi-level combinational logic
    multi_level_combinational MULTILEVEL_INSTANCE (
        .a(a),
        .b(b), 
        .c(c),
        .d(d),
        .result_blocking(result_blocking),
        .result_nonblocking(result_nonblocking),
        .result_continuous(result_continuous)
    );
    
    // Initialize test vectors
    initial begin
        for (int i = 0; i < 16; i++) begin
            test_vectors[i] = i[3:0];
        end
    end
    
    // Test sequence
    initial begin
        // Dump waves for analysis
        $dumpfile("multilevel_comb_testbench.vcd");
        $dumpvars(0, multilevel_comb_testbench);
        
        $display("\n=== Multi-Level Combinational Logic Test ===");
        $display("Circuit: result = ((a & b) | (c & d)) ^ (a | c)");
        $display("Testing all 16 input combinations...\n");
        
        $display("Inputs | Expected | Blocking | NonBlock | Continuous | Status");
        $display("-------|----------|----------|----------|------------|--------");
        
        // Test all combinations
        for (int i = 0; i < 16; i++) begin
            // Apply test vector
            {a, b, c, d} = test_vectors[i];
            
            // Small delay to let combinational logic settle
            #1;
            
            // Check results and display
            $display("%b%b%b%b   |    %b     |    %b     |    %b     |     %b      | %s %s %s",
                     a, b, c, d, expected_result, 
                     result_blocking, result_nonblocking, result_continuous,
                     (result_blocking == expected_result) ? "B_OK" : "B_ERR",
                     (result_nonblocking == expected_result) ? "N_OK" : "N_ERR",
                     (result_continuous == expected_result) ? "C_OK" : "C_ERR");
            
            // Small delay between tests
            #5;
        end
        
        $display("\n--- Detailed Analysis ---");
        
        // Test specific cases to show timing effects
        $display("\nTesting timing behavior with specific transitions:");
        
        // Start with all zeros
        {a, b, c, d} = 4'b0000;
        #1;
        $display("Initial: abcd=0000 -> Block=%b, NonBlock=%b, Cont=%b (Expected=%b)", 
                 result_blocking, result_nonblocking, result_continuous, expected_result);
        
        // Change to case that should produce different result
        {a, b, c, d} = 4'b1100;  // (1&1)|(0&0) ^ (1|0) = 1|0 ^ 1 = 1^1 = 0
        #1;
        $display("After 1100: Block=%b, NonBlock=%b, Cont=%b (Expected=%b)", 
                 result_blocking, result_nonblocking, result_continuous, expected_result);
        
        // Another transition
        {a, b, c, d} = 4'b1010;  // (1&0)|(1&0) ^ (1|1) = 0|0 ^ 1 = 0^1 = 1
        #1; 
        $display("After 1010: Block=%b, NonBlock=%b, Cont=%b (Expected=%b)", 
                 result_blocking, result_nonblocking, result_continuous, expected_result);
        
        $display("\n--- Summary of Assignment Methods ---");
        
        $display("\n1. BLOCKING ASSIGNMENTS (=) in always_comb:");
        $display("   CORRECT for combinational logic"); 
        $display("   Sequential execution ensures intermediate values update first");
        $display("   Each level uses updated values from previous level");
        $display("   Behaves like real hardware propagation delays");
        
        $display("\n2. NON-BLOCKING ASSIGNMENTS (<=) in always_comb:");
        $display("   INCORRECT for combinational logic");
        $display("   Uses stale values from previous evaluation cycle");
        $display("   Can cause incorrect results and race conditions");
        $display("   Creates artificial delay that doesn't match hardware");
        
        $display("\n3. CONTINUOUS ASSIGNMENTS (assign):");
        $display("   BEST PRACTICE for combinational logic");
        $display("   Pure combinational behavior - no procedural timing"); 
        $display("   Clear, readable, and matches hardware exactly");
        $display("   Automatic sensitivity to all inputs");
        
        $display("\n--- Key Insights ---");
        $display("Multi-level combinational logic requires proper signal propagation");
        $display("Blocking assignments (=) work because they execute sequentially");
        $display("Non-blocking (<=) fails because it uses old intermediate values"); 
        $display("Continuous assignments (assign) are the gold standard");
        $display("Always use assign for pure combinational logic when possible");
        
        $display("\n=== Test Complete ===");
        $display("Check VCD for detailed signal transitions and timing");
        $finish;
    end

endmodule
```

Verilator Simulation Output:
=== MULTI-LEVEL COMBINATIONAL LOGIC TIMING ===
Circuit: result = ((a & b) | (c & d)) ^ (a | c)
BLOCKING (=):     Correct - sequential execution updates intermediates
NON-BLOCKING (<=): Commented out - incorrect for combinational logic
CONTINUOUS (assign): Best - pure combinational behavior

=== Multi-Level Combinational Logic Test ===
Circuit: result = ((a & b) | (c & d)) ^ (a | c)
Testing all 16 input combinations...

Inputs | Expected | Blocking | NonBlock | Continuous | Status
-------|----------|----------|----------|------------|--------
Input change detected: a=0, b=0, c=0, d=0
  Level 1 - AND1: block=0, nonblock=0, cont=0
  Level 1 - AND2: block=0, nonblock=0, cont=0
  Level 1 - OR:   block=0, nonblock=0, cont=0
  Level 2 - OR:   block=0, nonblock=0, cont=0
  Final Result:   block=0, nonblock=0, cont=0
  Expected Result: 0
---
0000   |    0     |    0     |    0     |     0      |  B_OK  N_OK  C_OK
Input change detected: a=0, b=0, c=0, d=0
  Level 1 -

0

In [48]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_6_examples/example_19__shift_register_comparison/"
files = [
    "shift_register_comparison.sv",
    "shift_register_comparison_testbench.sv",
]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// shift_register_comparison.sv
// Demonstrates critical differences between blocking and non-blocking assignments
// in shift register implementations - side-by-side comparison

module shift_register_comparison (
    input  logic       clk,
    input  logic       reset_n,
    input  logic       shift_enable,
    input  logic       serial_in,
    output logic [7:0] parallel_out_correct,
    output logic [7:0] parallel_out_incorrect,
    output logic       serial_out_correct,
    output logic       serial_out_incorrect
);

    // ========================================
    // CORRECT SHIFT REGISTER IMPLEMENTATION
    // Using non-blocking assignments (<=)
    // ========================================
    
    logic [7:0] shift_reg_correct;
    
    // CORRECT: Non-blocking assignments create proper shift register
    always_ff @(posedge clk or negedge reset_n) begin
        if (~reset_n) begin
            shift_reg_correct <= 8'b0;
        end else if (shift_enable) begin
            // All assignments happen simultaneously at clock edge
            shift_reg_correct[7] <= serial_in;           // New bit in
            shift_reg_correct[6] <= shift_reg_correct[7]; // Bit 7 -> 6
            shift_reg_correct[5] <= shift_reg_correct[6]; // Bit 6 -> 5
            shift_reg_correct[4] <= shift_reg_correct[5]; // Bit 5 -> 4
            shift_reg_correct[3] <= shift_reg_correct[4]; // Bit 4 -> 3
            shift_reg_correct[2] <= shift_reg_correct[3]; // Bit 3 -> 2
            shift_reg_correct[1] <= shift_reg_correct[2]; // Bit 2 -> 1
            shift_reg_correct[0] <= shift_reg_correct[1]; // Bit 1 -> 0
        end
    end
    
    // Outputs for correct implementation
    assign parallel_out_correct = shift_reg_correct;
    assign serial_out_correct = shift_reg_correct[0];

    // ========================================
    // INCORRECT SHIFT REGISTER IMPLEMENTATION  
    // Using blocking assignments (=)
    // ========================================
    
    logic [7:0] shift_reg_incorrect;
    
    // INCORRECT: Blocking assignments create wrong behavior
    always_ff @(posedge clk or negedge reset_n) begin
        if (~reset_n) begin
            shift_reg_incorrect = 8'b0;
        end else if (shift_enable) begin
            // PROBLEM: Assignments execute sequentially!
            // This creates a "shift-through" effect in one clock cycle
            shift_reg_incorrect[7] = serial_in;              // Executes first
            shift_reg_incorrect[6] = shift_reg_incorrect[7]; // Uses NEW value of [7]!
            shift_reg_incorrect[5] = shift_reg_incorrect[6]; // Uses NEW value of [6]!
            shift_reg_incorrect[4] = shift_reg_incorrect[5]; // Uses NEW value of [5]!
            shift_reg_incorrect[3] = shift_reg_incorrect[4]; // Uses NEW value of [4]!
            shift_reg_incorrect[2] = shift_reg_incorrect[3]; // Uses NEW value of [3]!
            shift_reg_incorrect[1] = shift_reg_incorrect[2]; // Uses NEW value of [2]!
            shift_reg_incorrect[0] = shift_reg_incorrect[1]; // Uses NEW value of [1]!
            // Result: serial_in propagates through ALL bits in ONE clock cycle!
        end
    end
    
    // Outputs for incorrect implementation
    assign parallel_out_incorrect = shift_reg_incorrect;
    assign serial_out_incorrect = shift_reg_incorrect[0];
    
    // Alternative correct implementation using concatenation (for reference)
    logic [7:0] shift_reg_concat;
    always_ff @(posedge clk or negedge reset_n) begin
        if (~reset_n) begin
            shift_reg_concat <= 8'b0;
        end else if (shift_enable) begin
            // This is also correct - concatenation with non-blocking assignment
            shift_reg_concat <= {serial_in, shift_reg_concat[7:1]};
        end
    end
    
    // Display educational information
    initial begin
        $display("=== SHIFT REGISTER ASSIGNMENT COMPARISON ===");
        $display("CORRECT (<=):   Each bit shifts one position per clock");
        $display("INCORRECT (=):  Input propagates through ALL bits in one clock");
        $display("=> Watch how serial_in='1' behaves differently:");
        $display("   - Correct: Takes 8 clocks to reach serial_out");
        $display("   - Incorrect: Reaches serial_out in 1 clock!");
        $display("===========================================");
    end

endmodule
```

```systemverilog
// shift_register_comparison_testbench.sv
module shift_register_testbench;

    // Testbench signals
    logic       clk;
    logic       reset_n;
    logic       shift_enable;
    logic       serial_in;
    logic [7:0] parallel_out_correct;
    logic [7:0] parallel_out_incorrect;
    logic       serial_out_correct;
    logic       serial_out_incorrect;
    
    // Test pattern for serial input
    logic test_pattern [0:15] = '{1,0,1,1,0,0,1,0,1,1,1,0,0,1,1,0};
    int pattern_index = 0;
    int cycle_count = 0;
    
    // Instantiate the shift register comparison
    shift_register_comparison SHIFT_REG_INSTANCE (
        .clk(clk),
        .reset_n(reset_n),
        .shift_enable(shift_enable),
        .serial_in(serial_in),
        .parallel_out_correct(parallel_out_correct),
        .parallel_out_incorrect(parallel_out_incorrect),
        .serial_out_correct(serial_out_correct),
        .serial_out_incorrect(serial_out_incorrect)
    );
    
    // Clock generation - 10ns period
    initial begin
        clk = 0;
        forever #5 clk = ~clk;
    end
    
    // Test sequence
    initial begin
        // Dump waves for detailed analysis
        $dumpfile("shift_register_testbench.vcd");
        $dumpvars(0, shift_register_testbench);
        
        // Initialize signals
        reset_n = 0;
        shift_enable = 0;
        serial_in = 0;
        
        $display("\n=== Shift Register Assignment Comparison ===");
        $display("8-bit shift register: MSB <- serial_in ... LSB -> serial_out");
        
        // Reset sequence
        repeat(2) @(posedge clk);
        reset_n = 1;
        shift_enable = 1;
        
        $display("\n--- Test 1: Single '1' bit propagation ---");
        
        // Test 1: Send single '1' bit and observe propagation
        @(posedge clk);
        serial_in = 1;
        cycle_count++;
        $display("Cycle %0d: Input='1' -> Correct=%8b (%0d), Incorrect=%8b (%0d)", 
                 cycle_count, parallel_out_correct, parallel_out_correct, 
                 parallel_out_incorrect, parallel_out_incorrect);
        
        // Follow with zeros to see the '1' propagate
        for (int i = 0; i < 10; i++) begin
            @(posedge clk);
            serial_in = 0;
            cycle_count++;
            $display("Cycle %0d: Input='0' -> Correct=%8b (%0d), Incorrect=%8b (%0d)", 
                     cycle_count, parallel_out_correct, parallel_out_correct, 
                     parallel_out_incorrect, parallel_out_incorrect);
        end
        
        $display("\n--- Test 2: Pattern shifting ---");
        
        // Reset for pattern test
        @(posedge clk);
        reset_n = 0;
        @(posedge clk);
        reset_n = 1;
        cycle_count = 0;
        
        // Shift in test pattern
        for (int i = 0; i < 16; i++) begin
            @(posedge clk);
            serial_in = test_pattern[i];
            cycle_count++;
            
            $display("Cycle %0d: Input='%0d' -> Correct=%8b, Incorrect=%8b", 
                     cycle_count, serial_in, parallel_out_correct, parallel_out_incorrect);
            $display("         Serial Out: Correct='%0d', Incorrect='%0d'", 
                     serial_out_correct, serial_out_incorrect);
        end
        
        $display("\n--- Test 3: Demonstrating the key difference ---");
        
        // Reset and show the critical difference
        @(posedge clk);
        reset_n = 0;
        @(posedge clk);
        reset_n = 1;
        
        // Show what happens when we input '1' after reset
        $display("\nAfter reset, both registers are 00000000");
        $display("Now inputting '1'...");
        
        @(posedge clk);
        serial_in = 1;
        $display("After 1 clock with input='1':");
        $display("  CORRECT:   %8b (1 shifted to bit 7 only)", parallel_out_correct);
        $display("  INCORRECT: %8b (1 propagated through ALL bits!)", parallel_out_incorrect);
        
        @(posedge clk);
        serial_in = 0;
        $display("After 2nd clock with input='0':");
        $display("  CORRECT:   %8b (1 shifted from bit 7 to bit 6)", parallel_out_correct);
        $display("  INCORRECT: %8b (all 0s because 1 'fell out' immediately)", parallel_out_incorrect);
        
        $display("\n--- Analysis Summary ---");
        $display("BLOCKING ASSIGNMENTS (=) PROBLEM:");
        $display("- Execute sequentially within same clock cycle");
        $display("- Each assignment uses the NEW value from previous assignment");
        $display("- Creates 'shift-through' effect - data propagates through ALL stages");
        $display("- Serial input appears at serial output in SAME clock cycle!");
        $display("- NOT a real shift register - just a wire with delay");
        
        $display("\nNON-BLOCKING ASSIGNMENTS (<=) CORRECT:");
        $display("- All assignments scheduled simultaneously");
        $display("- Each assignment uses OLD values from previous clock cycle");
        $display("- Creates proper shift register behavior");
        $display("- Takes 8 clock cycles for serial input to reach serial output");
        $display("- Each bit position stores data for exactly one clock cycle");
        
        $display("\n=== Test Complete ===");
        $display("Examine VCD file to see detailed timing behavior");
        $finish;
    end
    
    // Monitor for educational purposes
    always @(posedge clk) begin
        if (reset_n && shift_enable) begin
            // Show internal register state
            $display("  Internal: Correct[7:0]=%b, Incorrect[7:0]=%b", 
                     SHIFT_REG_INSTANCE.shift_reg_correct, 
                     SHIFT_REG_INSTANCE.shift_reg_incorrect);
        end
    end

endmodule
```

Verilator Simulation Output:
=== SHIFT REGISTER ASSIGNMENT COMPARISON ===
CORRECT (<=):   Each bit shifts one position per clock
INCORRECT (=):  Input propagates through ALL bits in one clock
=> Watch how serial_in='1' behaves differently:
   - Correct: Takes 8 clocks to reach serial_out
   - Incorrect: Reaches serial_out in 1 clock!

=== Shift Register Assignment Comparison ===
8-bit shift register: MSB <- serial_in ... LSB -> serial_out

--- Test 1: Single '1' bit propagation ---
  Internal: Correct[7:0]=00000000, Incorrect[7:0]=00000000
Cycle 1: Input='1' -> Correct=00000000 (0), Incorrect=00000000 (0)
  Internal: Correct[7:0]=00000000, Incorrect[7:0]=00000000
Cycle 2: Input='0' -> Correct=10000000 (128), Incorrect=11111111 (255)
  Internal: Correct[7:0]=10000000, Incorrect[7:0]=11111111
Cycle 3: Input='0' -> Correct=01000000 (64), Incorrect=00000000 (0)
  Internal: Correct[7:0]=01000000, Incorrect[7:0]=00000000
Cycle 4: Input='0' -> Correct=00100000 (32), Incorrect=00000000 (0)
  I

0

## Race Conditions and Common Pitfalls

#### Cross-Coupled Latches
Example showing race conditions in feedback systems

In [49]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_6_examples/example_21__cross_coupled_latches/"
files = ["cross_coupled_latches.sv", "cross_coupled_latches_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// cross_coupled_latches.sv
// Cross-coupled NOR latches demonstrating race conditions in feedback systems
module cross_coupled_latches (
  input  logic set_n,    // Active-low set input
  input  logic reset_n,  // Active-low reset input
  output logic q,        // Latch output Q
  output logic q_n       // Latch output Q_bar (complement)
);

  // Cross-coupled NOR gates with propagation delays
  // These delays can cause race conditions and metastability
  
  always_comb begin
    // NOR gate 1: q_n = ~(set_n | q)
    #2 q_n = ~(set_n | q);
  end
  
  always_comb begin
    // NOR gate 2: q = ~(reset_n | q_n)  
    #2 q = ~(reset_n | q_n);
  end

  // Monitor for race conditions
  always @(set_n, reset_n) begin
    if (set_n == 0 && reset_n == 0) begin
      $display("WARNING: Race condition! Both SET and RESET active at time %0t", $time);
    end
  end

endmodule
```

```systemverilog
// cross_coupled_latches_testbench.sv
module cross_coupled_latches_testbench;

  // Testbench signals
  logic set_n, reset_n;
  logic q, q_n;
  
  // Instantiate the cross-coupled latches
  cross_coupled_latches LATCH_INSTANCE (
    .set_n(set_n),
    .reset_n(reset_n),
    .q(q),
    .q_n(q_n)
  );

  initial begin
    // Dump waves for analysis
    $dumpfile("cross_coupled_latches_testbench.vcd");
    $dumpvars(0, cross_coupled_latches_testbench);
    
    $display("Cross-Coupled Latches Race Condition Demo");
    $display("==========================================");
    
    // Initialize - both inputs inactive (high)
    set_n = 1'b1;
    reset_n = 1'b1;
    #10;
    $display("Initial state: SET_N=%b, RESET_N=%b, Q=%b, Q_N=%b", set_n, reset_n, q, q_n);
    
    // Test normal SET operation
    $display("\n--- Testing SET operation ---");
    set_n = 1'b0;  // Activate SET
    #10;
    $display("After SET: SET_N=%b, RESET_N=%b, Q=%b, Q_N=%b", set_n, reset_n, q, q_n);
    
    set_n = 1'b1;  // Deactivate SET
    #10;
    $display("SET released: SET_N=%b, RESET_N=%b, Q=%b, Q_N=%b", set_n, reset_n, q, q_n);
    
    // Test normal RESET operation
    $display("\n--- Testing RESET operation ---");
    reset_n = 1'b0;  // Activate RESET
    #10;
    $display("After RESET: SET_N=%b, RESET_N=%b, Q=%b, Q_N=%b", set_n, reset_n, q, q_n);
    
    reset_n = 1'b1;  // Deactivate RESET
    #10;
    $display("RESET released: SET_N=%b, RESET_N=%b, Q=%b, Q_N=%b", set_n, reset_n, q, q_n);
    
    // Demonstrate RACE CONDITION - both inputs active simultaneously
    $display("\n--- RACE CONDITION TEST ---");
    $display("Activating both SET and RESET simultaneously...");
    set_n = 1'b0;
    reset_n = 1'b0;
    #15;
    $display("Both active: SET_N=%b, RESET_N=%b, Q=%b, Q_N=%b", set_n, reset_n, q, q_n);
    
    // Release both at slightly different times to show unpredictable behavior
    $display("Releasing SET first...");
    set_n = 1'b1;
    #3;
    $display("SET released: SET_N=%b, RESET_N=%b, Q=%b, Q_N=%b", set_n, reset_n, q, q_n);
    
    reset_n = 1'b1;
    #10;
    $display("Both released: SET_N=%b, RESET_N=%b, Q=%b, Q_N=%b", set_n, reset_n, q, q_n);
    
    // Try the race condition again, but release RESET first this time
    $display("\n--- Second RACE CONDITION TEST ---");
    $display("Again activating both SET and RESET...");
    set_n = 1'b0;
    reset_n = 1'b0;
    #15;
    $display("Both active: SET_N=%b, RESET_N=%b, Q=%b, Q_N=%b", set_n, reset_n, q, q_n);
    
    $display("Releasing RESET first this time...");
    reset_n = 1'b1;
    #3;
    $display("RESET released: SET_N=%b, RESET_N=%b, Q=%b, Q_N=%b", set_n, reset_n, q, q_n);
    
    set_n = 1'b1;
    #10;
    $display("Both released: SET_N=%b, RESET_N=%b, Q=%b, Q_N=%b", set_n, reset_n, q, q_n);
    
    $display("\n==========================================");
    $display("Simulation complete - Check VCD for timing details");
    $display("Notice how race conditions create unpredictable states!");
    
    $finish;
  end

endmodule
```

Verilator Simulation Output:
Cross-Coupled Latches Race Condition Demo
Initial state: SET_N=1, RESET_N=1, Q=0, Q_N=0

--- Testing SET operation ---
After SET: SET_N=0, RESET_N=1, Q=0, Q_N=1
SET released: SET_N=1, RESET_N=1, Q=0, Q_N=0

--- Testing RESET operation ---
After RESET: SET_N=1, RESET_N=0, Q=1, Q_N=0
RESET released: SET_N=1, RESET_N=1, Q=0, Q_N=0

--- RACE CONDITION TEST ---
Activating both SET and RESET simultaneously...
Both active: SET_N=0, RESET_N=0, Q=1, Q_N=0
Releasing SET first...
SET released: SET_N=1, RESET_N=0, Q=1, Q_N=0
Both released: SET_N=1, RESET_N=1, Q=0, Q_N=0

--- Second RACE CONDITION TEST ---
Again activating both SET and RESET...
Both active: SET_N=0, RESET_N=0, Q=1, Q_N=0
Releasing RESET first this time...
RESET released: SET_N=0, RESET_N=1, Q=0, Q_N=1
Both released: SET_N=1, RESET_N=1, Q=0, Q_N=0

Simulation complete - Check VCD for timing details
Notice how race conditions create unpredictable states!
Process finished with return code: 0
Removing Chapt

0

#### Clock Domain Crossing
Synchronizer circuits to safely cross clock domains

In [50]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_6_examples/example_22__clock_domain_crossing/"
files = ["clock_domain_crossing.sv", "clock_domain_crossing_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// clock_domain_crossing.sv
// Demonstrates safe clock domain crossing using synchronizer circuits

module clock_domain_crossing (
  // Clock Domain A (source)
  input  logic clk_a,
  input  logic rst_a_n,
  input  logic data_a,
  
  // Clock Domain B (destination)
  input  logic clk_b,
  input  logic rst_b_n,
  output logic data_b_sync,
  
  // Unsafe direct crossing (for comparison)
  output logic data_b_unsafe
);

  // Unsafe direct crossing - DO NOT USE IN REAL DESIGNS!
  // This creates metastability issues
  assign data_b_unsafe = data_a;

  // Safe 2-FF synchronizer chain for clock domain crossing
  logic sync_ff1, sync_ff2;
  
  always_ff @(posedge clk_b or negedge rst_b_n) begin
    if (!rst_b_n) begin
      sync_ff1 <= 1'b0;
      sync_ff2 <= 1'b0;
    end else begin
      // Two-stage synchronizer to minimize metastability
      sync_ff1 <= data_a;        // First FF may go metastable
      sync_ff2 <= sync_ff1;      // Second FF resolves metastability
    end
  end
  
  assign data_b_sync = sync_ff2;

  // Metastability detection (simplified)
  logic metastable_detected;
  
  always_ff @(posedge clk_b) begin
    // Simple metastability indicator
    // In real hardware, this would be more sophisticated
    if (sync_ff1 !== 1'b0 && sync_ff1 !== 1'b1) begin
      metastable_detected <= 1'b1;
      $display("WARNING: Metastability detected at time %0t", $time);
    end else begin
      metastable_detected <= 1'b0;
    end
  end

endmodule


// Additional module: Enhanced CDC with handshake protocol
module cdc_handshake (
  // Clock Domain A
  input  logic clk_a,
  input  logic rst_a_n,
  input  logic send_req,
  input  logic [7:0] data_in,
  output logic ready,
  
  // Clock Domain B  
  input  logic clk_b,
  input  logic rst_b_n,
  output logic data_valid,
  output logic [7:0] data_out
);

  // Handshake signals
  logic req_sync, ack_sync;
  logic req_toggle, ack_toggle;
  
  // Clock Domain A: Request side
  always_ff @(posedge clk_a or negedge rst_a_n) begin
    if (!rst_a_n) begin
      req_toggle <= 1'b0;
      data_out <= 8'h00;
      ready <= 1'b1;
    end else begin
      if (send_req && ready) begin
        req_toggle <= ~req_toggle;  // Toggle to signal new data
        data_out <= data_in;        // Capture data
        ready <= 1'b0;
      end else if (req_sync == ack_sync) begin
        ready <= 1'b1;  // Acknowledge received
      end
    end
  end
  
  // Synchronize request toggle to clock domain B
  logic req_ff1, req_ff2;
  always_ff @(posedge clk_b or negedge rst_b_n) begin
    if (!rst_b_n) begin
      req_ff1 <= 1'b0;
      req_ff2 <= 1'b0;
    end else begin
      req_ff1 <= req_toggle;
      req_ff2 <= req_ff1;
    end
  end
  assign req_sync = req_ff2;
  
  // Clock Domain B: Detect new data
  logic req_sync_prev;
  always_ff @(posedge clk_b or negedge rst_b_n) begin
    if (!rst_b_n) begin
      req_sync_prev <= 1'b0;
      data_valid <= 1'b0;
      ack_toggle <= 1'b0;
    end else begin
      req_sync_prev <= req_sync;
      data_valid <= (req_sync != req_sync_prev);
      if (req_sync != req_sync_prev) begin
        ack_toggle <= req_sync;  // Echo back the request
      end
    end
  end
  
  // Synchronize acknowledge back to clock domain A
  logic ack_ff1, ack_ff2;
  always_ff @(posedge clk_a or negedge rst_a_n) begin
    if (!rst_a_n) begin
      ack_ff1 <= 1'b0;
      ack_ff2 <= 1'b0;
    end else begin
      ack_ff1 <= ack_toggle;
      ack_ff2 <= ack_ff1;
    end
  end
  assign ack_sync = ack_ff2;

endmodule
```

```systemverilog
// clock_domain_crossing_testbench.sv
module clock_domain_crossing_testbench;

  // Clock and reset signals
  logic clk_a, clk_b;
  logic rst_a_n, rst_b_n;
  
  // Test signals for basic CDC
  logic data_a;
  logic data_b_sync, data_b_unsafe;
  
  // Test signals for handshake CDC
  logic send_req, ready, data_valid;
  logic [7:0] data_in, data_out;
  
  // Instantiate basic clock domain crossing
  clock_domain_crossing BASIC_CDC (
    .clk_a(clk_a),
    .rst_a_n(rst_a_n),
    .data_a(data_a),
    .clk_b(clk_b),
    .rst_b_n(rst_b_n),
    .data_b_sync(data_b_sync),
    .data_b_unsafe(data_b_unsafe)
  );
  
  // Instantiate handshake CDC
  cdc_handshake HANDSHAKE_CDC (
    .clk_a(clk_a),
    .rst_a_n(rst_a_n),
    .send_req(send_req),
    .data_in(data_in),
    .ready(ready),
    .clk_b(clk_b),
    .rst_b_n(rst_b_n),
    .data_valid(data_valid),
    .data_out(data_out)
  );

  // Generate Clock A (100 MHz)
  initial begin
    clk_a = 0;
    forever #5 clk_a = ~clk_a;  // 10ns period
  end
  
  // Generate Clock B (67 MHz - different frequency)
  initial begin
    clk_b = 0;
    forever #7.5 clk_b = ~clk_b;  // 15ns period
  end
  
  // Reset generation
  initial begin
    rst_a_n = 0;
    rst_b_n = 0;
    #20;
    rst_a_n = 1;
    rst_b_n = 1;
  end

  // Test stimulus
  initial begin
    // Setup waveform dumping
    $dumpfile("clock_domain_crossing_testbench.vcd");
    $dumpvars(0, clock_domain_crossing_testbench);
    
    $display("Clock Domain Crossing Demonstration");
    $display("==================================");
    
    // Initialize signals
    data_a = 1'b0;
    send_req = 1'b0;
    data_in = 8'h00;
    
    // Wait for reset deassertion
    wait(rst_a_n && rst_b_n);
    #30;
    
    $display("\n--- Basic 2-FF Synchronizer Test ---");
    
    // Test basic synchronizer with various data patterns
    @(posedge clk_a);
    data_a = 1'b1;
    $display("Time %0t: Data_A changed to %b", $time, data_a);
    
    // Wait to see synchronization
    repeat(10) @(posedge clk_b);
    $display("Time %0t: Data_B_sync = %b, Data_B_unsafe = %b", 
             $time, data_b_sync, data_b_unsafe);
    
    // Change data back to 0
    @(posedge clk_a);
    data_a = 1'b0;
    $display("Time %0t: Data_A changed to %b", $time, data_a);
    
    repeat(10) @(posedge clk_b);
    $display("Time %0t: Data_B_sync = %b, Data_B_unsafe = %b", 
             $time, data_b_sync, data_b_unsafe);
    
    $display("\n--- Handshake Protocol Test ---");
    
    // Test handshake CDC with multiple data transfers
    for (int i = 1; i <= 5; i++) begin
      // Wait for ready
      wait(ready);
      @(posedge clk_a);
      
      data_in = 8'(8'h10 * i);  // Test data: 0x10, 0x20, 0x30, etc.
      send_req = 1'b1;
      $display("Time %0t: Sending data 0x%02X", $time, data_in);
      
      @(posedge clk_a);
      send_req = 1'b0;
      
      // Wait for data to be received
      wait(data_valid);
      @(posedge clk_b);
      $display("Time %0t: Received data 0x%02X", $time, data_out);
      
      // Small delay between transfers
      #50;
    end
    
    $display("\n--- Race Condition Demonstration ---");
    
    // Show what happens with rapid changes
    repeat(20) begin
      @(posedge clk_a);
      data_a = ~data_a;
      #1;  // Very short pulse - might cause issues
    end
    
    #100;
    
    $display("\n==================================");
    $display("Key observations:");
    $display("1. 2-FF synchronizer adds 2 clock cycle latency");
    $display("2. Handshake protocol ensures reliable data transfer");
    $display("3. Unsafe crossing can cause metastability");
    $display("4. Different clock frequencies require careful design");
    $display("Simulation complete - Check VCD for timing details");
    
    $finish;
  end
  
  // Monitor for demonstration purposes
  always @(posedge clk_b) begin
    if (data_b_sync !== data_b_unsafe) begin
      $display("Time %0t: NOTICE - Sync vs Unsafe difference: sync=%b, unsafe=%b", 
               $time, data_b_sync, data_b_unsafe);
    end
  end

endmodule
```

Verilator Simulation Output:
Clock Domain Crossing Demonstration

--- Basic 2-FF Synchronizer Test ---
Time 55: Data_A changed to 1
Time 56: NOTICE - Sync vs Unsafe difference: sync=0, unsafe=1
Time 72: NOTICE - Sync vs Unsafe difference: sync=0, unsafe=1
Time 200: Data_B_sync = 1, Data_B_unsafe = 1
Time 205: Data_A changed to 0
Time 216: NOTICE - Sync vs Unsafe difference: sync=1, unsafe=0
Time 232: NOTICE - Sync vs Unsafe difference: sync=1, unsafe=0
Time 360: Data_B_sync = 0, Data_B_unsafe = 0

--- Handshake Protocol Test ---
Time 365: Sending data 0x10
Time 424: Received data 0x10
Time 475: Sending data 0x20
Time 536: Received data 0x20
Time 595: Sending data 0x30
Time 648: Received data 0x30
Time 705: Sending data 0x40
Time 760: Received data 0x40
Time 815: Sending data 0x50
Time 872: Received data 0x50

--- Race Condition Demonstration ---
Time 952: NOTICE - Sync vs Unsafe difference: sync=0, unsafe=1
Time 968: NOTICE - Sync vs Unsafe difference: sync=0, unsafe=1
Time 984: NOTICE

0

#### Multiple Driver Detection
Examples of inadvertent multiple drivers and resolution

In [51]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_6_examples/example_23__multiple_driver_detection/"
files = [
    "multiple_driver_detection.sv",
    "multiple_driver_detection_testbench.sv",
]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// multiple_driver_detection.sv
// Demonstrates multiple driver scenarios and resolution methods
// NOTE: This example simulates multiple driver problems without actually 
// creating them (to avoid compiler errors), but shows the concepts

module multiple_driver_detection (
  input  logic clk,
  input  logic rst_n,
  input  logic enable_a,
  input  logic enable_b,
  input  logic data_a,
  input  logic data_b,
  output logic output_contested,    // Simulates contested behavior
  output logic output_resolved,     // Properly resolved (GOOD)
  output logic output_tristate      // Using tri-state logic (GOOD)
);

  // BAD EXAMPLE: Simulated multiple driver behavior
  // In real code, this would be separate always blocks causing conflicts
  logic driver_a_active, driver_b_active;
  
  always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
      driver_a_active <= 1'b0;
    end else if (enable_a) begin
      driver_a_active <= data_a;
    end
  end
  
  always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
      driver_b_active <= 1'b0;
    end else if (enable_b) begin
      driver_b_active <= data_b;
    end
  end
  
  // PROBLEMATIC: This simulates contested behavior without actual multiple drivers
  // In real hardware, this would be multiple always blocks (which causes warnings)
  always_comb begin
    if (enable_a && driver_a_active && !(enable_b && driver_b_active)) begin
      output_contested = data_a;  // Driver A wins when only A active
    end else if (enable_b && driver_b_active && !(enable_a && driver_a_active)) begin
      output_contested = data_b;  // Driver B wins when only B active
    end else if (enable_a && driver_a_active && enable_b && driver_b_active) begin
      output_contested = 1'bx;    // Simulate conflict with X
    end else begin
      output_contested = 1'b0;    // Default value
    end
  end

  // GOOD EXAMPLE: Properly resolved using priority logic
  always_comb begin
    if (enable_a && driver_a_active) begin
      output_resolved = data_a;     // Driver A has priority
    end else if (enable_b && driver_b_active) begin
      output_resolved = data_b;     // Driver B is secondary
    end else begin
      output_resolved = 1'b0;       // Default value
    end
  end

  // GOOD EXAMPLE: Using tri-state logic
  logic tri_a, tri_b;
  
  assign tri_a = (enable_a && driver_a_active) ? data_a : 1'bz;
  assign tri_b = (enable_b && driver_b_active) ? data_b : 1'bz;
  assign output_tristate = tri_a | tri_b;  // Wired-OR resolution

endmodule


// Additional module showing bus contention scenarios
module bus_contention_demo (
  input  logic clk,
  input  logic rst_n,
  input  logic [1:0] select,
  input  logic [7:0] data_source_0,
  input  logic [7:0] data_source_1,
  input  logic [7:0] data_source_2,
  output logic [7:0] bus_contested,
  output logic [7:0] bus_resolved
);

  /* 
   * EDUCATIONAL NOTE: 
   * In real problematic code, bus_contested would be driven by multiple
   * always blocks like this (which creates compiler warnings):
   * 
   * always_comb begin
   *   if (select == 2'b00) bus_contested = data_source_0;
   *   else bus_contested = 8'hzz;
   * end
   * 
   * always_comb begin  // <-- This creates multiple drivers!
   *   if (select == 2'b01) bus_contested = data_source_1;
   *   else bus_contested = 8'hzz;
   * end
   * 
   * We simulate this behavior below without actual multiple drivers
   */

  // BAD: Simulated multiple drivers on bus (demonstrates contention)
  always_comb begin
    if (select == 2'b00) begin
      bus_contested = data_source_0;
    end else if (select == 2'b01) begin
      bus_contested = data_source_1;
    end else if (select == 2'b10) begin
      bus_contested = data_source_2;
    end else if (select == 2'b11) begin
      // Simulate contention with invalid select
      bus_contested = 8'hxx;  // Show X to represent bus fight
    end else begin
      bus_contested = 8'hzz;
    end
  end

  // GOOD: Single driver with proper multiplexing
  always_comb begin
    case (select)
      2'b00:   bus_resolved = data_source_0;
      2'b01:   bus_resolved = data_source_1;
      2'b10:   bus_resolved = data_source_2;
      default: bus_resolved = 8'h00;
    endcase
  end

  // Monitor for bus contention
  always @(bus_contested) begin
    if (^bus_contested === 1'bx) begin
      $display("WARNING: Bus contention detected at time %0t - value is X", $time);
    end
  end

endmodule
```

```systemverilog
// multiple_driver_detection_testbench.sv
module multiple_driver_detection_testbench;

  // Clock and reset
  logic clk, rst_n;
  
  // Test signals for main module
  logic enable_a, enable_b;
  logic data_a, data_b;
  logic output_contested, output_resolved, output_tristate;
  
  // Test signals for bus contention demo
  logic [1:0] select;
  logic [7:0] data_source_0, data_source_1, data_source_2;
  logic [7:0] bus_contested, bus_resolved;
  
  // Instantiate main multiple driver detection module
  multiple_driver_detection MAIN_DUT (
    .clk(clk),
    .rst_n(rst_n),
    .enable_a(enable_a),
    .enable_b(enable_b),
    .data_a(data_a),
    .data_b(data_b),
    .output_contested(output_contested),
    .output_resolved(output_resolved),
    .output_tristate(output_tristate)
  );
  
  // Instantiate bus contention demo
  bus_contention_demo BUS_DEMO (
    .clk(clk),
    .rst_n(rst_n),
    .select(select),
    .data_source_0(data_source_0),
    .data_source_1(data_source_1),
    .data_source_2(data_source_2),
    .bus_contested(bus_contested),
    .bus_resolved(bus_resolved)
  );

  // Generate clock (50 MHz)
  initial begin
    clk = 0;
    forever #10 clk = ~clk;  // 20ns period
  end
  
  // Reset generation
  initial begin
    rst_n = 0;
    #25;
    rst_n = 1;
  end

  // Test stimulus
  initial begin
    // Setup waveform dumping
    $dumpfile("multiple_driver_detection_testbench.vcd");
    $dumpvars(0, multiple_driver_detection_testbench);
    
    $display("Multiple Driver Detection Demonstration");
    $display("======================================");
    
    // Initialize signals
    enable_a = 0;
    enable_b = 0;
    data_a = 0;
    data_b = 0;
    select = 2'b00;
    data_source_0 = 8'hAA;
    data_source_1 = 8'hBB;
    data_source_2 = 8'hCC;
    
    // Wait for reset
    wait(rst_n);
    repeat(2) @(posedge clk);
    
    $display("\n--- Single Driver Test (No Conflict) ---");
    
    // Test with only one driver active
    data_a = 1'b1;
    enable_a = 1'b1;
    repeat(3) @(posedge clk);
    
    $display("Time %0t: Only A enabled - Contested=%b, Resolved=%b, Tristate=%b", 
             $time, output_contested, output_resolved, output_tristate);
    
    enable_a = 1'b0;
    data_b = 1'b1;
    enable_b = 1'b1;
    repeat(3) @(posedge clk);
    
    $display("Time %0t: Only B enabled - Contested=%b, Resolved=%b, Tristate=%b", 
             $time, output_contested, output_resolved, output_tristate);
    
    $display("\n--- Multiple Driver Conflict Test ---");
    
    // Create the problematic scenario: both drivers active
    data_a = 1'b1;
    data_b = 1'b0;
    enable_a = 1'b1;
    enable_b = 1'b1;
    
    repeat(3) @(posedge clk);
    
    $display("Time %0t: BOTH drivers active!", $time);
    $display("  Data A = %b, Data B = %b", data_a, data_b);
    $display("  Contested output = %b (undefined behavior!)", output_contested);
    $display("  Resolved output = %b (A has priority)", output_resolved);
    $display("  Tristate output = %b (wired-OR)", output_tristate);
    
    // Try opposite data values
    data_a = 1'b0;
    data_b = 1'b1;
    repeat(3) @(posedge clk);
    
    $display("Time %0t: Opposite data values", $time);
    $display("  Data A = %b, Data B = %b", data_a, data_b);
    $display("  Contested output = %b", output_contested);
    $display("  Resolved output = %b (A still has priority)", output_resolved);
    $display("  Tristate output = %b", output_tristate);
    
    $display("\n--- Bus Contention Test ---");
    
    // Test proper bus operation
    select = 2'b00;
    repeat(2) @(posedge clk);
    $display("Time %0t: Select=00, Bus_resolved=0x%02X (should be 0xAA)", 
             $time, bus_resolved);
    
    select = 2'b01;
    repeat(2) @(posedge clk);
    $display("Time %0t: Select=01, Bus_resolved=0x%02X (should be 0xBB)", 
             $time, bus_resolved);
    
    select = 2'b10;
    repeat(2) @(posedge clk);
    $display("Time %0t: Select=10, Bus_resolved=0x%02X (should be 0xCC)", 
             $time, bus_resolved);
    
    // Test invalid select (potential contention in bad design)
    select = 2'b11;
    repeat(2) @(posedge clk);
    $display("Time %0t: Select=11, Bus_resolved=0x%02X (default case)", 
             $time, bus_resolved);
    
    $display("\n--- X-State Propagation Test ---");
    
    // Force X values to show propagation
    force MAIN_DUT.driver_a_active = 1'bx;
    repeat(2) @(posedge clk);
    $display("Time %0t: Forced X in driver - outputs may show X", $time);
    release MAIN_DUT.driver_a_active;
    
    repeat(5) @(posedge clk);
    
    $display("\n======================================");
    $display("Key Observations:");
    $display("1. Multiple drivers create undefined behavior");
    $display("2. Proper resolution uses priority or muxing");
    $display("3. Tri-state logic allows multiple drivers with Z");
    $display("4. Bus contention can cause X propagation");
    $display("5. Always use single-driver design patterns");
    $display("Simulation complete - Check VCD for detailed timing");
    
    $finish;
  end
  
  // Monitor for X states
  always @(output_contested, bus_contested) begin
    if (^output_contested === 1'bx) begin
      $display("ALERT: X detected on contested output at time %0t", $time);
    end
    if (^bus_contested === 1'bx) begin
      $display("ALERT: X detected on contested bus at time %0t", $time);
    end
  end
  
  // Monitor driver conflicts
  always @(posedge clk) begin
    if (enable_a && enable_b && MAIN_DUT.driver_a_active && MAIN_DUT.driver_b_active) begin
      $display("CONFLICT: Both drivers active at time %0t", $time);
    end
  end

endmodule
```

Verilator Simulation Output:
Multiple Driver Detection Demonstration

--- Single Driver Test (No Conflict) ---
Time 110: Only A enabled - Contested=1, Resolved=1, Tristate=1
Time 170: Only B enabled - Contested=1, Resolved=1, Tristate=1

--- Multiple Driver Conflict Test ---
CONFLICT: Both drivers active at time 170
Time 230: BOTH drivers active!
  Data A = 1, Data B = 0
  Contested output = 1 (undefined behavior!)
  Resolved output = 1 (A has priority)
  Tristate output = 1 (wired-OR)
Time 290: Opposite data values
  Data A = 0, Data B = 1
  Contested output = 1
  Resolved output = 1 (A still has priority)
  Tristate output = 1

--- Bus Contention Test ---
Time 330: Select=00, Bus_resolved=0xaa (should be 0xAA)
Time 370: Select=01, Bus_resolved=0xbb (should be 0xBB)
Time 410: Select=10, Bus_resolved=0xcc (should be 0xCC)
Time 450: Select=11, Bus_resolved=0x00 (default case)

--- X-State Propagation Test ---
Time 490: Forced X in driver - outputs may show X

Key Observations:
1. Multip

0

## Process Control and Advanced Topics

#### Clock Divider Circuit
Frequency divider with even and odd division ratios

#### Metastability Synchronizer
Two-flip-flop synchronizer for asynchronous signal capture

#### Parameterized Delay Line
Configurable delay element using generate blocks

#### Handshake Protocol Controller
Ready/valid handshaking for data transfer protocols

#### Reset Synchronizer
Circuit for clean reset release across clock domains